# Primer EDA: Limpieza y Preparación de Datos

Se divide el Análisis Exploratorio de Datos en dos archivos con el objetivo de preparar el conjunto de datos correctamente antes del modelo. Esta división en dos etapas también tiene el objetivo de evitar la sobrecarga del notebook y reducir el uso de memoria y recursos del sistema.

La primera etapa se centra en la limpieza y preparación de los datos. Esto incluye la identificación de valores faltantes, la detección de outliers y la aplicación de transformaciones necesarias. Se prepara el conjunto de datos para un análisis más profundo.

### Importación de bibliotecas necesarias para análisis de datos y visualización

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import sys
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt

### Importación de funciones desde el módulo de procesamiento de datos

In [2]:
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from data_processing import (
    resumen_columnas,
    obtener_valores_unicos,
    analizar_precio_viviendas_por_variable,
    calcular_correlaciones,
    label_encoding,
    codificacion_ponderada,
    calcular_porcentaje_coincidencias,
    visualizar_correlaciones,
    aplicar_codificacion_ordinal_especifica,
    visualizar_correlaciones_grandes,
    rellenar_atributos_sotano,
    codificacion_loo,
    graficar_conteo_clases,
    crear_histograma,
    boxplot_train_test,
    distribucion_target_con_variable,
    distribucion_target_con_variable_vertical,
    describe_train_test,
)

### Carga del DataFrame unido

In [3]:
df= pd.read_pickle('../data/Inmobiliaria_Horizonte.pkl')

# Exploración de la Variable Objetivo

1. [SalePrice](../docs/descripcion_variables.md#variable-saleprice): Precio de venta de la propiedad en dólares. Esta es la variable objetivo que se intenta predecir.

Al analizar los valores nulos en la columna **SalePrice** de los conjuntos de entrenamiento y prueba, se observa que el conjunto de prueba está vacío. Esto implica que, al construir el modelo, no será posible utilizar el conjunto de prueba para evaluar las métricas de rendimiento. Por lo tanto, el modelo de regresión deberá desarrollarse exclusivamente utilizando los datos del conjunto de entrenamiento.

In [4]:
valores_nulos_train = df[df['Dataset'] == 'train']['SalePrice'].isna().sum()
valores_nulos_test = df[df['Dataset'] == 'test']['SalePrice'].isna().sum()

print("Valores nulos en 'SalePrice' en train:", valores_nulos_train)
print("Valores nulos en 'SalePrice' en test:", valores_nulos_test)

Valores nulos en 'SalePrice' en train: 0
Valores nulos en 'SalePrice' en test: 1459


Se aplica la transformación logarítmica para graficar, solo se hace a train porque test tiene todos sus valores como NaN

In [5]:
df['SalePrice_log'] = (df['SalePrice'] + 1).apply(np.log)

print(df[['SalePrice', 'SalePrice_log']].head())

   SalePrice  SalePrice_log
0   208500.0      12.247699
1   181500.0      12.109016
2   223500.0      12.317171
3   140000.0      11.849405
4   250000.0      12.429220


In [6]:
df["SalePrice"].describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

In [7]:
df["SalePrice_log"].describe()

count    1460.000000
mean       12.024057
std         0.399449
min        10.460271
25%        11.775105
50%        12.001512
75%        12.273736
max        13.534474
Name: SalePrice_log, dtype: float64

# Histograma de la target sin logaritmo

Este histograma representa la distribución de los precios de venta en el conjunto de entrenamiento.

La mayor parte de las observaciones se encuentran entre los 100k y 200k, donde la frecuencia alcanza su punto máximo (cerca de 500). La distribución tiene una cola alargada hacia la derecha, indicando la presencia de precios más altos que son menos frecuentes. Esto sugiere una distribución sesgada a la derecha, lo cual es común en precios de bienes inmuebles.

- **Media (rojo)**: Se encuentra a la derecha de la mediana, esto confirma el sesgo positivo (derecha) en los datos, causado por algunos precios de venta elevados. La media se ve afectada por estos valores altos y es un poco mayor que la mediana.

- **Mediana (verde)**: Representa el valor central de la distribución y está situada en la zona de mayor frecuencia (en torno a los precios entre 100k y 200k).

La mayoría de los precios de venta se concentran en el rango de precios accesibles, con algunos valores atípicos altos que incrementan la media.

In [8]:
media_train_log = np.mean(df['SalePrice_log'])
mediana_train_log = np.median(df[df["Dataset"] == "train"]['SalePrice_log']) # La mediana se ve afectada con los nan
media_train = np.mean(df['SalePrice'])
mediana_train = np.median(df[df["Dataset"] == "train"]['SalePrice']) # La mediana se ve afectada con los nan

fig = sp.make_subplots(rows=1, cols=2, shared_yaxes=False, horizontal_spacing=0.02,
                       subplot_titles=('Distribución de SalePrice', 'Distribución de SalePrice_log'))
# Gráfico de SalePrice
fig.add_trace(go.Histogram(x=df['SalePrice'], opacity=0.7, name='SalePrice'), row=1, col=1)
fig.add_vline(x=media_train, line_dash="dash", line_color="red", name="Media SalePrice", row=1, col=1)
fig.add_vline(x=mediana_train, line_dash="dash", line_color="green", name="Mediana SalePrice", row=1, col=1)

# Gráfico de SalePrice_log
fig.add_trace(go.Histogram(x=df['SalePrice_log'], opacity=0.7, name='SalePrice_log'), row=1, col=2)
fig.add_vline(x=media_train_log, line_dash="dash", line_color="red", name="Media SalePrice_log", row=1, col=2)
fig.add_vline(x=mediana_train_log, line_dash="dash", line_color="green", name="Mediana SalePrice_log", row=1, col=2)

# Actualizar el layout
fig.update_layout(
    title="Distribución de SalePrice y SalePrice_log en el Conjunto de Entrenamiento",
    xaxis_title='SalePrice',
    yaxis_title='Frecuencia',
    title_x=0.5,
    height=500,
    width=1300,
)

# fig.show()

### Gráfico de caja de la target sin logaritmo

Se representa mediante un Box Plot la distribución de los precios de venta del conjunto de entrenamiento.

- **Rango Intercuartílico (IQR)**: El rango intercuartílico va de aproximadamente 129.95k a 224k.

- **Mediana (Q2)**: El valor central de la distribución está alrededor de 163k, esto indica que el 50% de las casas tienen un precio de venta entre 129.95k y 224k.

- **Valores atípicos**: Se consideran outliers a los valores a partir de 340k. 

- **Lower Fence**: El límite inferior está en 34.9k, la casa más barata del conjunto de los datos.

- **Upper Fence**: El límite superior está en 340k, cualquier valor por encima de este se considera un valor atípico.

- **Mínimo (min)**: El precio mínimo es de 34.9k.

- **Máximo (max)**: El precio máximo llega a 755k.

La mayoría de los precios de venta están concentrados entre 129.95k y 224k, con algunos valores altos que son atípicos y que exceden el límite superior de 340k, alcanzando hasta 755k. Estos valores altos pueden distorsionar los análisis estadísticos si no se tratan adecuadamente.

In [ ]:
fig = sp.make_subplots(rows=2, cols=1, subplot_titles=['Box Plot de SalePrice', 'Box Plot de SalePrice Log'])

# Gráfico de caja para SalePrice (eje x para SalePrice y eje y para categorías)
fig.add_trace(go.Box(
    x=df[df["Dataset"] == "train"]['SalePrice'],  # Usamos 'x' para SalePrice
    name='SalePrice',
), row=1, col=1)

# Gráfico de caja para SalePrice_log (eje x para SalePrice_log y eje y para categorías)
fig.add_trace(go.Box(
    x=df[df["Dataset"] == "train"]['SalePrice_log'],  # Usamos 'x' para SalePrice_log
    name='SalePrice Log',
), row=2, col=1)

fig.update_layout(
    title="Box Plots de SalePrice y SalePrice Log en el Conjunto de Entrenamiento",
    title_x=0.5,
    xaxis_title='SalePrice',  
    yaxis_title='Distribución',  
    xaxis2_title='SalePrice Log', 
    height=500
)

# fig.show()


In [10]:
df.drop(columns=['SalePrice_log'], inplace=True)

# Correlaciones

En esta sección, se calculan las correlaciones entre las columnas numéricas del DataFrame para identificar relaciones significativas entre variables. Este análisis ayudará a tomar decisiones informadas sobre la imputación de valores faltantes y el tratamiento de las columnas, permitiendo una mejor comprensión de la estructura de los datos y sus interacciones.

Se mantiene la variable objetivo para facilitar la evaluación de la importancia de las columnas en relación con ella en etapas posteriores.

In [11]:
columnas_numericas = df.select_dtypes(include=['int64', 'int32', 'float64']).columns.tolist()

resultado_correlaciones = calcular_correlaciones(df, columnas_numericas)
resultado_correlaciones

,Columna_1,Columna_2,Correlación_Pearson,Correlación_Spearman,Correlación_Kendall
23,1stFlrSF,GrLivArea,0.562538,0.492163,0.410712
24,1stFlrSF,SalePrice,0.605852,0.575408,0.411556
28,2ndFlrSF,TotRmsAbvGrd,0.584586,0.554216,0.464292
27,2ndFlrSF,BedroomAbvGr,0.503506,0.500984,0.434331
26,2ndFlrSF,HalfBath,0.611362,0.619286,0.555538
25,2ndFlrSF,GrLivArea,0.655085,0.604574,0.488212
38,BedroomAbvGr,TotRmsAbvGrd,0.669737,0.663443,0.592821
20,BsmtFinSF1,BsmtFullBath,0.638847,0.667057,0.565482
19,BsmtFinSF1,TotalBsmtSF,0.536467,0.426810,0.324580
18,BsmtFinSF1,BsmtUnfSF,-0.477404,-0.547368,-0.388988


# División de Variables en Conjuntos

En esta sección, se agrupan las 79 variables explicativas del DataFrame en varios conjuntos con el fin de facilitar su análisis. Estas variables describen los aspectos de las viviendas residenciales en Ames, Iowa.

En esta división se trata de abarcar las correlaciones de arriba, para identificar patrones y relaciones entre las variables, además de relacionarlas con la variable objetivo `SalePrice`. Al organizar las variables en subconjuntos temáticos, se mejora la eficacia en la interpretación de los datos y en la toma de decisiones, optimizando así la aplicación de técnicas de análisis y modelado. Las correlaciones que no se logren abarcar en estos conjuntos se especificarán en el EDA 2, por si dan otra perspectiva de cómo tratar esas variables.

No se incluyen las columnas `Id` y `Dataset` ya que no aportan información para el modelo.

1. Características Generales del **Terreno**

2. Características del Entorno y **Vecindario**

3. Características del **Edificio** y Estilo

4. Características del **Techo** y Exterior

5. Características del **Sótano**

6. Características del **Interior** de la Vivienda

7. Características de las **Habitaciones** y Funcionalidad

8. Características de las Áreas de Entretenimiento y **Exteriores**

9. Características Misceláneas y de **Venta**

> La explicación de cada variable y sus valores está disponible en el archivo de documentación. Al hacer clic en cada variable, podrás acceder a su descripción detallada.

### 1. **Características Generales del Terreno**
> **Objetivo**: Analizar las características físicas del terreno para evaluar cómo impactan en el valor de la propiedad.

2. [MSSubClass](../docs/descripcion_variables.md#variable-mssubclass): Clase del edificio.
3. [MSZoning](../docs/descripcion_variables.md#variable-mszoning): Clasificación de zonificación general.
4. [LotFrontage](../docs/descripcion_variables.md#variable-lotfrontage): Pies lineales de calle conectados a la propiedad.
5. [LotArea](../docs/descripcion_variables.md#variable-lotarea): Tamaño del lote en pies cuadrados.
6. [Street](../docs/descripcion_variables.md#variable-street): Tipo de acceso por carretera.
7. [Alley](../docs/descripcion_variables.md#variable-alley): Tipo de acceso por callejón.
8. [LotShape](../docs/descripcion_variables.md#variable-lotshape): Forma general de la propiedad.
9. [LandContour](../docs/descripcion_variables.md#variable-landcontour): Planicidad de la propiedad.
10. [Utilities](../docs/descripcion_variables.md#variable-utilities): Tipo de servicios disponibles.
11. [LotConfig](../docs/descripcion_variables.md#variable-lotconfig): Configuración del lote.
12. [LandSlope](../docs/descripcion_variables.md#variable-landslope): Pendiente de la propiedad.

In [12]:
características_terreno = ["MSSubClass", "MSZoning", "LotFrontage", "LotArea", "Street", "Alley", 
                           "LotShape", "LandContour", "Utilities", "LotConfig", "LandSlope"]

df[["SalePrice"] + características_terreno].head()

,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope
0,208500.0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl
1,181500.0,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl
2,223500.0,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl
3,140000.0,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl
4,250000.0,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl


La variable `MSSubClass` asigna un código numérico a diferentes tipos de viviendas; sin embargo, estos números representan categorías sin un orden específico. Por eso, se convierte en tipo `object` antes de representarla en el resumen.

In [13]:
df['MSSubClass'] = df['MSSubClass'].astype('object')

> **Nota:** A partir de ahora, todos los conjuntos temáticos se representarán a partir de la función **resumen_columnas**, que proporcionará una visión general de los datos. Esta función incluirá información clave sobre cada columna, como el número de valores únicos, la cantidad y porcentaje de valores faltantes (NaN) en los conjuntos de entrenamiento y prueba, así como una breve descripción de los datos. Esto facilitará la comprensión de la calidad y la distribución de los datos en cada conjunto, permitiendo identificar rápidamente columnas que podrían requerir tratamiento.

### Resumen 

Hay varias columnas que contienen NaN en train, test o ambas. Se tratarán adecuadamente a partir de train para no causar problemas en el modelo de predicción. 

- **MSSubClass**: Tiene 16 valores únicos; al pasarlo a categórico, se comprueba que hay uno que sólo está presente en train, lo que podría afectar la generalización del modelo.

- **Utilities**: Deberán tratarse tanto los NaN como el valor único que está en train y no en test, reagrupándolo con otra categoría para asegurar que el modelo no encuentre problemas durante la evaluación en test.

In [14]:
resumen = resumen_columnas(df, características_terreno)
resumen

,Tipo de Dato,Nº Valores Únicos,Nº de NaN,Porcentaje NaN Train,Porcentaje NaN Test,Valores Únicos Train,Valores Únicos Test
MSSubClass,object,16,0,0.000000,0.000000,[],[150]
MSZoning,object,5,4,0.000000,0.274160,[],[]
LotFrontage,float64,128,486,17.739726,15.558602,[],[]
LotArea,int64,1951,0,0.000000,0.000000,[],[]
Street,object,2,0,0.000000,0.000000,[],[]
Alley,object,2,2721,93.767123,92.666210,[],[]
LotShape,object,4,0,0.000000,0.000000,[],[]
LandContour,object,4,0,0.000000,0.000000,[],[]
Utilities,object,2,2,0.000000,0.137080,[NoSeWa],[]
LotConfig,object,5,0,0.000000,0.000000,[],[]


In [15]:
valores_unicos = obtener_valores_unicos(df, características_terreno)

for key, value in valores_unicos.items():
    print(f"'{key}': {value},")

'MSSubClass': [20, 30, 40, 45, 50, 60, 70, 75, 80, 85, 90, 120, 150, 160, 180, 190],
'MSZoning': ['C (all)', 'FV', 'RH', 'RL', 'RM'],
'LotFrontage': [21.0, 22.0, 24.0, 25.0, 26.0, 28.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 128.0, 129.0, 130.0, 131.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 140.0, 141.0, 144.0, 149.0, 150.0, 152.0, 153.0, 155.0, 160.0, 168.0, 174.0, 182.0, 195.0, 200.0, 313.0],
'LotArea': [1300, 1470, 1476,

2. [MSSubClass](../docs/descripcion_variables.md#variable-mssubclass): Clase del edificio.

In [16]:
resultado = analizar_precio_viviendas_por_variable(df, 'MSSubClass')
resultado

,MSSubClass,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
5,60,239948.501672,575,19.698527,299,276
11,120,200779.080460,182,6.235012,87,95
7,75,192437.500000,23,0.787941,16,7
0,20,185224.811567,1079,36.964714,536,543
8,80,169736.551724,118,4.042480,58,60
6,70,166772.416667,128,4.385063,60,68
2,40,156125.000000,6,0.205550,4,2
9,85,147810.000000,48,1.644399,20,28
4,50,143302.972222,287,9.832134,144,143
13,160,138647.380952,128,4.385063,63,65


La variable `MSSubClass` tiene un valor faltante codificado como `150` en `test`, el cual corresponde a "Vivienda bifamiliar de dos pisos". En lugar de reemplazar este valor por la moda, se opta por sustituirlo con un valor similar, `85`, que corresponde a "Vivienda multifamiliar de dos pisos". Para mantener la consistencia en la clasificación de las viviendas en el conjunto de datos.

In [17]:
df['MSSubClass'].replace(150, 85, inplace=True)

`MSSubClass` presenta una baja correlación con la variable objetivo en su forma original. Para mejorar el análisis y la interpretación de los datos, decido agrupar estos códigos en categorías similares. Esta agrupación facilitará el análisis, permitirá identificar patrones más claramente y mejorará la calidad de las visualizaciones. Al tratar `MSSubClass` como una variable categórica, se podrán obtener mejores insights sobre el impacto de diferentes tipos de viviendas en el precio.

In [18]:
correlacion = df['SalePrice'].corr(df['MSSubClass'])
print(f'Correlación entre SalePrice y MSSubClass: {correlacion}')

Correlación entre SalePrice y MSSubClass: -0.08428413512659517


Se aplica una clasificación a la variable MSSubClass utilizando un mapeo que reorganiza los valores numéricos originales en categorías más interpretables basadas en las características funcionales de las propiedades. Este enfoque mejora el desempeño del modelo.

In [19]:
mapeo = {
    60: 'Unifamiliar',
    20: 'Bifamiliar',
    70: 'Unifamiliar',
    50: 'Bifamiliar',
    190: 'Unifamiliar (PUD)',
    45: 'Misceláneo',
    90: 'Unifamiliar (PUD)',
    120: 'Proximidad a Parques',
    30: 'Condominio',
    85: 'Multifamiliar',
    80: 'Unifamiliar',
    160: 'Unifamiliar',
    75: 'Unifamiliar (PUD)',
    180: 'Unifamiliar',
    40: 'Multifamiliar'
}

df['MSSubClass'] = df['MSSubClass'].map(mapeo)


### Diagrama de Caja de la Distribución de los Precios de las Casas por Clase del Edificio en el Conjunto de Entrenamiento

Los datos utilizados para generar el boxplot provienen del conjunto de entrenamiento. Aunque el DataFrame que se está utilizando es el general, el gráfico solo representa los precios de venta disponibles, que corresponden únicamente al conjunto de entrenamiento.

In [20]:
# distribucion_target_con_variable(df, 'SalePrice', 'MSSubClass', title='Impacto de la Clase del edificio en el Precio de Venta')

MSSubClass la distribución de esta variable categórica es similar en ambos conjuntos. tiene distribuciones consistentes en ambas particiones

In [21]:
resultado = analizar_precio_viviendas_por_variable(df, 'MSSubClass')
resultado

,MSSubClass,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
5,Unifamiliar,206843.779592,966,33.093525,490,476
4,Proximidad a Parques,200779.080460,182,6.235012,87,95
0,Bifamiliar,176347.245588,1366,46.796848,680,686
3,Multifamiliar,149195.833333,55,1.884207,24,31
6,Unifamiliar (PUD),141954.448980,193,6.611853,98,95
2,Misceláneo,108591.666667,18,0.616650,12,6
1,Condominio,95829.724638,139,4.761905,69,70


In [22]:
# graficar_conteo_clases(df, columna='MSSubClass')

In [23]:
# No aplicar codificación LOO, empeora

codificacion_ponderada(df, 'MSSubClass', 'SalePrice')

La nueva representación de `MSSubClass` captura mejor la relación entre el tipo de vivienda y el precio de venta. A medida que cambia la clase de la vivienda, también tiende a cambiar el precio de venta, aunque la relación todavía es relativamente débil.

In [24]:
correlacion = df['SalePrice'].corr(df['MSSubClass'])
print(f'Correlación entre SalePrice y MSSubClass: {correlacion}')

Correlación entre SalePrice y MSSubClass: 0.1773898547641043


3. [MSZoning](../docs/descripcion_variables.md#variable-mszoning): Clasificación de zonificación general.

In [25]:
resultado = analizar_precio_viviendas_por_variable(df, 'MSZoning')
resultado

,MSZoning,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
1,FV,214014.061538,139,4.761905,65,74
3,RL,191004.994787,2265,77.595067,1151,1114
2,RH,131558.375000,26,0.890716,16,10
4,RM,126316.830275,460,15.758822,218,242
0,C (all),74528.000000,25,0.856458,10,15


Al haber solo 4 valores faltantes en la columna `MSZoning`, aplico la moda.

In [26]:
num_nan = df['MSZoning'].isna().sum()
print(num_nan)

4


In [27]:
df[df['MSZoning'].isna()][['Dataset', 'MSZoning']]

,Dataset,MSZoning
1915,test,NaN
2216,test,NaN
2250,test,NaN
2904,test,NaN


Imputar valores nulos de MSZoning en el df usando la moda calculada del conjunto de entrenamiento

In [28]:
train_mszoning_moda = df[df['Dataset'] == 'train']['MSZoning'].mode()[0]
df['MSZoning'].fillna(train_mszoning_moda, inplace=True)
train_mszoning_moda

'RL'

In [29]:
mapeo= {
        "RL": "Zona Baja Densidad",
        "RM": "Zona Media Densidad",
        "C (all)": "Zona Comercial",
        "FV": "Zona Planificada",
        "RH": "Zona Alta Densidad"
    }
df['MSZoning'] = df['MSZoning'].map(mapeo)

In [30]:
resultado = analizar_precio_viviendas_por_variable(df, 'MSZoning')
resultado

,MSZoning,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
4,Zona Planificada,214014.061538,139,4.761905,65,74
1,Zona Baja Densidad,191004.994787,2269,77.732100,1151,1118
0,Zona Alta Densidad,131558.375000,26,0.890716,16,10
3,Zona Media Densidad,126316.830275,460,15.758822,218,242
2,Zona Comercial,74528.000000,25,0.856458,10,15


In [31]:
# cambiar esta grafica por boxplot
# graficar_conteo_clases(df, columna='MSZoning')

La correlación entre `SalePrice` y `MSZoning` es de **0.2402**. Este valor indica una relación positiva moderada entre el tipo de zona y el precio de venta. A medida que cambian las categorías de `MSZoning`, el precio de venta tiende a aumentar, lo que sugiere que ciertos tipos de zonificación pueden estar asociados con precios más altos en el mercado inmobiliario. Sin embargo, dado que la correlación no es muy alta, también implica que otros factores podrían influir más en el precio de venta.

In [32]:
# No aplicar codificación LOO, empeora

codificacion_ponderada(df, 'MSZoning', 'SalePrice')

In [33]:
correlacion = df['MSZoning'].corr(df['SalePrice'])
print(correlacion)

0.240161626501391


4. [LotFrontage](../docs/descripcion_variables.md#variable-lotfrontage): Pies lineales de calle conectados a la propiedad.

La variable **LotFrontage** presenta un total de **486** valores NaN de un total de **2919** registros.
- **Conjunto de Entrenamiento**: 17.74% de valores NaN
- **Conjunto de Prueba**: 15.56% de valores NaN

Ambos conjuntos presentan valores similares en sus estadísticas centrales (media, mediana, cuartiles). Sin embargo, se observa una diferencia significativa en el valor máximo, lo que podría ser indicativo de valores atípicos en el conjunto de entrenamiento.

In [34]:
describe_train_test(df, 'LotFrontage')

,Train,Test
count,1201.000000,1232.000000
mean,70.049958,68.580357
std,24.284752,22.376841
min,21.000000,21.000000
25%,59.000000,58.000000
50%,69.000000,67.000000
75%,80.000000,80.000000
max,313.000000,200.000000


In [35]:
crear_histograma(df, 'LotFrontage', title="Distribución de LotFrontage antes de la imputación", color="blue")

La variable **LotFrontage** está correlacionada con **LotArea** de la siguiente manera:

- **Correlación de Pearson**: 0.4899
- **Correlación de Spearman**: 0.6571
- **Correlación de Kendall**: 0.5129

In [36]:
boxplot_train_test(df, 'LotFrontage', 'Dataset', title="Distribución LotFrontage train test")

Se prueba a imputar la mediana, distinta de `nan`. Finalmente, se decide utilizar **KNN Imputer** debido a que este método basa su imputación en los valores de los vecinos más cercanos, lo que lo hace menos susceptible a los efectos de los outliers y a las distribuciones no normales en los datos. 

Además, para evitar **data leakage**, durante la imputación se utiliza únicamente el conjunto de entrenamiento.

In [37]:
print(f"Valores faltantes antes de la imputación: {df['LotFrontage'].isna().sum()}")

neighbors = range(10, 50)
correlations = []

for n in neighbors:
        imputer = KNNImputer(n_neighbors=n)
        
        # Ajustar el imputador solo con el conjunto de entrenamiento
        df.loc[df['Dataset'] == 'train', 'LotFrontage'] = imputer.fit_transform(
            df.loc[df['Dataset'] == 'train', ['LotFrontage', 'LotArea']]
        )[:, 0]
        
        # Aplicar la imputación al conjunto de prueba
        df.loc[df['Dataset'] == 'test', 'LotFrontage'] = imputer.transform(
            df.loc[df['Dataset'] == 'test', ['LotFrontage', 'LotArea']]
        )[:, 0]

print(f"Valores faltantes después de la imputación: {df['LotFrontage'].isna().sum()}")

Valores faltantes antes de la imputación: 486
Valores faltantes después de la imputación: 0


### Comparación de las estadísticas antes y después de la imputación

Se comparan las estadísticas de **LotFrontage** antes y después de la imputación:

- **Mean**:  En ambos conjuntos, la media ha aumentado ligeramente después de la imputación. Esto es normal después de la imputación, especialmente si los valores imputados tienden a ser más cercanos a la media general en lugar de simplemente reemplazar los valores ausentes.

- **Std**: La desviación estándar aumentó ligeramente en ambos conjuntos. Esto indica que los valores imputados han introducido algo de variabilidad adicional en los datos. Sin embargo, estos cambios son menores, lo que sugiere que la imputación no ha alterado significativamente la dispersión general de los datos.

- **Min y max**: Los valores extremos en ambos conjuntos no se vieron afectados por la imputación, manteniendo los valores originales.

La imputación parece haber sido exitosa, ya que los cambios son pequeños en términos de estadísticas generales como la media, la desviación estándar y los valores extremos.

In [38]:
describe_train_test(df, 'LotFrontage')

,Train,Test
count,1460.000000,1459.000000
mean,70.981575,69.611035
std,23.478104,21.710554
min,21.000000,21.000000
25%,60.000000,60.000000
50%,70.000000,70.000000
75%,82.000000,80.300000
max,313.000000,200.000000


### Análisis de la Distribución de `LotFrontage`

- Los **valores imputados** parecen seguir la misma tendencia que los valores originales, lo que significa que el proceso de imputación no introdujo grandes cambios en la forma de la distribución de la variable.
- Aunque se imputaron valores faltantes, la **media** y la **desviación estándar** de los datos no se vieron afectadas de manera considerable.

En general, la **distribución de los datos no ha cambiado significativamente** después de la imputación. 

In [39]:
# crear_histograma(df, 'LotFrontage', title="Distribución de LotFrontage después de la imputación", color="green", nbins=20)

In [40]:
print(sorted(df["LotFrontage"].unique()))

[21.0, 22.0, 22.2, 24.0, 24.3, 25.0, 26.0, 27.8, 28.0, 28.1, 30.0, 31.0, 32.0, 33.0, 34.0, 34.4, 35.0, 36.0, 36.2, 37.0, 37.3, 37.8, 38.0, 38.2, 38.9, 39.0, 39.9, 40.0, 41.0, 42.0, 42.2, 42.8, 43.0, 44.0, 44.8, 45.0, 45.3, 45.5, 46.0, 46.1, 46.3, 46.6, 47.0, 47.5, 48.0, 49.0, 50.0, 51.0, 51.8, 52.0, 52.7, 53.0, 53.5, 53.6, 54.0, 54.3, 54.6, 55.0, 55.2, 55.3, 55.4, 56.0, 56.3, 56.6, 57.0, 57.8, 58.0, 58.3, 58.8, 59.0, 59.1, 59.5, 60.0, 60.4, 60.9, 61.0, 61.1, 61.2, 61.4, 61.5, 61.9, 62.0, 62.1, 62.4, 63.0, 63.1, 63.2, 63.5, 63.8, 63.9, 64.0, 64.1, 64.4, 64.5, 64.6, 64.7, 64.8, 64.9, 65.0, 65.1, 65.6, 65.8, 66.0, 66.2, 66.3, 66.5, 67.0, 67.1, 67.2, 67.4, 67.7, 68.0, 68.1, 68.2, 68.3, 68.4, 68.5, 68.6, 68.8, 68.9, 69.0, 69.3, 69.6, 69.7, 69.9, 70.0, 70.2, 70.3, 70.4, 70.5, 70.6, 70.8, 71.0, 71.1, 71.2, 71.5, 71.6, 71.7, 71.8, 71.9, 72.0, 72.1, 72.3, 72.5, 72.7, 72.8, 73.0, 73.2, 73.3, 73.4, 73.5, 73.6, 73.8, 73.9, 74.0, 74.4, 74.6, 74.9, 75.0, 75.4, 75.6, 75.7, 75.8, 75.9, 76.0, 76.4, 76.

### Decisión de no hacer binning a `LotFrontage` 

El binning agrupa los valores de una variable en intervalos o "bins". Al convertir `LotFrontage` en una variable categórica utilizando diferentes números de bins, se observa que se pierde información detallada sobre las variaciones continuas de esta variable. Esto, a su vez, reduce la correlación con `SalePrice` y `LotArea`; Por lo que se toma la decisión de **no aplicar binning** a `LotFrontage` y mantenerla como una variable continua.

```python
df['LotFrontage'] = pd.cut(df['LotFrontage'], bins= 10, labels=False)

5. [LotArea](../docs/descripcion_variables.md#variable-lotarea): Tamaño del lote en pies cuadrados.

1 pie cuadrado ≈ 0.092903 metros cuadrados.

In [41]:
describe_train_test(df, 'LotArea')

,Train,Test
count,1460.000000,1459.000000
mean,10516.828082,9819.161069
std,9981.264932,4955.517327
min,1300.000000,1470.000000
25%,7553.500000,7391.000000
50%,9478.500000,9399.000000
75%,11601.500000,11517.500000
max,215245.000000,56600.000000


In [42]:
# boxplot_train_test(df, 'LotArea', 'Dataset', title="Comparación de la Distribución de LotArea Antes de la Transformación Logarítmica")

Muestra la relación entre el tamaño del lote original y el precio de venta.

In [43]:
# Gráfico de dispersión de LotArea vs SalePrice
fig1 = px.scatter(df, x="LotArea", y="SalePrice", 
                  title="Relación entre LotArea y SalePrice", 
                  labels={"LotArea": "Tamaño del Lote (LotArea)", "SalePrice": "Precio de Venta (SalePrice)"})
# fig1.show()

Se crea una columna con LotArea (Tamaño del lote en pies cuadrados) en logaritmo con el objetivo de observar la distribución.

In [44]:
df['LotArea_log'] = (df['LotArea'] + 1).apply(np.log)

In [45]:
# boxplot_train_test(df, 'LotArea_log', 'Dataset', title="Comparación de la Distribución de LotArea Después de la Transformación Logarítmica")

Muestra la relación entre el tamaño del lote transformado logarítmicamente y el precio de venta.

In [46]:
# Gráfico de dispersión de LotArea_log vs SalePrice
fig2 = px.scatter(df, x="LotArea_log", y="SalePrice", 
                  title="Relación entre LotArea (Log) y SalePrice", 
                  labels={"LotArea_log": "Tamaño del Lote (Log)", "SalePrice": "Precio de Venta (SalePrice)"})
# fig2.show()

In [47]:
df.drop(columns=['LotArea_log'], inplace=True)

6. [Street](../docs/descripcion_variables.md#variable-street): Tipo de acceso por carretera.

In [48]:
resultado = analizar_precio_viviendas_por_variable(df, 'Street')
resultado

,Street,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
1,Pave,181130.538514,2907,99.5889,1454,1453
0,Grvl,130190.500000,12,0.4111,6,6


In [49]:
# distribucion_target_con_variable(df, 'SalePrice', 'Street', title='Impacto del Tipo de Acceso por carretera en el Precio de Venta')

Inicialmente, se había codificado la variable **Street** con la función codificación_ponderada, pero se obtienen mejores resultados utilizando la técnica de **get dummies**.

In [50]:
df = pd.get_dummies(df, columns=['Street'], drop_first=True)

7. [Alley](../docs/descripcion_variables.md#variable-alley): Tipo de acceso por callejón.

La variable `Alley` tiene `2721 NaN`. Voy a probar rellenarlos con el valor "Sin_acceso_callejon" para ver cómo se comportan los datos. Sin embargo, es importante destacar que estos valores faltantes corresponden al 93% de los datos, lo que indica que esta variable está muy incompleta. Si esta estrategia de imputación no mejora la calidad de los datos, consideraré eliminar la variable debido a la gran cantidad de valores faltantes.

In [51]:
df['Alley'].fillna('Sin_acceso_callejon', inplace=True)

In [52]:
resultado = analizar_precio_viviendas_por_variable(df, 'Alley')
resultado

,Alley,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
2,Sin_acceso_callejon,183452.131483,2721,93.216855,1369,1352
1,Pave,168000.585366,78,2.672148,41,37
0,Grvl,122219.080000,120,4.110997,50,70


In [53]:
distribucion_target_con_variable(df, 'SalePrice', 'Alley', title='Impacto del Tipo de Acceso por callejón en el Precio de Venta')

Se consideraron dos técnicas para manejar la variable **Alley**, codificación one-hot  y codificación ponderada; sin embargo, finalmente se decidió retirar esta variable del modelo, ya que ninguna de las técnicas aplicadas logró mejorar significativamente el rendimiento.

In [54]:
codificacion_ponderada(df, 'Alley', 'SalePrice')

8. [LotShape](../docs/descripcion_variables.md#variable-lotshape): Forma general de la propiedad.

En el análisis del precio de las viviendas, se ha considerado la posibilidad de agrupar las categorías **IR2** (Moderadamente irregular) e **IR3** (Muy irregular) de la variable **LotShape**. Sin embargo, se ha decidido mantener estas categorías separadas por la diferencia del **Precio_Promedio**, esta diferencia de **23,796.87** sugiere que las propiedades en estas categorías pueden ser percibidas de manera diferente en el mercado.

In [55]:
resultado_LotShape = analizar_precio_viviendas_por_variable(df, 'LotShape')
resultado_LotShape

,LotShape,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
1,IR2,239833.365854,76,2.603631,41,35
2,IR3,216036.500000,16,0.548133,10,6
0,IR1,206101.665289,968,33.162042,484,484
3,Reg,164754.818378,1859,63.686194,925,934


In [56]:
distribucion_target_con_variable(df, 'SalePrice', 'LotShape', title='Impacto de la Forma general de la propiedad en el Precio de Venta')

In [57]:
# NO GET DUMMIES
codificacion_ponderada(df, 'LotShape', 'SalePrice')

9. [LandContour](../docs/descripcion_variables.md#variable-landcontour): Planicidad de la propiedad.

In [58]:
resultado_LandContour = analizar_precio_viviendas_por_variable(df, 'LandContour')
resultado_LandContour

,LandContour,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
1,HLS,231533.940000,120,4.110997,50,70
2,Low,203661.111111,60,2.055498,36,24
3,Lvl,180183.746758,2622,89.825283,1311,1311
0,Bnk,143104.079365,117,4.008222,63,54


In [59]:
# distribucion_target_con_variable(df, 'SalePrice', 'LandContour', title='Impacto de la Planicidad de la propiedad en el Precio de Venta')

In [60]:
codificacion_ponderada(df, 'LandContour', 'SalePrice')

10. [Utilities](../docs/descripcion_variables.md#variable-utilities): Tipo de servicios disponibles.

In [61]:
resultado_Utilities = analizar_precio_viviendas_por_variable(df, 'Utilities')
resultado_Utilities

,Utilities,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
0,AllPub,180950.95682,2916,99.897225,1459,1457
1,NoSeWa,137500.00000,1,0.034258,1,0


In [62]:
utilities= df["Utilities"].value_counts(dropna=False)
print(utilities)

Utilities
AllPub    2916
NaN          2
NoSeWa       1
Name: count, dtype: int64


### Decisión sobre la variable `Utilities`

En la columna `Utilities`, el valor **[NoSeWa]** (sin alcantarillado) está presente en el conjunto de entrenamiento, pero no en el conjunto de prueba. Además, este valor tiene solo un único registro y presenta 2 valores `NaN`. Dado que **todas, menos tres viviendas, tienen `AllPub`** (todos los servicios públicos como agua, electricidad y gas), **no tiene sentido utilizar esta variable** en el modelo. 

In [63]:
df.drop(columns=['Utilities'], inplace=True)

11. [LotConfig](../docs/descripcion_variables.md#variable-lotconfig): Configuración del lote.

In [64]:
resultado_LotConfig = analizar_precio_viviendas_por_variable(df, 'LotConfig')
resultado_LotConfig 

,LotConfig,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
1,CulDSac,223854.617021,176,6.029462,94,82
3,FR3,208475.000000,14,0.479616,4,10
0,Corner,181623.425856,511,17.505995,263,248
2,FR2,177934.574468,85,2.911956,47,38
4,Inside,176938.047529,2133,73.072970,1052,1081


In [65]:
# distribucion_target_con_variable(df, 'SalePrice', 'LotConfig', title='Impacto de la Configuración del lote en el Precio de Venta')

In [66]:
codificacion_ponderada(df, 'LotConfig', 'SalePrice')

12. [LandSlope](../docs/descripcion_variables.md#variable-landslope): Pendiente de la propiedad.

In [67]:
valores_LandSlope = {
    "Sev": "Terreno severamente inclinado",
    "Mod": "Terreno moderadamente inclinado",
    "Gtl": "Terreno ligeramente inclinado"
}

df["LandSlope"] = df["LandSlope"].map(valores_LandSlope)

In [68]:
resultado_LandSlope = analizar_precio_viviendas_por_variable(df, 'LandSlope')
resultado_LandSlope

,LandSlope,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
2,Terreno severamente inclinado,204379.230769,16,0.548133,13,3
1,Terreno moderadamente inclinado,196734.138462,125,4.282288,65,60
0,Terreno ligeramente inclinado,179956.799566,2778,95.169579,1382,1396


In [69]:
# distribucion_target_con_variable(df, 'SalePrice', 'LandSlope', title='Impacto de la Pendiente de la propiedad en el Precio de Venta')

Se realizaron varias pruebas para incorporar la variable `LandSlope` al modelo, pero se observaron inconsistencias en los resultados.

- Codificación ordinal: Se asignaron valores ordinales a los tipos de terreno (`Sev`, `Mod`, `Gtl`). Sin embargo, los terrenos severamente inclinados (`Sev`) presentaron precios más altos, lo cual contradice la lógica esperada, ya que una mayor inclinación debería correlacionarse con precios más bajos.
- Codificación ponderada: Se asignaron pesos a cada categoría ya que la codificación ordinal no seguía la lógica esperada.
- Codificación con `get_dummies`: Tampoco mejoró el modelo.

Se decidió finalmente eliminar la variable `LandSlope` del modelo, ya que su inclusión resultaba en valores más dispersos y menos similares a los datos de prueba.

In [70]:
codificacion_ponderada(df, 'LandSlope', 'SalePrice')

In [71]:
# Correlaciones de las características del terreno después de la limpieza de datos

columnas_a_analizar = ["SalePrice", "MSSubClass", "MSZoning", "LotFrontage", "LotArea", 'Street_Pave', 'Alley',
                           "LotShape", "LandContour", "LotConfig", "LandSlope" ] 

visualizar_correlaciones(df, columnas_a_analizar)

### 2. **Características del Entorno y Vecindario**
> **Objetivo**: Evaluar la influencia de la ubicación y las condiciones externas en el valor de la propiedad.

13. [Neighborhood](../docs/descripcion_variables.md#variable-neighborhood): Ubicaciones físicas dentro de los límites de la ciudad.
14. [Condition1](../docs/descripcion_variables.md#variable-condition1): Proximidad a la carretera principal o ferrocarril.
15. [Condition2](../docs/descripcion_variables.md#variable-condition2): Proximidad a una segunda carretera principal o ferrocarril.

In [72]:
caracteristicas_vecindario = ['Neighborhood', 'Condition1', 'Condition2']

df[["SalePrice"] + caracteristicas_vecindario].head()

,SalePrice,Neighborhood,Condition1,Condition2
0,208500.0,CollgCr,Norm,Norm
1,181500.0,Veenker,Feedr,Norm
2,223500.0,CollgCr,Norm,Norm
3,140000.0,Crawfor,Norm,Norm
4,250000.0,NoRidge,Norm,Norm


In [73]:
resumen = resumen_columnas(df, caracteristicas_vecindario)
resumen

,Tipo de Dato,Nº Valores Únicos,Nº de NaN,Porcentaje NaN Train,Porcentaje NaN Test,Valores Únicos Train,Valores Únicos Test
Neighborhood,object,25,0,0.0,0.0,[],[]
Condition1,object,9,0,0.0,0.0,[],[]
Condition2,object,8,0,0.0,0.0,"[RRNn, RRAn, RRAe]",[]


In [74]:
valores_unicos = obtener_valores_unicos(df, caracteristicas_vecindario)

for key, value in valores_unicos.items():
    print(f"'{key}': {value},")

'Neighborhood': ['Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr', 'Crawfor', 'Edwards', 'Gilbert', 'IDOTRR', 'MeadowV', 'Mitchel', 'NAmes', 'NPkVill', 'NWAmes', 'NoRidge', 'NridgHt', 'OldTown', 'SWISU', 'Sawyer', 'SawyerW', 'Somerst', 'StoneBr', 'Timber', 'Veenker'],
'Condition1': ['Artery', 'Feedr', 'Norm', 'PosA', 'PosN', 'RRAe', 'RRAn', 'RRNe', 'RRNn'],
'Condition2': ['Artery', 'Feedr', 'Norm', 'PosA', 'PosN', 'RRAe', 'RRAn', 'RRNn'],


13. [Neighborhood](../docs/descripcion_variables.md#variable-neighborhood): Ubicaciones físicas dentro de los límites de la ciudad.

In [75]:
resultado = analizar_precio_viviendas_por_variable(df, 'Neighborhood')
resultado

,Neighborhood,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
15,NoRidge,335295.317073,71,2.432340,41,30
16,NridgHt,316270.623377,166,5.686879,77,89
22,StoneBr,310499.000000,51,1.747174,25,26
23,Timber,242247.447368,72,2.466598,38,34
24,Veenker,238772.727273,24,0.822199,11,13
21,Somerst,225379.837209,182,6.235012,86,96
4,ClearCr,212565.428571,44,1.507366,28,16
6,Crawfor,210624.725490,103,3.528606,51,52
5,CollgCr,197965.773333,267,9.146968,150,117
0,Blmngtn,194870.882353,28,0.959233,17,11


Se decidió modificar la variable de los vecindarios para clasificar las viviendas según sus precios por vecindario en cuatro categorías: **Alto**, **Medio Alto**, **Medio**, **Medio Bajo** y **Bajo**.

Esta categorización ayuda a simplificar la información y mejorar la comprensión del modelo respecto a los vecindarios. Al reducir la diversidad de opciones, se logró una mejoría en las predicciones, como se reflejó en una disminución del RMSLE.

- Bajo: precio menor a 130,000 
- Medio-Bajo: precios entre 130,000 y 165,000
- Medio: precios entre 165,000 y 200,000 
- Medio-Alto: precios entre 200,000 y 250,000
- Alto: precio mayor o igual a 250,000

Agregar una categoría más de Muy alto ya empeora el modelo.

In [76]:
df["SalePrice"].describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

In [77]:
vecindarios_precio = {
    'NoRidge': 'Alto',
    'NridgHt': 'Alto',
    'StoneBr': 'Alto',
    'Timber': 'Medio-Alto',
    'Veenker': 'Medio-Alto',
    'Somerst': 'Medio-Alto',
    'ClearCr': 'Medio',
    'Crawfor': 'Medio',
    'CollgCr': 'Medio',
    'Blmngtn': 'Medio',
    'Gilbert': 'Medio',
    'NWAmes': 'Medio',
    'SawyerW': 'Medio',
    'Mitchel': 'Medio-Bajo',
    'NAmes': 'Medio-Bajo',
    'NPkVill': 'Medio-Bajo',
    'SWISU': 'Medio-Bajo',
    'Blueste': 'Medio-Bajo',
    'Sawyer': 'Medio-Bajo',
    'OldTown': 'Bajo',
    'Edwards': 'Bajo',
    'BrkSide': 'Bajo',
    'BrDale': 'Bajo',
    'IDOTRR': 'Bajo',
    'MeadowV': 'Bajo'
}

df['Neighborhood'] = df['Neighborhood'].map(vecindarios_precio)


In [78]:
resultado = analizar_precio_viviendas_por_variable(df, 'Neighborhood')
resultado

,Neighborhood,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
0,Alto,320716.230769,288,9.866393,143,145
3,Medio-Alto,231219.029630,278,9.523810,135,143
2,Medio,196377.061269,863,29.564919,457,406
4,Medio-Bajo,145103.007812,789,27.029805,384,405
1,Bajo,122006.111437,701,24.015074,341,360


In [79]:
distribucion_target_con_variable_vertical(df, 'SalePrice', 'Neighborhood', title="Impacto de los vecindarios en Ames en el Precio de Venta")

El modelo entiende mejor la codificación ordinal para esta variable que la codificación ponderada, después de ajustar bien los rangos de valores de las casas, mejora significativamente las predicciones.

In [80]:
categorias_Neighborhood = ['Alto', 'Medio-Alto', 'Medio', 'Medio-Bajo', 'Bajo']

df = aplicar_codificacion_ordinal_especifica(df, 'Neighborhood', categorias_Neighborhood)

14. [Condition1](../docs/descripcion_variables.md#variable-condition1): Proximidad a la carretera principal o ferrocarril.
15. [Condition2](../docs/descripcion_variables.md#variable-condition2): Proximidad a una segunda carretera principal o ferrocarril.

In [81]:
calcular_porcentaje_coincidencias(df, 'Condition1', 'Condition2') 

"El porcentaje de coincidencias entre 'Condition1' y 'Condition2' es de 86.47%."

In [82]:
resultado = analizar_precio_viviendas_por_variable(df, 'Condition1')
resultado

,Condition1,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
3,PosA,225875.000000,20,0.685166,8,12
4,PosN,215184.210526,39,1.336074,19,20
8,RRNn,212400.000000,9,0.308325,5,4
7,RRNe,190750.000000,6,0.205550,2,4
2,Norm,184495.492063,2511,86.022610,1260,1251
6,RRAn,184396.615385,50,1.712915,26,24
1,Feedr,142475.481481,164,5.618362,81,83
5,RRAe,138400.000000,28,0.959233,11,17
0,Artery,135091.666667,92,3.151764,48,44


In [83]:
resultado = analizar_precio_viviendas_por_variable(df, 'Condition2')
resultado

,Condition2,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
3,PosA,325000.000000,4,0.137033,1,3
4,PosN,284875.000000,4,0.137033,2,2
5,RRAe,190000.000000,1,0.034258,1,0
2,Norm,181169.405536,2889,98.972251,1445,1444
6,RRAn,136905.000000,1,0.034258,1,0
1,Feedr,121166.666667,13,0.445358,6,7
0,Artery,106500.000000,5,0.171292,2,3
7,RRNn,96750.000000,2,0.068517,2,0


Antes de eliminar la variable `Condition2`, se comprobó si había casos en los que `Condition1` indicara una condición favorable para la ubicación de la casa mientras que `Condition2` no lo hacía. Sin embargo, tras la revisión, no se encontraron diferencias significativas.

In [84]:
# Filtrar las filas donde Condition1 es positiva o neutra y Condition2 es diferente
condition_Norm = df[(df['Condition1'] == 'Norm') & (df['Condition2'] != 'Norm')]
condition_PosA = df[(df['Condition1'] == 'PosA') & (df['Condition2'] != 'PosA')]
condition_PosN = df[(df['Condition1'] == 'PosN') & (df['Condition2'] != 'PosN')]

# Mostrar los valores únicos de Condition2 en esta condición
condition2_condition_Norm = condition_Norm['Condition2'].unique()
condition2_condition_PosA = condition_PosA['Condition2'].unique()
condition2_condition_PosA = condition_PosN['Condition2'].unique()

print(condition2_condition_Norm)
print(condition2_condition_PosA)
print(condition2_condition_PosA)

[]
['Norm']
['Norm']


Al no aportar ninguna información extra, se decide borrar la columna porque tiene un porcentaje de coincidencia con `Condition1` del 86.47%. Parece que el valor `Norm` de `Condition2` representa el 98.97%, y junto con las variables que están en train pero no en test: `RRAe`, `RRNn`, `RRAn` suman más del 99% de los datos.

In [85]:
df.drop(columns=['Condition2'], inplace=True)

Para mejorar la representación de la variable Condition1 se decide agrupar según el impacto

- **Positiva**:
  - `PosN`: Proximidad positiva (ej., áreas verdes o parques).
  - `PosA`: Proximidad positiva a una carretera.

- **Neutra**:
  - `Norm`: Normal (sin influencias significativas).
  - `RRNn`: Proximidad a una carretera no elevada.
  - `RRNe`:  A menos de 200 pies del ferrocarril Este-Oeste.

- **Negativa**:
  - `Feedr`: Frente a la carretera (ruido).
  - `Artery`: Proximidad a una carretera principal (tráfico y ruido).
  - `RRAe`: Proximidad a una carretera elevada (impacto visual negativo).
  - `RRAn`: Proximidad a una carretera.

Esta agrupación permite una mejor representación y un análisis más preciso del impacto de la proximidad a carreteras y otras vías en los precios de las viviendas. Mejora las predicciones. Añadir un valor intermedio de negativo no mejora.

In [86]:
mapeo_Condition1 = {
    'PosA': 'Positiva',
    'PosN': 'Positiva',
    'Norm': 'Neutra',
    'RRNn': 'Neutra',
    'RRNe': 'Neutra',
    'RRAn': 'Negativa', 
    'Feedr': 'Negativa',
    'RRAe': 'Negativa',
    'Artery': 'Negativa'
}

df['Condition1'] = df['Condition1'].map(mapeo_Condition1)

In [87]:
resultado = analizar_precio_viviendas_por_variable(df, 'Condition1')
resultado

,Condition1,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
2,Positiva,218351.851852,59,2.021240,27,32
1,Neutra,184615.485399,2526,86.536485,1267,1259
0,Negativa,146636.301205,334,11.442275,166,168


Mejor que codificación ponderada.

In [88]:
categorias_Neighborhood = ['Positiva', 'Neutra', 'Negativa']

df = aplicar_codificacion_ordinal_especifica(df, 'Condition1', categorias_Neighborhood)

In [89]:
columnas_a_analizar = ["SalePrice", 'Neighborhood', 'Condition1']

visualizar_correlaciones(df, columnas_a_analizar)


### 3. **Características del Edificio y Estilo**
> **Objetivo**: Entender cómo el tipo, estilo y calidad del edificio afectan al precio de venta.

16. [BldgType](../docs/descripcion_variables.md#variable-bldgtype): Tipo de vivienda.
17. [HouseStyle](../docs/descripcion_variables.md#variable-housestyle): Estilo de la vivienda.
18. [OverallQual](../docs/descripcion_variables.md#variable-overallqual): Calidad general de los materiales y acabados.
19. [OverallCond](../docs/descripcion_variables.md#variable-overallcond): Calificación de la condición general.
20. [YearBuilt](../docs/descripcion_variables.md#variable-yearbuilt): Fecha de construcción original.
21. [YearRemodAdd](../docs/descripcion_variables.md#variable-yearremodadd): Fecha de remodelación.

In [90]:
caracteristicas_edificio = ["BldgType", "HouseStyle", "OverallQual", "OverallCond", "YearBuilt", "YearRemodAdd"]

df[["SalePrice"] + caracteristicas_edificio].head()

,SalePrice,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd
0,208500.0,1Fam,2Story,7,5,2003,2003
1,181500.0,1Fam,1Story,6,8,1976,1976
2,223500.0,1Fam,2Story,7,5,2001,2002
3,140000.0,1Fam,2Story,7,5,1915,1970
4,250000.0,1Fam,2Story,8,5,2000,2000


In [91]:
resumen = resumen_columnas(df, caracteristicas_edificio)
resumen

,Tipo de Dato,Nº Valores Únicos,Nº de NaN,Porcentaje NaN Train,Porcentaje NaN Test,Valores Únicos Train,Valores Únicos Test
BldgType,object,5,0,0.0,0.0,[],[]
HouseStyle,object,8,0,0.0,0.0,[2.5Fin],[]
OverallQual,int64,10,0,0.0,0.0,[],[]
OverallCond,int64,9,0,0.0,0.0,[],[]
YearBuilt,int64,118,0,0.0,0.0,[],[]
YearRemodAdd,int64,61,0,0.0,0.0,[],[]


In [92]:
valores_unicos = obtener_valores_unicos(df, caracteristicas_edificio)

for key, value in valores_unicos.items():
    print(f"'{key}': {value},")

'BldgType': ['1Fam', '2fmCon', 'Duplex', 'Twnhs', 'TwnhsE'],
'HouseStyle': ['1.5Fin', '1.5Unf', '1Story', '2.5Fin', '2.5Unf', '2Story', 'SFoyer', 'SLvl'],
'OverallQual': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
'OverallCond': [1, 2, 3, 4, 5, 6, 7, 8, 9],
'YearBuilt': [1872, 1875, 1879, 1880, 1882, 1885, 1890, 1892, 1893, 1895, 1896, 1898, 1900, 1901, 1902, 1904, 1905, 1906, 1907, 1908, 1910, 1911, 1912, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010],
'YearRemodAdd': [1950, 1951, 1

16. [BldgType](../docs/descripcion_variables.md#variable-bldgtype): Tipo de vivienda.

In [93]:
diccionario_BldgType = {
    '1Fam': 'Vivienda unifamiliar',
    '2fmCon': 'Duplex (conversion)',
    'Duplex': 'Duplex estandar',
    'TwnhsE': 'Casa adosada esquina',
    'Twnhs': 'Casa adosada interior'
}

df['BldgType'] = df['BldgType'].map(diccionario_BldgType)

In [94]:
resultado = analizar_precio_viviendas_por_variable(df, 'BldgType')
resultado

,BldgType,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
4,Vivienda unifamiliar,185763.807377,2425,83.076396,1220,1205
0,Casa adosada esquina,181959.342105,227,7.776636,114,113
1,Casa adosada interior,135911.627907,96,3.288798,43,53
3,Duplex estandar,133541.076923,109,3.734156,52,57
2,Duplex (conversion),128432.258065,62,2.124015,31,31


In [95]:
# distribucion_target_con_variable(df, 'SalePrice', 'BldgType', title='Impacto del Tipo de vivienda en el Precio de Venta')

Se trata de mejorar las predicciones uniendo **Tipo de vivienda** por 'Vivienda unifamiliar', 'Casa adosada' y 'Duplex'. Este agrupamiento mejora la correlación, pero el mayor incremento se logra al **codificar ordinalmente** el tipo de vivienda de mejor a peor, aumentando la correlación con la variable objetivo a **-0.176025**. A pesar de esta mejora, sigue siendo mejor eliminar la columna en la predicción utilizando **XGBoost Regressor**, ya que optimiza el modelo eliminando información redundante.

In [96]:
categorias_BldgType = ['Vivienda unifamiliar', 'Casa adosada esquina', 'Casa adosada interior', 'Duplex estandar', 'Duplex (conversion)']

df = aplicar_codificacion_ordinal_especifica(df, 'BldgType', categorias_BldgType)

17. [HouseStyle](../docs/descripcion_variables.md#variable-housestyle): Estilo de la vivienda.

In [97]:
diccionario_HouseStyle = {
    '1Story': 'Una planta',
    '1.5Fin': 'Una y media terminada',
    '1.5Unf': 'Una y media sin terminar',
    '2Story': 'Dos plantas',
    '2.5Fin': 'Dos y media terminada',
    '2.5Unf': 'Dos y media sin terminar',
    'SFoyer': 'Entrada separada por escaleras',
    'SLvl': 'Entrada con desniveles internos'
}
df['HouseStyle'] = df['HouseStyle'].map(diccionario_HouseStyle)

In [98]:
resultado = analizar_precio_viviendas_por_variable(df, 'HouseStyle')
resultado

,HouseStyle,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
2,Dos y media terminada,220000.000000,8,0.274066,8,0
0,Dos plantas,210051.764045,872,29.873244,445,427
5,Una planta,175985.477961,1471,50.393971,726,745
3,Entrada con desniveles internos,166703.384615,128,4.385063,65,63
1,Dos y media sin terminar,157354.545455,24,0.822199,11,13
7,Una y media terminada,143116.740260,314,10.757109,154,160
4,Entrada separada por escaleras,135074.486486,83,2.843440,37,46
6,Una y media sin terminar,110150.000000,19,0.650908,14,5


Como 2.5Fin está en train pero no en test representado, lo cambio por el valor más parecido tanto en número de plantas como en precio promedio, que es 2.5Unf.

In [99]:
df['HouseStyle'] = df['HouseStyle'].replace({'Dos y media terminada': 'Dos plantas'})

In [100]:
# distribucion_target_con_variable(df, 'SalePrice', 'HouseStyle', title='Impacto del Estilo de la vivienda en el Precio de Venta')

Se trata de categorizar por alta media y baja calidad, pero se obtienen peores resultados. 

Al utilizar una codificación ordinal, es posible asignar pesos específicos a cada categoría, reflejando la relevancia del estilo de la vivienda en relación con su calidad. Por ejemplo, casas sin terminar o con menos plantas suelen tener una calidad percibida más baja. Esta codificación mejora la correlación en atributos como el año de construcción, logrando un mejor ajuste. Sin embargo, incluso con esta mejora, la correlación con la variable objetivo se mantiene en aproximadamente 0.3. Finalmente, al incluir este atributo en el modelo, los resultados empeoran, por lo que se decide eliminarlo.

In [101]:
categorias_HouseStyle = ["Dos plantas", "Una planta", "Entrada con desniveles internos", 
               "Dos y media sin terminar", "Una y media terminada", "Entrada separada por escaleras", 
               "Una y media sin terminar"]

df = aplicar_codificacion_ordinal_especifica(df, 'HouseStyle', categorias_HouseStyle)

18. [OverallQual](../docs/descripcion_variables.md#variable-overallqual): Calidad general de los materiales y acabados.

In [102]:
resultado = analizar_precio_viviendas_por_variable(df, 'OverallQual')
resultado

,OverallQual,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
9,10,438588.388889,31,1.062008,18,13
8,9,367513.023256,107,3.665639,43,64
7,8,274735.535714,342,11.716341,168,174
6,7,207716.423197,600,20.554985,319,281
5,6,161603.034759,731,25.042823,374,357
4,5,133523.347607,825,28.263104,397,428
3,4,108420.655172,226,7.742378,116,110
2,3,87473.750000,40,1.370332,20,20
1,2,51770.333333,13,0.445358,3,10
0,1,50150.000000,4,0.137033,2,2


In [103]:
# distribucion_target_con_variable_vertical(df, 'SalePrice', 'OverallQual', title="Impacto de la Calidad general de los materiales en el Precio de Venta")

In [104]:
fig = px.scatter(df, x='OverallQual', y='SalePrice',
                 title='Relación entre OverallQual y SalePrice',
                 labels={'OverallQual': 'Calidad General', 'SalePrice': 'Precio de Venta (USD)'},
                 trendline="ols") 

fig.update_layout(height=600, title_x=0.5)
# fig.show()

### Explicación sobre la aplicación de Leave-One-Out Encoding (LOO) en `OverallQual`

La variable **`OverallQual`** es una de las más relevantes en el modelo, ya que aparece como la que más ayuda a predecir **`SalePrice`** en las **feature importances**. Se probaron varias estrategias para mejorar el rendimiento del modelo:

Primero, se cambió **`OverallQual`** a categorías como **"Calidad Alta"**, **"Calidad Media"** y **"Calidad Baja"**, con codificación ponderada y ordinal. Se aplicó codificación ponderada, pero también **empeoró el modelo**. Se simplificaron los valores de **`OverallQual`**, agrupando 1, 2 y 3 para darles mayor representación, pero **no hubo cambio** en el rendimiento.
   
La principal razón para aplicar LOO es mejorar la capacidad de generalización del modelo. Aunque **`OverallQual`** tiene una fuerte relación lineal con **`SalePrice`**, su aplicación directa puede causar sobreajuste (overfitting), especialmente si hay categorías con pocas muestras. El uso de **LOO** suaviza la influencia de las categorías al calcular su valor basado en la media de la variable objetivo, excluyendo el valor actual. Esto ayuda a que el modelo aprenda patrones de forma más robusta, reduciendo la **varianza** y permitiendo que generalice mejor a datos no vistos. Además, a pesar de aplicar LOO, la relación entre **`OverallQual`** y **`SalePrice`** sigue siendo de 0.8.

Antes de la aplicación de LOO, las curvas de aprendizaje mostraban una tendencia a **sobreajustarse**, el modelo estaba memorizando los datos en lugar de aprender patrones generales. Al aplicar LOO, ambas curvas de aprendizaje ahora descienden de manera más estable, hay una mejora en la **capacidad predictiva** y en la **generalización** del modelo.

Esta técnica ha resultado en una mejora en las métricas de evaluación y en el rendimiento general del modelo, lo que hace que el modelo sea más robusto y capaz de hacer predicciones más precisas en datos no vistos.

In [105]:
df = codificacion_loo(df, 'OverallQual')

In [106]:
fig = px.scatter(df, x='OverallQual', y='SalePrice',
                 title='Relación entre OverallQual y SalePrice',
                 labels={'OverallQual': 'Calidad General', 'SalePrice': 'Precio de Venta (USD)'},
                 trendline="ols") 

fig.update_layout(height=600, title_x=0.5)
# fig.show()

19. [OverallCond](../docs/descripcion_variables.md#variable-overallcond): Calificación de la condición general.

In [107]:
resultado = analizar_precio_viviendas_por_variable(df, 'OverallCond')
resultado

,OverallCond,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
8,9,216004.545455,41,1.404591,22,19
4,5,203146.914738,1645,56.354916,821,824
6,7,158145.487805,390,13.360740,205,185
7,8,155651.736111,144,4.933196,72,72
5,6,153961.591270,531,18.191161,252,279
1,2,141986.400000,10,0.342583,5,5
3,4,120438.438596,101,3.460089,57,44
2,3,101929.400000,50,1.712915,25,25
0,1,61000.000000,7,0.239808,1,6


In [108]:
# distribucion_target_con_variable_vertical(df, 'SalePrice', 'OverallCond', title="Impacto de la Calificación de la condición general en el Precio de Venta")

In [109]:
# Empeora con codificación LOO
codificacion_ponderada(df, 'OverallCond', 'SalePrice')

20. [YearBuilt](../docs/descripcion_variables.md#variable-yearbuilt): Fecha de construcción original.
21. [YearRemodAdd](../docs/descripcion_variables.md#variable-yearremodadd): Fecha de remodelación.

In [110]:
print(sorted(df["YearBuilt"].unique()))

[1872, 1875, 1879, 1880, 1882, 1885, 1890, 1892, 1893, 1895, 1896, 1898, 1900, 1901, 1902, 1904, 1905, 1906, 1907, 1908, 1910, 1911, 1912, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010]


In [111]:
print(sorted(df["YearRemodAdd"].unique()))

[1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010]


Comprobación de si el año de la remodelacion es menor o = al de venta de la casa y mayor o = que el año de construcción 

YearBuilt < YearRemodAdd < YrSold

In [112]:
def verificar_filas_false(df):
    def verificar_fila(row):
        if pd.isna(row['YearBuilt']) or pd.isna(row['YearRemodAdd']) or pd.isna(row['YrSold']):
            return True  # Considerar filas con NaN como válidas, se tratarán de otra manera
        return row['YearBuilt'] <= row['YearRemodAdd'] <= row['YrSold']

    # Aplicar la lógica inversa (~) para obtener filas incorrectas
    filas_false = df[~df.apply(verificar_fila, axis=1)]
    return filas_false

filas_no_entre = verificar_filas_false(df)

print(len(filas_no_entre))

4


In [113]:
filas_no_entre[['Id','YearBuilt', 'YearRemodAdd', 'YrSold', 'MoSold', 'Dataset']]

,Id,YearBuilt,YearRemodAdd,YrSold,MoSold,Dataset
523,524,2007,2008,2007,10,train
1876,1877,2002,2001,2009,4,test
2295,2296,2007,2008,2007,6,test
2549,2550,2008,2009,2007,10,test


### Corrección de Incongruencias en los Datos

En el conjunto de datos, se han identificado algunos registros con inconsistencias en las fechas, específicamente entre las columnas `YearBuilt`, `YearRemodAdd` y `YrSold`. Estas inconsistencias no tienen sentido lógico y podrían afectar negativamente el análisis y los modelos predictivos.

Dado que estas incongruencias son pocas, se decide corregirlas manualmente. Aunque algunos de estos registros pertenecen al conjunto de prueba (`test`), se opta por realizar estas modificaciones para garantizar la coherencia de los datos.


In [114]:
df.loc[df['Id'] == 524, 'YearRemodAdd'] = 2007
df.loc[df['Id'] == 1877, 'YearRemodAdd'] = 2002
df.loc[df['Id'] == 2296, 'YearRemodAdd'] = 2007
df.loc[df['Id'] == 2550, 'YrSold'] = 2009

In [115]:
filas_no_entre = verificar_filas_false(df)
print(len(filas_no_entre))

0


Se identificó una fila en el conjunto de datos donde el valor de `YrSold` (año de venta) era menor que `YearBuilt` (año de construcción). Para asegurar que no existan más casos similares, se realiza una comprobación en todo el conjunto de datos.

In [116]:
print("¿Existen valores nulos en la columna 'YrSold'?", df['YrSold'].isna().any())

filas_yearbuilt_mayor_yrsold = df[df['YearBuilt'] > df['YrSold']]
print("Número de filas donde 'YearBuilt' es mayor que 'YrSold':", len(filas_yearbuilt_mayor_yrsold))

¿Existen valores nulos en la columna 'YrSold'? False
Número de filas donde 'YearBuilt' es mayor que 'YrSold': 0


### Creación de la Nueva Columna `Antigüedad_Remodelacion`

Se agrega una nueva columna al DataFrame llamada `Antigüedad_Remodelacion`, que calcula la edad de la casa en el momento de su remodelación. Esta columna se define como la diferencia entre el año de remodelación (`YearRemodAdd`) y el año de construcción (`YearBuilt`).
  - Un valor de **0** indica que no ha habido remodelación, ya que el año de remodelación es igual al año de construcción. Este caso es considerado el peor, ya que refleja una falta de mantenimiento o actualizaciones desde la construcción inicial.
  - Valores mayores (por ejemplo, 1, 5, etc.) representan la cantidad de años que pasaron desde la construcción de la casa hasta su remodelación. Cuanto mayor sea este valor, más antigua fue la remodelación.

In [117]:
df['Antigüedad_Remodelacion'] = df['YearRemodAdd'] - df['YearBuilt']

In [118]:
columnas_a_analizar = ["SalePrice", "BldgType", "HouseStyle", "OverallQual", "OverallCond", 'Antigüedad_Remodelacion', 'YearRemodAdd', 'YearBuilt']

visualizar_correlaciones(df, columnas_a_analizar)

### 4. **Características del Techo y Exterior**
> **Objetivo**: Examinar cómo las características del exterior del edificio pueden influir en el precio.

22. [RoofStyle](../docs/descripcion_variables.md#variable-roofstyle): Tipo de techo.
23. [RoofMatl](../docs/descripcion_variables.md#variable-roofmatl): Material del techo.
24. [Exterior1st](../docs/descripcion_variables.md#variable-exterior1st): Recubrimiento exterior de la casa.
25. [Exterior2nd](../docs/descripcion_variables.md#variable-exterior2nd): Recubrimiento exterior adicional.
26. [MasVnrType](../docs/descripcion_variables.md#variable-masvnrtype): Tipo de revestimiento de mampostería.
27. [MasVnrArea](../docs/descripcion_variables.md#variable-masvnrarea): Área de revestimiento de mampostería en pies cuadrados.
28. [ExterQual](../docs/descripcion_variables.md#variable-exterqual): Calidad del material exterior.
29. [ExterCond](../docs/descripcion_variables.md#variable-extercond): Condición presente del material en el exterior.

In [119]:
características_techo = ["RoofStyle", "RoofMatl", "Exterior1st", "Exterior2nd", "MasVnrType", "MasVnrArea", "ExterQual", "ExterCond"]

df[["SalePrice"] + características_techo].head()

,SalePrice,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond
0,208500.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA
1,181500.0,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA
2,223500.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA
3,140000.0,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA
4,250000.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA


In [120]:
resumen = resumen_columnas(df, características_techo)
resumen

,Tipo de Dato,Nº Valores Únicos,Nº de NaN,Porcentaje NaN Train,Porcentaje NaN Test,Valores Únicos Train,Valores Únicos Test
RoofStyle,object,6,0,0.000000,0.000000,[],[]
RoofMatl,object,8,0,0.000000,0.000000,"[Roll, Metal, ClyTile, Membran]",[]
Exterior1st,object,15,1,0.000000,0.068540,"[ImStucc, Stone]",[]
Exterior2nd,object,16,1,0.000000,0.068540,[Other],[]
MasVnrType,object,3,1766,59.726027,61.274846,[],[]
MasVnrArea,float64,444,23,0.547945,1.028101,[],[]
ExterQual,object,4,0,0.000000,0.000000,[],[]
ExterCond,object,5,0,0.000000,0.000000,[],[]


In [121]:
valores_unicos = obtener_valores_unicos(df, características_techo)

for key, value in valores_unicos.items():
    print(f"'{key}': {value},")


'RoofStyle': ['Flat', 'Gable', 'Gambrel', 'Hip', 'Mansard', 'Shed'],
'RoofMatl': ['ClyTile', 'CompShg', 'Membran', 'Metal', 'Roll', 'Tar&Grv', 'WdShake', 'WdShngl'],
'Exterior1st': ['AsbShng', 'AsphShn', 'BrkComm', 'BrkFace', 'CBlock', 'CemntBd', 'HdBoard', 'ImStucc', 'MetalSd', 'Plywood', 'Stone', 'Stucco', 'VinylSd', 'Wd Sdng', 'WdShing'],
'Exterior2nd': ['AsbShng', 'AsphShn', 'Brk Cmn', 'BrkFace', 'CBlock', 'CmentBd', 'HdBoard', 'ImStucc', 'MetalSd', 'Other', 'Plywood', 'Stone', 'Stucco', 'VinylSd', 'Wd Sdng', 'Wd Shng'],
'MasVnrType': ['BrkCmn', 'BrkFace', 'Stone'],
'MasVnrArea': [0.0, 1.0, 3.0, 11.0, 14.0, 16.0, 18.0, 20.0, 22.0, 23.0, 24.0, 27.0, 28.0, 30.0, 31.0, 32.0, 34.0, 36.0, 38.0, 39.0, 40.0, 41.0, 42.0, 44.0, 45.0, 46.0, 47.0, 48.0, 50.0, 51.0, 52.0, 53.0, 54.0, 56.0, 57.0, 58.0, 60.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 72.0, 74.0, 75.0, 76.0, 80.0, 81.0, 82.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100

22. [RoofStyle](../docs/descripcion_variables.md#variable-roofstyle): Tipo de techo.

Para Ames, Iowa, teniendo en cuenta su clima con inviernos fríos y nevados, veranos cálidos y posibles tormentas, el orden de techos de mejor a peor sería:

- **Hip (A cuatro aguas):** Excelente resistencia al viento y gran capacidad de drenaje para nieve y lluvia, ideal para el clima variable de Ames.

- **Gable (A dos aguas):** Diseño simple y eficiente para drenar nieve y lluvia. Aunque menos resistente al viento que un techo a cuatro aguas, sigue siendo muy adecuado.

- **Mansard (De doble inclinación):** Proporciona espacio adicional en el ático, pero su inclinación superior menos pronunciada puede acumular algo de nieve.

- **Gambrel (Techo granero):** Espacioso y estéticamente adecuado para áreas rurales cercanas, aunque menos resistente al viento y con posibles problemas de acumulación de nieve.

- **Shed (Techo inclinado):** Diseño simple y moderno, pero menos eficiente para drenaje en comparación con techos a dos o cuatro aguas en áreas con nevadas significativas.

- **Flat (Plano):** Económico, pero altamente susceptible a problemas de drenaje en inviernos con nevadas intensas, requiriendo mantenimiento frecuente.

Probablemente, debido a la escasa variabilidad de los datos, lo que sería mejor no corresponde con el precio promedio en los datos de entrenamiento.

In [122]:
resultado = analizar_precio_viviendas_por_variable(df, 'RoofStyle')
resultado

,RoofStyle,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
5,Shed,225000.000000,5,0.171292,2,3
3,Hip,218876.933566,551,18.876328,286,265
0,Flat,194690.000000,20,0.685166,13,7
4,Mansard,180568.428571,11,0.376841,7,4
1,Gable,171483.956179,2310,79.136691,1141,1169
2,Gambrel,148909.090909,22,0.753683,11,11


Hay una diferencia significativa en el precio promedio de las casas según el estilo del techo. El tipo de techo Shed tiene el precio promedio más alto, seguido de Hip, mientras que Gambrel tiene el precio promedio más bajo. Esto sugiere que el estilo del techo puede tener un impacto en el precio de la vivienda.

Hay una gran variación en el número de viviendas para cada estilo. Gable tiene una cantidad mucho mayor (2310) en comparación con otros estilos como Shed (5) o Mansard (11). Este desbalance puede influir en la codificación, ya que los estilos con menos muestras podrían introducir ruido en el modelo.

In [123]:
mapeo_RoofStyle = {
    'Gable': 'A dos aguas',
    'Hip': 'A cuatro aguas',
    'Flat': 'Plano',
    'Mansard': 'Doble inclinación',
    'Gambrel': 'Techo granero',
    'Shed': 'Techo inclinado'
}
df['RoofStyle'] = df['RoofStyle'].map(mapeo_RoofStyle)

In [124]:
distribucion_target_con_variable_vertical(df, 'SalePrice', 'RoofStyle', title="Impacto del Tipo de Techo en el Precio de Venta")

23. [RoofMatl](../docs/descripcion_variables.md#variable-roofmatl): Material del techo.

Para el clima de Iowa, que incluye condiciones extremas como inviernos fríos, nevadas frecuentes, cambios bruscos de temperatura y ocasionalmente tormentas severas, algunos tipos de teja son más adecuados que otros.

- **Metal:** Ideal para el clima de Iowa debido a su durabilidad extrema y resistencia a la nieve, hielo y viento. También ofrece buena resistencia al fuego. Puede soportar las inclemencias del tiempo durante largos periodos.

- **ClyTile:** La teja de arcilla es una excelente opción para climas fríos y nevados, ya que ofrece gran resistencia al agua y bajas temperaturas. Es duradera y requiere poco mantenimiento.

- **CompShg:** Las tejas compuestas son muy resistentes y ofrecen buena protección contra los elementos. Su durabilidad y aislamiento térmico son adecuados para el clima de Iowa.

- **Tar&Grv:** Aunque más económica, las tejas de asfalto y grava pueden soportar el clima de Iowa, pero su durabilidad es menor comparada con otras opciones. Requiere mantenimiento periódico.

- **WdShngl:** Las tejas de madera funcionan bien en Iowa si se mantienen adecuadamente. Sin embargo, requieren mayor cuidado, especialmente en invierno y en condiciones húmedas.

- **WdShake:** Similar a **WdShngl**, pero menos uniforme y menos resistente al clima extremo de Iowa.

- **Membran:** Ideal para techos planos o casi planos, pero en general, no es muy común en climas fríos. Su durabilidad es baja en comparación con otros tipos.

- **Roll:** Menos recomendable para climas como el de Iowa debido a su corta duración y funcionalidad limitada.

Probablemente, debido a la escasa variabilidad de los datos, lo que sería mejor no corresponde con el precio promedio en los datos de entrenamiento.

In [125]:
resultado = analizar_precio_viviendas_por_variable(df, 'RoofMatl')
resultado

,RoofMatl,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
7,WdShngl,390250.000000,7,0.239808,6,1
2,Membran,241500.000000,1,0.034258,1,0
6,WdShake,241400.000000,9,0.308325,5,4
5,Tar&Grv,185406.363636,23,0.787941,11,12
3,Metal,180000.000000,1,0.034258,1,0
1,CompShg,179803.679219,2876,98.526893,1434,1442
0,ClyTile,160000.000000,1,0.034258,1,0
4,Roll,137000.000000,1,0.034258,1,0


En el conjunto de datos, se observó que algunas categorías de la columna **RoofMatl** solo están presentes en el conjunto de entrenamiento y no en el conjunto de prueba. Estas categorías son: Metal, Membran, Roll, ClyTile.
Si los valores faltantes en la columna **RoofMatl** del conjunto de prueba se reemplazaran por la moda, casi todos los datos adoptarían el valor `"CompShg"`. Aunque la mayoría de las casas tienen **teja compuesta** como material predominante, los materiales de techo diferentes tienen una influencia significativa en el precio.

Para simplificar la codificación y mejorar la representatividad en el modelo, se establecieron dos categorías según la calidad del material del techo como alta y baja, pero al ser el 98% de los datos de Alta Calidad no se representaba bien que unos materiales fueran peor que otros, por lo que se busca otra forma de clasificarlos

In [126]:
mapeo_RoofMatl = {
    "CompShg": "Techo Tradicional",
    "ClyTile": "Techo Tradicional",
    "Membran": "Techo Moderno",
    "Metal": "Techo Moderno",
    "Roll": "Techo Básico",
    "Tar&Grv": "Techo Básico",
    "WdShngl": "Techo Madera",
    "WdShake": "Techo Madera"
}

df['RoofMatl']= df['RoofMatl'].map(mapeo_RoofMatl)

In [127]:
resultado = analizar_precio_viviendas_por_variable(df, 'RoofMatl')
resultado

,RoofMatl,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
1,Techo Madera,322590.909091,16,0.548133,11,5
2,Techo Moderno,210750.000000,2,0.068517,2,0
0,Techo Básico,181372.500000,24,0.822199,12,12
3,Techo Tradicional,179789.878746,2877,98.561151,1435,1442


In [128]:
distribucion_target_con_variable(df, 'SalePrice', 'RoofMatl', title="Impacto del Material de Techo en el Precio de Venta")

Se probaron distintas técnicas para codificar las variables **RoofMatl** y **RoofStyle** con el objetivo de hacerlas más representativas en el modelo. 

Primero se probó Asignación de Puntuaciones: **Material del Techo (RoofMatl):** Se asignaron puntuaciones más altas a los materiales que se considerarían más adecuados para el clima de Iowa, priorizando aquellos que ofrecen mayor durabilidad y menor costo. **Estilo del Techo (RoofStyle):** Aunque menos relevante que el material, se otorgó un peso moderado basado en el diseño general del inmueble y su impacto estético. Esta técnica no resultó efectiva. 

- En el caso de **RoofMatl**, se identificaron muchas categorías presentes en el conjunto de entrenamiento pero no en el de prueba, se reducen las categorías.  
  
- En **RoofStyle** sí que se establece una jerarquía, y se comprueba que mejora la información al estar más correlacionada con **RoofMatl**.

- Se creó una variable que combinaba **RoofMatl** y **RoofStyle**, asignando una puntuación que reflejaba la calidad general del techo considerando ambos factores, que se borró finalmente al no aportar información extra.  

Sin embargo, a pesar de estos esfuerzos, las variables mostraron una baja variabilidad en los datos, lo que disminuyó su impacto en el modelo. Por lo tanto, se determinó que era mejor eliminarlas.

In [129]:
combinaciones_unicas = df.groupby(['RoofStyle', 'RoofMatl']).size().reset_index(name='Count')
combinaciones_unicas

,RoofStyle,RoofMatl,Count
0,A cuatro aguas,Techo Madera,6
1,A cuatro aguas,Techo Tradicional,545
2,A dos aguas,Techo Básico,7
3,A dos aguas,Techo Madera,5
4,A dos aguas,Techo Tradicional,2298
5,Doble inclinación,Techo Madera,3
6,Doble inclinación,Techo Tradicional,8
7,Plano,Techo Básico,17
8,Plano,Techo Moderno,2
9,Plano,Techo Tradicional,1


In [130]:
categorias_RoofStyle = ["A cuatro aguas", "A dos aguas", "Doble inclinación", "Techo granero", "Techo inclinado", "Plano"]

df = aplicar_codificacion_ordinal_especifica(df, 'RoofStyle', categorias_RoofStyle)

In [131]:
codificacion_ponderada(df, 'RoofMatl', 'SalePrice')

24. [Exterior1st](../docs/descripcion_variables.md#variable-exterior1st): Recubrimiento exterior de la casa.
25. [Exterior2nd](../docs/descripcion_variables.md#variable-exterior2nd): Recubrimiento exterior adicional.

In [132]:
resultado = analizar_precio_viviendas_por_variable(df, 'Exterior1st')
resultado

,Exterior1st,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
7,ImStucc,262000.000000,1,0.034258,1,0
10,Stone,258500.000000,2,0.068517,2,0
5,CemntBd,231690.655738,126,4.316547,61,65
12,VinylSd,213732.900971,1025,35.114765,515,510
3,BrkFace,194573.000000,87,2.980473,50,37
9,Plywood,175942.379630,221,7.571086,108,113
6,HdBoard,163077.450450,442,15.142172,222,220
11,Stucco,162990.000000,43,1.473107,25,18
14,WdShing,150655.076923,56,1.918465,26,30
13,Wd Sdng,149841.645631,411,14.080164,206,205


In [133]:
resultado = analizar_precio_viviendas_por_variable(df, 'Exterior2nd')
resultado

,Exterior2nd,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
9,Other,319000.000000,1,0.034258,1,0
7,ImStucc,252070.000000,15,0.513875,10,5
5,CmentBd,230093.833333,126,4.316547,60,66
13,VinylSd,214432.460317,1014,34.737924,504,510
3,BrkFace,195818.000000,47,1.610140,25,22
10,Plywood,168112.387324,270,9.249743,142,128
6,HdBoard,167661.565217,406,13.908873,207,199
15,Wd Shng,161328.947368,81,2.774923,38,43
11,Stone,158224.800000,6,0.205550,5,1
12,Stucco,155905.153846,47,1.610140,26,21


In [134]:
calcular_porcentaje_coincidencias(df, 'Exterior1st', 'Exterior2nd') 

"El porcentaje de coincidencias entre 'Exterior1st' y 'Exterior2nd' es de 84.99%."

In [135]:
# Localizo la unica fila que tiene en ambas columnas NaN
fila_nan_exterior1st = df[df['Exterior1st'].isna()]
fila_filtrada = fila_nan_exterior1st[['Exterior1st', 'Exterior2nd', 'Dataset']]
fila_filtrada

,Exterior1st,Exterior2nd,Dataset
2151,NaN,NaN,test


In [136]:
# Como solo tienen un valor faltante ambas, les aplico la moda al valor faltante
moda_exterior1st = df[df['Dataset'] == 'train']['Exterior1st'].mode()[0]
moda_exterior2nd = df[df['Dataset'] == 'train']['Exterior2nd'].mode()[0]

df['Exterior1st'].fillna(moda_exterior1st, inplace=True)
df['Exterior2nd'].fillna(moda_exterior2nd, inplace=True)

In [137]:
exterior1st = df['Exterior1st'].unique().tolist()
exterior2nd = df['Exterior2nd'].unique().tolist()

diferencia_exterior1st = list(set(exterior1st) - set(exterior2nd))
diferencia_exterior2nd = list(set(exterior2nd) - set(exterior1st))

print(sorted(diferencia_exterior1st))
print(sorted(diferencia_exterior2nd))

['BrkComm', 'CemntBd', 'WdShing']
['Brk Cmn', 'CmentBd', 'Other', 'Wd Shng']


In [138]:
fila_other = df[df['Exterior2nd'] == 'Other']
fila_other_filtrada = fila_other[['Exterior1st', 'Exterior2nd']]
fila_other_filtrada

,Exterior1st,Exterior2nd
595,VinylSd,Other


Reemplazar valores en la columna Exterior2nd que son los mismos que Exterior1st pero están nombrados de forma diferente

In [139]:
mapeo = {
    'Other': 'VinylSd',
    'Brk Cmn': 'BrkComm',
    'CmentBd': 'CemntBd',
    'Wd Shng': 'WdShing'
}

df['Exterior2nd'] = df['Exterior2nd'].replace(mapeo)

In [140]:
calcular_porcentaje_coincidencias(df, 'Exterior1st', 'Exterior2nd') 

"El porcentaje de coincidencias entre 'Exterior1st' y 'Exterior2nd' es de 90.85%."

Como Exterior1st tiene en train pero no en test las variables ImStucc, Stone y tienen ambas columnas más de un 90% de coincidencia, decido quedarme solo con la columna Exterior2nd

In [141]:
df.drop(columns=['Exterior1st'], inplace=True)

Para que se entienda más la variable Exterior2nd se clasifican los materiales utilizados para el recubrimiento exterior de las casas. El modelo entiende mejor la variable con menos opciones, ya que algunas estaban infrarepresentadas.

- Grupo 1: Materiales tradicionales (Ladrillo, Estuco, Piedra)

- Grupo 2: Materiales de madera (Revestimiento de madera, Tejas de madera, Tablero duro)

- Grupo 3: Materiales más modernos (Tablero de cemento, Revestimiento metálico, Tejas de asfalto, Tejas de asbesto)

- Grupo 4: Bloques y revestimientos básicos (Bloque de concreto, Estuco sintético)

In [142]:
mapeo_Exterior2nd = {
    "AsbShng": "Tradicionales",
    "AsphShn": "Tradicionales",
    "BrkComm": "Tradicionales",
    "BrkFace": "Tradicionales",
    "CBlock": "Básicos",
    "CemntBd": "Modernos",
    "HdBoard": "Modernos",
    "ImStucc": "Tradicionales",
    "MetalSd": "Modernos",
    "Other": "Tradicionales",
    "Plywood": "Madera",
    "PreCast": "Modernos",
    "Stone": "Tradicionales",
    "Stucco": "Tradicionales",
    "VinylSd": "Tradicionales",
    "Wd Sdng": "Madera",
    "WdShing": "Madera",
}

df["Exterior2nd"] = df["Exterior2nd"].map(mapeo_Exterior2nd)

In [143]:
resultado = analizar_precio_viviendas_por_variable(df, 'Exterior2nd')
resultado

,Exterior2nd,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
3,Tradicionales,206715.439268,1195,40.938678,601,594
2,Modernos,167504.060291,979,33.538883,481,498
1,Madera,157120.726790,742,25.419664,377,365
0,Básicos,105000.000000,3,0.102775,1,2


In [144]:
codificacion_ponderada(df, 'Exterior2nd', 'SalePrice')

26. [MasVnrType](../docs/descripcion_variables.md#variable-masvnrtype): Tipo de revestimiento de mampostería.

In [145]:
resultado = analizar_precio_viviendas_por_variable(df, 'MasVnrType')
resultado

,MasVnrType,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
2,Stone,265583.625000,249,8.530319,128,121
1,BrkFace,204691.871910,879,30.113052,445,434
0,BrkCmn,146318.066667,25,0.856458,15,10


In [146]:
total_filas = len(df)

nan_filas = df['MasVnrType'].isna().sum()

porcentaje_nan = (nan_filas / total_filas) * 100
print(f"El porcentaje de NaN en 'MasVnrType' es: {porcentaje_nan:.2f}%")

El porcentaje de NaN en 'MasVnrType' es: 60.50%


In [147]:
resultado = analizar_precio_viviendas_por_variable(df, 'MasVnrType')
resultado

,MasVnrType,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
2,Stone,265583.625000,249,8.530319,128,121
1,BrkFace,204691.871910,879,30.113052,445,434
0,BrkCmn,146318.066667,25,0.856458,15,10


Se filtran las filas que MasVnrArea no sea 0 ni NaN para rellenar NaN de MasVnrType. 
Como el Tipo de revestimiento de mampostería utilizado depende del año de construcción también, lo filtro para buscar la moda más exacta.

In [148]:
nan_filas = df['MasVnrType'].isna().sum()
nan_filas

1766

In [149]:
columnas_rellenar_MasVnrType = df[(df['MasVnrArea'] != 0) & (df['MasVnrArea'].notna()) & (df['MasVnrType'].isna())]
print(len(columnas_rellenar_MasVnrType))
columnas_rellenar_MasVnrType[["Id", "MasVnrType", "MasVnrArea", "YearBuilt", "Dataset"]]

8


,Id,MasVnrType,MasVnrArea,YearBuilt,Dataset
624,625,NaN,288.0,1972,train
773,774,NaN,1.0,1958,train
1230,1231,NaN,1.0,1977,train
1300,1301,NaN,344.0,1999,train
1334,1335,NaN,312.0,1970,train
1669,1670,NaN,285.0,2008,test
2452,2453,NaN,1.0,1956,test
2610,2611,NaN,198.0,1961,test


In [150]:
def rellenar_moda_MasVnrType(df, año, id):
    # Filtrar solo 'train' para calcular la moda del MasVnrType en el año específico
    moda_MasVnrType_año = df[(df['Dataset'] == 'train') & (df['YearBuilt'] == año)]['MasVnrType'].mode()[0]

    # Reemplazar NaN en todo el DataFrame, pero solo en el ID específico con la moda del año específico
    df.loc[(df['Id'] == id) & (df['MasVnrType'].isna()), 'MasVnrType'] = moda_MasVnrType_año
    
    print(f"La Moda del año {año} para el ID {id} es: {moda_MasVnrType_año}")

Aplicar la función solo a los IDs específicos

In [151]:
rellenar_moda_MasVnrType(df, 1972, 625)
rellenar_moda_MasVnrType(df, 1958, 774)
rellenar_moda_MasVnrType(df, 1977, 1231)
rellenar_moda_MasVnrType(df, 1999, 1301)
rellenar_moda_MasVnrType(df, 1970, 1335)
rellenar_moda_MasVnrType(df, 2008, 1670)
rellenar_moda_MasVnrType(df, 1956, 2453)
rellenar_moda_MasVnrType(df, 1961, 2611)

La Moda del año 1972 para el ID 625 es: BrkFace
La Moda del año 1958 para el ID 774 es: BrkFace
La Moda del año 1977 para el ID 1231 es: BrkFace
La Moda del año 1999 para el ID 1301 es: BrkFace
La Moda del año 1970 para el ID 1335 es: BrkFace
La Moda del año 2008 para el ID 1670 es: Stone
La Moda del año 1956 para el ID 2453 es: BrkFace
La Moda del año 1961 para el ID 2611 es: BrkCmn


In [152]:
# reemplazo los nan por otro
df['MasVnrType'].fillna('Sin Mamposteria', inplace=True)

Se simplifican las categorías similares agrupando las clases BrkFace y BrkCmn.

In [153]:
diccionario_MasVnrType = {
    'BrkFace': 'Ladrillo',
    'Stone': 'Piedra',
    'BrkCmn': 'Ladrillo',
    'Sin Mamposteria': 'Sin Mamposteria'
}

df['MasVnrType'] = df['MasVnrType'].map(diccionario_MasVnrType)

In [154]:
resultado = analizar_precio_viviendas_por_variable(df, 'MasVnrType')
resultado

,MasVnrType,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
1,Piedra,265583.625000,250,8.564577,128,122
0,Ladrillo,202370.546237,911,31.209318,465,446
2,Sin Mamposteria,156918.036909,1758,60.226105,867,891


27. [MasVnrArea](../docs/descripcion_variables.md#variable-masvnrarea): Área de revestimiento de mampostería en pies cuadrados.

In [155]:
# Rellenar NaN en MasVnrArea con 0 solo donde MasVnrType sea 'Sin Mamposteria'

df['MasVnrArea'] = df.apply(lambda x: 0 if x['MasVnrType'] == 'Sin Mamposteria' and pd.isna(x['MasVnrArea']) else x['MasVnrArea'], axis=1)

In [156]:
#Contar filas donde MasVnrType es 'Sin Mamposteria' y MasVnrArea es diferente de 0
condicion_masvnrtype = df['MasVnrType'] == 'Sin Mamposteria'
condicion_masvnrarea = df['MasVnrArea'] != 0
count = len(df[condicion_masvnrtype & condicion_masvnrarea])
print(count)

0


In [157]:
nan_count = df['MasVnrArea'].isna().sum()
print(f"Número de filas con MasVnrArea NaN: {nan_count}")

Número de filas con MasVnrArea NaN: 0


In [158]:
describe_train_test(df, 'MasVnrArea')

,Train,Test
count,1460.000000,1459.000000
mean,103.117123,99.673749
std,180.731373,177.001792
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,164.250000,162.000000
max,1600.000000,1290.000000


In [159]:
df_area= df[["MasVnrArea", "SalePrice"]]
df_area.describe()

,MasVnrArea,SalePrice
count,2919.000000,1460.000000
mean,101.396026,180921.195890
std,178.854579,79442.502883
min,0.000000,34900.000000
25%,0.000000,129975.000000
50%,0.000000,163000.000000
75%,163.500000,214000.000000
max,1600.000000,755000.000000


In [160]:
# La mayoría de las casas en el conjunto de datos no tienen revestimiento de mampostería 

total_registros = df['MasVnrArea'].count()  # Este cuenta los valores no nulos

# Contar los registros que tienen un valor de 0
contador_ceros = (df['MasVnrArea'] == 0).sum()

# Calcular el porcentaje
porcentaje_ceros = (contador_ceros / total_registros) * 100
print(f"El porcentaje de registros con MasVnrArea igual a 0 es: {porcentaje_ceros:.2f}%")

El porcentaje de registros con MasVnrArea igual a 0 es: 60.33%


In [161]:
boxplot_train_test(df, 'MasVnrArea', 'Dataset', title="Distribución de MasVnrArea")

Ahora tanto MasVnrType como MasVnrArea tienen representación en el modelo.

In [162]:
# CODIFICO AQUÍ MasVnrType PORQUE ME RESULTA DE AYUDA CON los NaN de MasVnrArea
codificacion_ponderada(df, 'MasVnrType', 'SalePrice')

28. [ExterQual](../docs/descripcion_variables.md#variable-exterqual): Calidad del material exterior.
29. [ExterCond](../docs/descripcion_variables.md#variable-extercond): Condición actual del material en el exterior.

In [163]:
resultado = analizar_precio_viviendas_por_variable(df, 'ExterQual')
resultado

,ExterQual,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
0,Ex,367360.961538,107,3.665639,52,55
2,Gd,231633.510246,979,33.538883,488,491
3,TA,144341.313466,1798,61.596437,906,892
1,Fa,87985.214286,35,1.199041,14,21


In [164]:
resultado = analizar_precio_viviendas_por_variable(df, 'ExterCond')
resultado

,ExterCond,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
0,Ex,201333.333333,12,0.411100,3,9
4,TA,184034.896256,2538,86.947585,1282,1256
2,Gd,168897.568493,299,10.243234,146,153
1,Fa,102595.142857,67,2.295307,28,39
3,Po,76500.000000,3,0.102775,1,2


In [165]:
calcular_porcentaje_coincidencias(df, 'ExterQual', 'ExterCond') 

"El porcentaje de coincidencias entre 'ExterQual' y 'ExterCond' es de 55.50%."

In [166]:
escala_calidad = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NoAplica': 0}
df['ExterQual'] = df['ExterQual'].map(escala_calidad)
df['ExterCond'] = df['ExterCond'].map(escala_calidad)

Teniendo en cuenta que ExterQual se refiere a la calidad del material exterior con el que se construyó la casa y ExterCond la condición actual del material en el exterior, se calculan cuántas filas tienen una ExterCond mejor.

Se intenta crear una nueva variable que refleje si el estado actual mejora, se mantiene o empeora respecto a la calidad original, pero empeora el modelo.

In [167]:
mejores_extercond = df[df['ExterCond'] > df['ExterQual']]
print(mejores_extercond.shape[0])

244


In [168]:
columnas_a_analizar = ["SalePrice", "RoofMatl", "RoofStyle", "Exterior2nd", "MasVnrType", "MasVnrArea", 
                           "ExterQual", "ExterCond"]

visualizar_correlaciones(df, columnas_a_analizar)

### 5. **Características del Sótano**
> **Objetivo**: Evaluar la calidad y el uso del espacio del sótano.

30. [Foundation](../docs/descripcion_variables.md#variable-foundation): Tipo de cimentación.
31. [BsmtQual](../docs/descripcion_variables.md#variable-bsmtqual): Altura del sótano.
32. [BsmtCond](../docs/descripcion_variables.md#variable-bsmtcond): Condición general del sótano.
33. [BsmtExposure](../docs/descripcion_variables.md#variable-bsmtexposure): Exposición del sótano.
34. [BsmtFinType1](../docs/descripcion_variables.md#variable-bsmtfintype1): Calidad del área terminada del sótano.
35. [BsmtFinSF1](../docs/descripcion_variables.md#variable-bsmtfinsf1): Pies cuadrados terminados del tipo 1.
36. [BsmtFinType2](../docs/descripcion_variables.md#variable-bsmtfintype2): Calidad del segundo área terminada.
37. [BsmtFinSF2](../docs/descripcion_variables.md#variable-bsmtfinsf2): Pies cuadrados terminados del tipo 2.
38. [BsmtUnfSF](../docs/descripcion_variables.md#variable-bsmtunfsf): Pies cuadrados sin terminar del área del sótano.
39. [TotalBsmtSF](../docs/descripcion_variables.md#variable-totalbsmtsf): Pies cuadrados totales del área del sótano.

In [169]:
caracteristicas_sotano = ["Foundation", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinSF1", 
                           "BsmtFinType2", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF"]

df[["SalePrice"] + caracteristicas_sotano].head()

,SalePrice,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF
0,208500.0,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0
1,181500.0,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0
2,223500.0,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0
3,140000.0,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0
4,250000.0,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0


In [170]:
resumen = resumen_columnas(df, caracteristicas_sotano)
resumen

,Tipo de Dato,Nº Valores Únicos,Nº de NaN,Porcentaje NaN Train,Porcentaje NaN Test,Valores Únicos Train,Valores Únicos Test
Foundation,object,6,0,0.000000,0.000000,[],[]
BsmtQual,object,4,81,2.534247,3.015764,[],[]
BsmtCond,object,4,82,2.534247,3.084304,[],[]
BsmtExposure,object,4,82,2.602740,3.015764,[],[]
BsmtFinType1,object,6,79,2.534247,2.878684,[],[]
BsmtFinSF1,float64,991,1,0.000000,0.068540,[],[]
BsmtFinType2,object,6,80,2.602740,2.878684,[],[]
BsmtFinSF2,float64,272,1,0.000000,0.068540,[],[]
BsmtUnfSF,float64,1135,1,0.000000,0.068540,[],[]
TotalBsmtSF,float64,1058,1,0.000000,0.068540,[],[]


In [171]:
valores_unicos = obtener_valores_unicos(df, caracteristicas_sotano)
for key, value in valores_unicos.items():
    print(f"'{key}': {value},")


'Foundation': ['BrkTil', 'CBlock', 'PConc', 'Slab', 'Stone', 'Wood'],
'BsmtQual': ['Ex', 'Fa', 'Gd', 'TA'],
'BsmtCond': ['Fa', 'Gd', 'Po', 'TA'],
'BsmtExposure': ['Av', 'Gd', 'Mn', 'No'],
'BsmtFinType1': ['ALQ', 'BLQ', 'GLQ', 'LwQ', 'Rec', 'Unf'],
'BsmtFinSF1': [0.0, 2.0, 16.0, 20.0, 24.0, 25.0, 27.0, 28.0, 32.0, 33.0, 35.0, 36.0, 40.0, 41.0, 42.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 60.0, 63.0, 64.0, 65.0, 68.0, 70.0, 72.0, 73.0, 75.0, 76.0, 77.0, 78.0, 80.0, 81.0, 85.0, 88.0, 94.0, 96.0, 100.0, 104.0, 108.0, 110.0, 111.0, 113.0, 114.0, 116.0, 119.0, 120.0, 121.0, 122.0, 125.0, 126.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 138.0, 140.0, 141.0, 143.0, 144.0, 148.0, 149.0, 150.0, 152.0, 154.0, 155.0, 156.0, 162.0, 165.0, 167.0, 168.0, 169.0, 170.0, 172.0, 173.0, 175.0, 176.0, 179.0, 180.0, 181.0, 182.0, 185.0, 186.0, 187.0, 188.0, 189.0, 190.0, 191.0, 192.0, 193.0, 194.0, 196.0, 197.0, 198.0, 200.0, 201.0, 203.0, 204.0, 205.0, 206.0, 207.0, 208.0, 209.0

39. [TotalBsmtSF](../docs/descripcion_variables.md#variable-totalbsmtsf): Pies cuadrados totales del área del sótano.

In [172]:
describe_train_test(df, 'TotalBsmtSF')

,Train,Test
count,1460.000000,1458.000000
mean,1057.429452,1046.117970
std,438.705324,442.898624
min,0.000000,0.000000
25%,795.750000,784.000000
50%,991.500000,988.000000
75%,1298.250000,1305.000000
max,6110.000000,5095.000000


In [173]:
df['TotalBsmtSF'].isna().sum()

1

Para rellenar los `NaN` en el resto de las columnas, se comienza analizando el único `NaN` en `TotalBsmtSF`, que representa los pies cuadrados del sótano. Al filtrar las columnas relacionadas, se observa que en esa fila el único valor disponible es `Foundation`, cuyo valor es `PConc`. `PConc` (Concreto) está frecuentemente asociado con casas que tienen sótanos, ya que el concreto es común en la construcción de sótanos. 

Por esta razón, se decide rellenar el valor de `TotalBsmtSF` utilizando la mediana de las casas que tienen sótano.

In [174]:
columnas_relacionadas = ["BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinSF1",  
                         "BsmtFinType2", "BsmtFinSF2", "BsmtUnfSF", "Dataset", "Foundation"]

filas_nan = df[df['TotalBsmtSF'].isna()][columnas_relacionadas]
filas_nan

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,Dataset,Foundation
2120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test,PConc


### Calcular la mediana en train solo de los valores mayores a 0 en 'TotalBsmtSF'

Se decide no eliminar la fila ya que es del conjunto de test. Antes de imputar el valor faltante, se calcula la mediana de `TotalBsmtSF` utilizando únicamente los valores mayores a 0 en train. 

In [175]:
mediana_totalbsmt = df[df['Dataset'] == 'train']['TotalBsmtSF'][df['TotalBsmtSF'] > 0].median()
print(mediana_totalbsmt)

df['TotalBsmtSF'].fillna(mediana_totalbsmt, inplace=True)

1004.0


In [176]:
fig = px.scatter(df, x="TotalBsmtSF", y="SalePrice",
                 title="Gráfico de Dispersión de Precio de Venta vs. Área del Sótano",
                 labels={"SalePrice": "Precio de Venta (USD)", "TotalBsmtSF": "Área Total del Sótano (pies cuadrados)"},
                 trendline="ols")

fig.update_layout(title_x=0.5)
fig.show()

Cantidad de casas con y sin sótano

In [177]:
sin_sotano = df[df['TotalBsmtSF'] == 0]
con_sotano = df[df['TotalBsmtSF'] > 0]

print(f"Cantidad de casas sin sótano (TotalBsmtSF = 0): {len(sin_sotano)}")
print(f"Cantidad de casas con sótano (TotalBsmtSF > 0): {len(con_sotano)}")

Cantidad de casas sin sótano (TotalBsmtSF = 0): 78
Cantidad de casas con sótano (TotalBsmtSF > 0): 2841


Distribución de las casas con sótano en pies cuadrados totales del área del sótano.  

In [178]:
fig = px.histogram(con_sotano, 
                   x='TotalBsmtSF', 
                   title='Distribución de TotalBsmtSF para casas con sótano',
                   labels={'TotalBsmtSF': 'TotalBsmtSF (Pies Cuadrados)'},
                   template='plotly_white')

fig.update_layout(xaxis_title='TotalBsmtSF (Pies Cuadrados)', 
                  yaxis_title='Frecuencia', title_x=0.5)
fig.show()

Se comprueban las estadísticas descriptivas para el rango menor, excluyendo los 0, para ver si tiene sentido el valor mínimo, el sótano más pequeño es de 105.000000 pies cuadrados que equivaldría a aproximadamente 9.75 metros cuadrados, por lo que el valor mínimo tienen sentido.

In [179]:
rango_pequeno_sotano = df[(df['TotalBsmtSF'] > 0) & (df['TotalBsmtSF'] <= 499)]

print("Estadísticas descriptivas para TotalBsmtSF en el rango de 1 a 499 pies cuadrados:")
print(rango_pequeno_sotano['TotalBsmtSF'].describe())

Estadísticas descriptivas para TotalBsmtSF en el rango de 1 a 499 pies cuadrados:
count     97.000000
mean     389.938144
std       89.036605
min      105.000000
25%      360.000000
50%      392.000000
75%      462.000000
max      498.000000
Name: TotalBsmtSF, dtype: float64


### Creación de una nueva variable

Para determinar si tiene sótano o no "TieneSotano", con los pies cuadrados totales del sótano. Se comprueba que no hay NaN en TotalBsmtSF antes de crear la columna.


In [180]:
nan_total = df['TotalBsmtSF'].isna().sum()
print(f"Hay {nan_total} valores nulos en la columna 'TotalBsmtSF'.")

Hay 0 valores nulos en la columna 'TotalBsmtSF'.


In [181]:
df['TieneSotano'] = df['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)

In [182]:
fig = px.histogram(
    df,
    y='TieneSotano',
    color='Dataset',
    barmode='group',
    opacity=0.7, 
    title='Distribución de TieneSotano en Train y Test',
    labels={'TieneSotano': 'Tiene Sótano', 'count': 'Cantidad'},
    
)
fig.update_layout(title_x=0.5)
fig.show()

### Rellenar atributos categóricos y numéricos de sótano

En este código, se utiliza una función llamada `rellenar_atributos_sotano` para reemplazar los valores nulos en las columnas de atributos del sótano cuando la casa no tiene sótano (`TieneSotano == 0`). La idea es que si una casa no tiene sótano, no se pueden asignar categorías como "bueno", "malo" o cualquier otra relacionada con las características del sótano, ya que estas no aplican. Por eso, en esos casos se reemplaza con el valor `'NoAplica'`.

La función se asegura de que los valores sean consistentes con los atributos relacionados con el sótano. Busca inconsistencias al revisar todas las filas donde `TieneSotano == 0`. En estas filas, los atributos relacionados deberían estar rellenados como `'NoAplica'`. De esta manera, se asegura que no haya valores no nulos que puedan afectar o distorsionar los análisis.

Se utiliza la misma función para rellenarlos atributos numéricos, se imputan los valores nulos con `0` en lugar de `'NoAplica'` cuando `TieneSotano = 0`.

No se detectan inconsistencias en los atributos numéricos ni categóricos.

Se rellenan los atributos categóricos

In [183]:
rellenar_atributos_sotano(df, 'BsmtQual') 
rellenar_atributos_sotano(df, 'BsmtExposure') 
rellenar_atributos_sotano(df, 'BsmtCond') 
rellenar_atributos_sotano(df, 'BsmtFinType1') 
rellenar_atributos_sotano(df, 'BsmtFinType2') 

Se rellenan los atributos numéricos

In [184]:
rellenar_atributos_sotano(df, 'BsmtFinSF1') 
rellenar_atributos_sotano(df, 'BsmtFinSF2') 
rellenar_atributos_sotano(df, 'BsmtUnfSF') 

Se comprueba que muchos de los `NaN` correspondían a casas que **no tienen sótano** (`TieneSotano == 0`). Aunque este proceso redujo considerablemente el número de `NaN` en las columnas relacionadas con atributos del sótano, todavía quedan algunos valores faltantes. 

Estos valores serán tratados de manera individual para garantizar su correcta gestión y evitar posibles inconsistencias en el dataset.

In [185]:
resumen = resumen_columnas(df, caracteristicas_sotano + ['TieneSotano'])
resumen

,Tipo de Dato,Nº Valores Únicos,Nº de NaN,Porcentaje NaN Train,Porcentaje NaN Test,Valores Únicos Train,Valores Únicos Test
Foundation,object,6,0,0.000000,0.00000,[],[]
BsmtQual,object,5,3,0.000000,0.20562,[],[]
BsmtCond,object,5,4,0.000000,0.27416,[],[]
BsmtExposure,object,5,4,0.068493,0.20562,[],[]
BsmtFinType1,object,7,1,0.000000,0.06854,[],[]
BsmtFinSF1,float64,991,1,0.000000,0.06854,[],[]
BsmtFinType2,object,7,2,0.068493,0.06854,[],[]
BsmtFinSF2,float64,272,1,0.000000,0.06854,[],[]
BsmtUnfSF,float64,1135,1,0.000000,0.06854,[],[]
TotalBsmtSF,float64,1058,0,0.000000,0.00000,[],[]


Como son pocas las filas que tienen `NaN`, se localizan e imprimen estas filas relacionadas con el sótano para verificar si poseen `NaN` en más de una columna y si corresponden a train o test.

In [186]:
columnas_imprimir = caracteristicas_sotano + ['Dataset', 'SalePrice', 'TieneSotano']

filas_con_nan = df[df[caracteristicas_sotano].isna().any(axis=1)]
filas_con_nan[columnas_imprimir]

,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Dataset,SalePrice,TieneSotano
332,PConc,Gd,TA,No,GLQ,1124.0,NaN,479.0,1603.0,3206.0,train,284000.0,1
948,PConc,Gd,TA,NaN,Unf,0.0,Unf,0.0,936.0,936.0,train,192500.0,1
1487,PConc,Gd,TA,NaN,Unf,0.0,Unf,0.0,1595.0,1595.0,test,NaN,1
2040,CBlock,Gd,NaN,Mn,GLQ,1044.0,Rec,382.0,0.0,1426.0,test,NaN,1
2120,PConc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1004.0,test,NaN,1
2185,CBlock,TA,NaN,No,BLQ,1033.0,Unf,0.0,94.0,1127.0,test,NaN,1
2217,Stone,NaN,Fa,No,Unf,0.0,Unf,0.0,173.0,173.0,test,NaN,1
2218,PConc,NaN,TA,No,Unf,0.0,Unf,0.0,356.0,356.0,test,NaN,1
2348,CBlock,Gd,TA,NaN,Unf,0.0,Unf,0.0,725.0,725.0,test,NaN,1
2524,CBlock,TA,NaN,Av,ALQ,755.0,Unf,0.0,240.0,995.0,test,NaN,1


30. [Foundation](../docs/descripcion_variables.md#variable-foundation): Tipo de cimentación.

Tipo de cimentación: La columna Foundation indica el tipo de cimentación de la casa (por ejemplo, PConc, CBlock, BrkTil, etc.). Esto puede influir en la presencia, calidad y características del sótano. Por ejemplo:

- PConc (Concrete Piers): Generalmente sugiere que hay un sótano.
- CBlock (Concrete Block) o BrkTil (Brick Tile): Estos pueden también indicar la posibilidad de un sótano, aunque no siempre garantizan su existencia.

In [187]:
resultado = analizar_precio_viviendas_por_variable(df, 'Foundation')
resultado

,Foundation,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
2,PConc,225230.442040,1308,44.809866,647,661
5,Wood,185666.666667,5,0.171292,3,2
4,Stone,165959.166667,11,0.376841,6,5
1,CBlock,149805.714511,1235,42.309010,634,601
0,BrkTil,132291.075342,311,10.654334,146,165
3,Slab,107365.625000,49,1.678657,24,25


Aunque las estadísticas de las predicciones agrupando las categorías "Stone" y "Wood" en "Otro", se decide mantener el cambio porque:

- Mejora la correlación con otras variables, permitiendo que el modelo identifique patrones comunes.
- Estas categorías representaban pocos datos, lo que podía generar ruido, si se combinan reducen la dispersión. 
- Agrupar las categorías menos frecuentes ayuda a mejorar la robustez y evita el sobreajuste a categorías pequeñas.

In [188]:
df['Foundation'] = df['Foundation'].replace({'Stone': 'Otro', 'Wood': 'Otro'})

In [189]:
resultado = analizar_precio_viviendas_por_variable(df, 'Foundation')
resultado

,Foundation,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
3,PConc,225230.442040,1308,44.809866,647,661
2,Otro,172528.333333,16,0.548133,9,7
1,CBlock,149805.714511,1235,42.309010,634,601
0,BrkTil,132291.075342,311,10.654334,146,165
4,Slab,107365.625000,49,1.678657,24,25


In [190]:
codificacion_ponderada(df, 'Foundation', 'SalePrice')

31. [BsmtQual](../docs/descripcion_variables.md#variable-bsmtqual): Altura del sótano.

In [191]:
resultado = analizar_precio_viviendas_por_variable(df, 'BsmtQual')
resultado

,BsmtQual,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
0,Ex,327041.041322,258,8.838643,121,137
2,Gd,202688.478964,1209,41.418294,618,591
4,TA,140759.818182,1283,43.953409,649,634
1,Fa,115692.028571,88,3.014731,35,53
3,NoAplica,105652.891892,78,2.672148,37,41


In [192]:
df['BsmtQual'].isna().sum()

3

Se rellenan los tres NaN con la moda de los que sí tienen sótano.

In [193]:
moda_BsmtQual = df[(df['TieneSotano'] == 1) & (df['Dataset'] == 'train')]['BsmtQual'].mode()[0]
print(moda_BsmtQual)

df['BsmtQual'].fillna(moda_BsmtQual, inplace=True)

TA


In [194]:
escala_calidad = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NoAplica': 0}
df['BsmtQual'] = df['BsmtQual'].map(escala_calidad)

32. [BsmtCond](../docs/descripcion_variables.md#variable-bsmtcond): Condición general del sótano.

In [195]:
resultado = analizar_precio_viviendas_por_variable(df, 'BsmtCond')
resultado

,BsmtCond,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
1,Gd,213599.907692,122,4.179514,65,57
4,TA,183632.620900,2606,89.277150,1311,1295
0,Fa,121809.533333,104,3.562864,45,59
2,NoAplica,105652.891892,78,2.672148,37,41
3,Po,64000.000000,5,0.171292,2,3


In [196]:
df['BsmtCond'].isna().sum()

4

In [197]:
moda_BsmtCond= df[(df['TieneSotano'] == 1) & (df['Dataset'] == 'train')]['BsmtCond'].mode()[0]
print(moda_BsmtCond)

df['BsmtCond'].fillna(moda_BsmtCond, inplace=True)

TA


In [198]:
df['BsmtCond'] = df['BsmtCond'].replace('Po', 'NoAplica')

### Decisión sobre la combinación de categorías 'Po' y 'NoAplica' en la columna 'BsmtCond'

Se prueba a combinar las categorías 'Po' y 'NoAplica' en una sola, ya que tener un sótano con una condición general pobre ('Po') hace que sea prácticamente equivalente a no tener sótano ('NoAplica'). Incluso, en promedio, las casas con sótanos en condición 'Po' tienen un precio inferior al de aquellas sin sótano.

Sin embargo, se decide mantener ambas categorías separadas porque las estadísticas de las predicciones no varían tras la combinación, emperona las correlaciones y eliminar esta distinción podría perjudicar el rendimiento del modelo en futuras iteraciones. Preservar ambas categorías permite al modelo contar con más información específica sobre las condiciones de los sótanos.

In [199]:
escala_calidad = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NoAplica': 0} 
df['BsmtCond'] = df['BsmtCond'].map(escala_calidad)

33. [BsmtExposure](../docs/descripcion_variables.md#variable-bsmtexposure): Exposición del sótano.

In [200]:
resultado = analizar_precio_viviendas_por_variable(df, 'BsmtExposure')
resultado

,BsmtExposure,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
1,Gd,257689.805970,276,9.455293,134,142
0,Av,206643.420814,418,14.319973,221,197
2,Mn,192789.657895,239,8.187736,114,125
3,No,165652.295908,1904,65.227818,953,951
4,NoAplica,105652.891892,78,2.672148,37,41


In [201]:
df['BsmtExposure'].isna().sum()

4

In [202]:
moda_BsmtExposure= df[(df['TieneSotano'] == 1) & (df['Dataset'] == 'train')]['BsmtExposure'].mode()[0]
print(moda_BsmtExposure)

df['BsmtExposure'].fillna(moda_BsmtExposure, inplace=True)

No


In [203]:
categorias = ["Gd", "Av", "Mn", "No", "NoAplica"]

df = aplicar_codificacion_ordinal_especifica(df, 'BsmtExposure', categorias)

34. [BsmtFinType1](../docs/descripcion_variables.md#variable-bsmtfintype1): Calidad del área terminada del sótano.

In [204]:
resultado = analizar_precio_viviendas_por_variable(df, 'BsmtFinType1')
resultado

,BsmtFinType1,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
2,GLQ,235413.720096,849,29.085303,418,431
6,Unf,170670.576744,851,29.153820,430,421
0,ALQ,161573.068182,429,14.696814,220,209
3,LwQ,151852.702703,154,5.275779,74,80
1,BLQ,149493.655405,269,9.215485,148,121
5,Rec,146889.248120,288,9.866393,133,155
4,NoAplica,105652.891892,78,2.672148,37,41


In [205]:
df['BsmtFinType1'].isna().sum()

1

In [206]:
moda_BsmtFinType1= df[(df['TieneSotano'] == 1) & (df['Dataset'] == 'train')]['BsmtFinType1'].mode()[0]
print(moda_BsmtFinType1)

df['BsmtFinType1'].fillna(moda_BsmtFinType1, inplace=True)

Unf


In [207]:
# Como esta variable es la que se queda en 0 nan voy a observar a partir de esta los nan del resto

In [208]:
# parece que que pueda ser un área recreativa no le dan gran importancia, viendo el precio promedio de las viviendas
#  prefieren que cumpla la función de sótano, por lo que la pongo antes de calidad mínima como peso de importancia en la clasificacion

categorias = ["GLQ", "ALQ", "Rec", "BLQ", "LwQ", "Unf", "NoAplica"]

df = aplicar_codificacion_ordinal_especifica(df, 'BsmtFinType1', categorias)

35. [BsmtFinSF1](../docs/descripcion_variables.md#variable-bsmtfinsf1): Pies cuadrados terminados del tipo 1.

In [209]:
describe_train_test(df, 'BsmtFinSF1')

,Train,Test
count,1460.000000,1458.000000
mean,443.639726,439.203704
std,456.098091,455.268042
min,0.000000,0.000000
25%,0.000000,0.000000
50%,383.500000,350.500000
75%,712.250000,753.500000
max,5644.000000,4010.000000


In [210]:
df['BsmtFinSF1'].isna().sum()

1

In [211]:
mediana_BsmtFinSF1 = df[(df['TieneSotano'] == 1) & (df['Dataset'] == 'train')]['BsmtFinSF1'].median()
print(mediana_BsmtFinSF1)

df['BsmtFinSF1'].fillna(mediana_BsmtFinSF1, inplace=True)

400.0


36. [BsmtFinType2](../docs/descripcion_variables.md#variable-bsmtfintype2): Calidad del segundo área terminada.

In [212]:
resultado = analizar_precio_viviendas_por_variable(df, 'BsmtFinType2')
resultado

,BsmtFinType2,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
0,ALQ,209942.105263,52,1.781432,19,33
6,Unf,184694.690287,2493,85.405961,1256,1237
2,GLQ,180982.142857,34,1.164782,14,20
5,Rec,164917.129630,105,3.597122,54,51
3,LwQ,164364.130435,87,2.980473,46,41
1,BLQ,151101.000000,68,2.329565,33,35
4,NoAplica,105652.891892,78,2.672148,37,41


In [213]:
df['BsmtFinType2'].isna().sum()

2

In [214]:
moda_BsmtFinType2= df[(df['TieneSotano'] == 1) & (df['Dataset'] == 'train')]['BsmtFinType2'].mode()[0]
print(moda_BsmtFinType2)

df['BsmtFinType2'].fillna(moda_BsmtFinType2, inplace=True)

Unf


In [215]:
categorias = ["GLQ", "ALQ", "Rec", "BLQ", "LwQ", "Unf", "NoAplica"]

df = aplicar_codificacion_ordinal_especifica(df, 'BsmtFinType2', categorias)

37. [BsmtFinSF2](../docs/descripcion_variables.md#variable-bsmtfinsf2): Pies cuadrados terminados del tipo 2.

In [216]:
describe_train_test(df, 'BsmtFinSF2')

,Train,Test
count,1460.000000,1458.000000
mean,46.549315,52.619342
std,161.319273,176.753926
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,1474.000000,1526.000000


In [217]:
df['BsmtFinSF2'].isna().sum()

1

In [218]:
mediana_BsmtFinSF2 = df[(df['TieneSotano'] == 1) & (df['Dataset'] == 'train')]['BsmtFinSF2'].median()
print(mediana_BsmtFinSF2)

df['BsmtFinSF2'].fillna(mediana_BsmtFinSF2, inplace=True)

0.0


In [219]:
# Crear el histograma de la columna BsmtFinSF2
fig = px.histogram(df, x="BsmtFinSF2", 
                   title="Distribución de BsmtFinSF2", 
                   labels={"BsmtUnfSF": "Pies cuadrados terminados del tipo 2"},)

fig.update_layout(
    xaxis_title="Pies cuadrados terminados del tipo 2 (BsmtFinSF2)",
    yaxis_title="Frecuencia",
    template="plotly_white",
    showlegend=False, title_x=0.5)

fig.show()

### Creación de una nueva columna 'NumeroSotanos' para determinar la cantidad de sótanos

Para crear la nueva columna se inicia en O y se le asigna:
- 1 si TieneSotano es 1 y BsmtFinSF2 > 0 (primer sótano)
- 2 si TieneSotano es 1 y BsmtFinSF2 > 0 (segundo sótano)

In [220]:
df['NumeroSotanos'] = 0  

df.loc[(df['TieneSotano'] == 1) & (df['BsmtFinSF1'] > 0), 'NumeroSotanos'] = 1
df.loc[(df['TieneSotano'] == 1) & (df['BsmtFinSF2'] > 0), 'NumeroSotanos'] = 2

Que tenga dos sótanos en vez de 1 no refleja que el valor promedio de las casas sea más alto. Al analizar la información de los sótanos, se identificó que hay muy pocas casas con dos sótanos, y de estas, son pocas las que tienen áreas terminadas. Eso puede explicar que tener dos sótanos no hace que suba el precio promedio.

Por este motivo, se observa que las columnas relacionadas con el segundo sótano no aportan un valor significativo para mejorar las predicciones del modelo. Se decide eliminarlas para evitar que afecten negativamente la capacidad del modelo para realizar predicciones precisas.

In [221]:
resultado = analizar_precio_viviendas_por_variable(df, 'NumeroSotanos')
resultado

,NumeroSotanos,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
1,1,191995.378935,1643,56.286399,826,817
2,2,169217.113772,347,11.887633,167,180
0,0,165519.282655,929,31.825968,467,462


38. [BsmtUnfSF](../docs/descripcion_variables.md#variable-bsmtunfsf): Pies cuadrados sin terminar del área del sótano.

In [222]:
describe_train_test(df, 'BsmtUnfSF')

,Train,Test
count,1460.000000,1458.000000
mean,567.240411,554.294925
std,441.866955,437.260486
min,0.000000,0.000000
25%,223.000000,219.250000
50%,477.500000,460.000000
75%,808.000000,797.750000
max,2336.000000,2140.000000


In [223]:
df['BsmtUnfSF'].isna().sum()

1

In [224]:
mediana_BsmtUnfSF = df[(df['TieneSotano'] == 1) & (df['Dataset'] == 'train')]['BsmtUnfSF'].median()
print(mediana_BsmtUnfSF)

df['BsmtUnfSF'].fillna(mediana_BsmtUnfSF, inplace=True)

490.0


In [225]:
# Crear el histograma de la columna BsmtUnfSF
fig = px.histogram(df, x="BsmtUnfSF", 
                   title="Distribución de BsmtUnfSF", 
                   labels={"BsmtUnfSF": "Área de sótano sin terminar (BsmtUnfSF)"},
                   color_discrete_sequence=["blue"])

fig.update_layout(
    xaxis_title="Área de sótano sin terminar (BsmtUnfSF)",
    yaxis_title="Frecuencia",
    template="plotly_white",
    title_font_size=14,
    xaxis_title_font_size=12,
    yaxis_title_font_size=12,
    showlegend=False, title_x=0.5)

fig.show()

Se incluyen las columnas resultantes del análisis del sótano, junto con las nuevas columnas creadas y las que se han mantenido, para analizar las correlaciones.

In [226]:
columnas_a_analizar = ["SalePrice", "TieneSotano", "Foundation", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinSF1", 
                           "BsmtFinType2", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "NumeroSotanos"]


visualizar_correlaciones_grandes(df, columnas_a_analizar)

### 6. **Características del Interior de la Vivienda**
> **Objetivo**: Entender cómo las características internas y el diseño del hogar impactan en el precio.

40. [Heating](../docs/descripcion_variables.md#variable-heating): Tipo de calefacción.
41. [HeatingQC](../docs/descripcion_variables.md#variable-heatingqc): Calidad y condición de la calefacción.
42. [CentralAir](../docs/descripcion_variables.md#variable-centralair): Aire acondicionado central.
43. [Electrical](../docs/descripcion_variables.md#variable-electrical): Sistema eléctrico.
44. [1stFlrSF](../docs/descripcion_variables.md#variable-1stflrsf): Pies cuadrados del primer piso.
45. [2ndFlrSF](../docs/descripcion_variables.md#variable-2ndflrsf): Pies cuadrados del segundo piso.
46. [LowQualFinSF](../docs/descripcion_variables.md#variable-lowqualfinsf): Pies cuadrados de calidad baja terminados.
47. [GrLivArea](../docs/descripcion_variables.md#variable-grlivarea): Pies cuadrados de área habitable sobre el nivel del suelo.

In [227]:
características_interior = ["Heating", "HeatingQC", "CentralAir", "Electrical", "1stFlrSF", "2ndFlrSF", 
                           "LowQualFinSF", "GrLivArea"]

df[["SalePrice"] + características_interior].head()

,SalePrice,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea
0,208500.0,GasA,Ex,Y,SBrkr,856,854,0,1710
1,181500.0,GasA,Ex,Y,SBrkr,1262,0,0,1262
2,223500.0,GasA,Ex,Y,SBrkr,920,866,0,1786
3,140000.0,GasA,Gd,Y,SBrkr,961,756,0,1717
4,250000.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198


In [228]:
resumen = resumen_columnas(df, características_interior)
resumen

,Tipo de Dato,Nº Valores Únicos,Nº de NaN,Porcentaje NaN Train,Porcentaje NaN Test,Valores Únicos Train,Valores Únicos Test
Heating,object,6,0,0.000000,0.0,"[OthW, Floor]",[]
HeatingQC,object,5,0,0.000000,0.0,[],[]
CentralAir,object,2,0,0.000000,0.0,[],[]
Electrical,object,5,1,0.068493,0.0,[Mix],[]
1stFlrSF,int64,1083,0,0.000000,0.0,[],[]
2ndFlrSF,int64,635,0,0.000000,0.0,[],[]
LowQualFinSF,int64,36,0,0.000000,0.0,[],[]
GrLivArea,int64,1292,0,0.000000,0.0,[],[]


In [229]:
valores_unicos = obtener_valores_unicos(df, características_interior)

for key, value in valores_unicos.items():
    print(f"'{key}': {value},")

'Heating': ['Floor', 'GasA', 'GasW', 'Grav', 'OthW', 'Wall'],
'HeatingQC': ['Ex', 'Fa', 'Gd', 'Po', 'TA'],
'CentralAir': ['N', 'Y'],
'Electrical': ['FuseA', 'FuseF', 'FuseP', 'Mix', 'SBrkr'],
'1stFlrSF': [334, 372, 407, 432, 438, 442, 448, 453, 480, 483, 492, 494, 495, 498, 502, 516, 520, 525, 526, 529, 530, 536, 540, 546, 548, 551, 561, 565, 567, 572, 575, 576, 581, 585, 596, 599, 600, 605, 608, 612, 616, 617, 621, 624, 625, 626, 628, 630, 636, 640, 641, 646, 647, 649, 658, 660, 661, 662, 663, 664, 665, 666, 671, 672, 673, 676, 679, 680, 682, 684, 686, 687, 689, 691, 693, 694, 696, 697, 698, 702, 703, 704, 707, 708, 709, 712, 713, 714, 715, 716, 717, 720, 723, 725, 727, 728, 729, 730, 732, 733, 734, 735, 736, 738, 740, 741, 742, 743, 744, 745, 747, 750, 751, 752, 753, 754, 755, 756, 757, 759, 760, 761, 763, 764, 765, 767, 768, 769, 770, 772, 773, 774, 778, 779, 780, 781, 782, 783, 784, 786, 788, 789, 790, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 806, 807, 808, 

40. [Heating](../docs/descripcion_variables.md#variable-heating): Tipo de calefacción.

In [230]:
resultado = analizar_precio_viviendas_por_variable(df, 'Heating')
resultado

,Heating,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
1,GasA,182021.195378,2874,98.458376,1428,1446
2,GasW,166632.166667,27,0.924974,18,9
4,OthW,125750.000000,2,0.068517,2,0
5,Wall,92100.000000,6,0.205550,4,2
3,Grav,75271.428571,9,0.308325,7,2
0,Floor,72500.000000,1,0.034258,1,0


Para simplificar las categorías de la variable `Heating` y mejorar la capacidad del modelo de interpretar los datos, se reemplazaron las categorías minoritarias con `"Otro"`.

In [231]:
categorias_Heating = ['OthW', 'Wall', 'Grav', 'Floor']

df['Heating'] = df['Heating'].replace(categorias_Heating, 'Otro')


In [232]:
resultado = analizar_precio_viviendas_por_variable(df, 'Heating')
resultado

,Heating,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
0,GasA,182021.195378,2874,98.458376,1428,1446
1,GasW,166632.166667,27,0.924974,18,9
2,Otro,87092.857143,18,0.616650,14,4


Al analizar la distribución de la variable `Heating`, se observa que el **98.45%** de los datos pertenecen a la categoría **Gas de Ciudad (GasA)**, mientras que las otras categorías tienen muy pocas observaciones. A pesar de la agrupación de las categorías menores en la categoría `"Otro"`, la falta de variedad en los tipos de calefacción hace que esta variable no sea particularmente significativa para el modelo.

41. [HeatingQC](../docs/descripcion_variables.md#variable-heatingqc): Calidad y condición de la calefacción.

In [233]:
resultado = analizar_precio_viviendas_por_variable(df, 'HeatingQC')
resultado

,HeatingQC,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
0,Ex,214914.429150,1493,51.147653,741,752
2,Gd,156858.871369,474,16.238438,241,233
4,TA,142362.876168,857,29.359370,428,429
1,Fa,123919.489796,92,3.151764,49,43
3,Po,87000.000000,3,0.102775,1,2


In [234]:
agrupacion_calefaccion = df.groupby(['Heating', 'HeatingQC']).size().reset_index(name='Count')
agrupacion_calefaccion

,Heating,HeatingQC,Count
0,GasA,Ex,1491
1,GasA,Fa,72
2,GasA,Gd,471
3,GasA,Po,2
4,GasA,TA,838
5,GasW,Ex,2
6,GasW,Fa,5
7,GasW,Gd,3
8,GasW,TA,17
9,Otro,Fa,15


Se realizaron ciertos reemplazos en la variable `HeatingQC` basados en el tipo de calefacción (`Heating`). Los valores originales en `HeatingQC` que se reemplazaron o agruparon corresponden a categorías con pocas observaciones o con una calidad no representativa. Estos cambios tienen como objetivo reducir la dispersión de los datos, mejorar la interpretación del modelo y darle el peso adecuado a la variable `Heating`, que aunque tiene una gran mayoría de observaciones en `GasA`, sigue siendo relevante para la predicción. 

Estos ajustes mejoran la calidad de las predicciones y el rendimiento del modelo.

In [235]:
df.loc[(df['Heating'] == 'Otro') & (df['HeatingQC'] == 'TA'), 'HeatingQC'] = 'Po'
df.loc[(df['Heating'] == 'Otro') & (df['HeatingQC'] == 'Fa'), 'HeatingQC'] = 'Po'
df.loc[(df['Heating'] == 'GasW') & (df['HeatingQC'] == 'TA'), 'HeatingQC'] = 'Gd'
df.loc[(df['Heating'] == 'GasW') & (df['HeatingQC'] == 'Ex'), 'HeatingQC'] = 'Gd'
df.loc[(df['Heating'] == 'GasW') & (df['HeatingQC'] == 'Fa'), 'HeatingQC'] = 'Po'

In [236]:
distribucion_target_con_variable(df, 'SalePrice', 'HeatingQC', title="Impacto de HeatingQC en el Precio de Venta")

In [237]:
resultado = analizar_precio_viviendas_por_variable(df, 'HeatingQC')
resultado

,HeatingQC,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
0,Ex,214968.460081,1491,51.079137,739,752
2,Gd,158068.374016,493,16.889346,254,239
4,TA,141697.185542,838,28.708462,415,423
1,Fa,137978.088235,72,2.466598,34,38
3,Po,91016.666667,25,0.856458,18,7


In [238]:
codificacion_ponderada(df, 'Heating', 'SalePrice')

In [239]:
escala_calidad = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NoAplica': 0}
df['HeatingQC'] = df['HeatingQC'].map(escala_calidad)

42. [CentralAir](../docs/descripcion_variables.md#variable-centralair): Aire acondicionado central.

In [240]:
resultado = analizar_precio_viviendas_por_variable(df, 'CentralAir')
resultado

,CentralAir,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
1,Y,186186.709890,2723,93.285372,1365,1358
0,N,105264.073684,196,6.714628,95,101


In [241]:
categorias = ['Y', 'N']

df = aplicar_codificacion_ordinal_especifica(df, 'CentralAir', categorias)

43. [Electrical](../docs/descripcion_variables.md#variable-electrical): Sistema eléctrico.

In [242]:
resultado = analizar_precio_viviendas_por_variable(df, 'Electrical')
resultado

,Electrical,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
4,SBrkr,186825.113193,2671,91.503940,1334,1337
0,FuseA,122196.893617,188,6.440562,94,94
1,FuseF,107675.444444,50,1.712915,27,23
2,FuseP,97333.333333,8,0.274066,3,5
3,Mix,67000.000000,1,0.034258,1,0


In [243]:
# reemplazar mix por FuseP por precio_promedio aproximado
df['Electrical'] = df['Electrical'].replace('Mix', 'FuseP')

In [244]:
# Al haber solo 1 valor faltante en esta columna, aplico la moda
num_nan = df['Electrical'].isna().sum()
print(num_nan)

moda_Electrical= df[df['Dataset'] == 'train']['Electrical'].mode()[0]
print(moda_Electrical)
df['Electrical'].fillna(moda_Electrical, inplace=True)

1
SBrkr


Se realizó un reemplazo en la variable `Electrical` para agrupar todas las categorías que no son `SBrkr` bajo una nueva categoría llamada `Otro`. Aunque este cambio puede parecer una forma de simplificar la variable y mejorar su representación en el modelo, su impacto real es limitado debido a la distribución desigual de las categorías.

In [245]:
codificacion_ponderada(df, 'Electrical', 'SalePrice')

44. [1stFlrSF](../docs/descripcion_variables.md#variable-1stflrsf): Pies cuadrados del primer piso.

In [246]:
describe_train_test(df, '1stFlrSF')

,Train,Test
count,1460.000000,1459.000000
mean,1162.626712,1156.534613
std,386.587738,398.165820
min,334.000000,407.000000
25%,882.000000,873.500000
50%,1087.000000,1079.000000
75%,1391.250000,1382.500000
max,4692.000000,5095.000000


45. [2ndFlrSF](../docs/descripcion_variables.md#variable-2ndflrsf): Pies cuadrados del segundo piso.

In [247]:
describe_train_test(df, '2ndFlrSF')

,Train,Test
count,1460.000000,1459.000000
mean,346.992466,325.967786
std,436.528436,420.610226
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,728.000000,676.000000
max,2065.000000,1862.000000


46. [LowQualFinSF](../docs/descripcion_variables.md#variable-lowqualfinsf): Pies cuadrados de calidad baja terminados.

In [248]:
describe_train_test(df, 'LowQualFinSF')

,Train,Test
count,1460.000000,1459.000000
mean,5.844521,3.543523
std,48.623081,44.043251
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,572.000000,1064.000000


In [249]:
# Contar ceros en 'LowQualFinSF' para los conjuntos de datos 'train' y 'test'
train_cero_LowQualFinSF = len(df[(df['LowQualFinSF'] == 0) & (df['Dataset'] == 'train')])
test_cero_LowQualFinSF = len(df[(df['LowQualFinSF'] == 0) & (df['Dataset'] == 'test')])

# Contar valores mayores que 0 en 'LowQualFinSF' para los conjuntos de datos 'train' y 'test'
train_mayor_cero_LowQualFinSF = len(df[(df['LowQualFinSF'] > 0) & (df['Dataset'] == 'train')])
test_mayor_cero_LowQualFinSF = len(df[(df['LowQualFinSF'] > 0) & (df['Dataset'] == 'test')])

# Imprimir los resultados
print(f'Número de valores 0 en LowQualFinSF (train): {train_cero_LowQualFinSF}')
print(f'Número de valores 0 en LowQualFinSF (test): {test_cero_LowQualFinSF}')
print(f'Número de valores mayores a 0 en LowQualFinSF (train): {train_mayor_cero_LowQualFinSF}')
print(f'Número de valores mayores a 0 en LowQualFinSF (test): {test_mayor_cero_LowQualFinSF}')


Número de valores 0 en LowQualFinSF (train): 1434
Número de valores 0 en LowQualFinSF (test): 1445
Número de valores mayores a 0 en LowQualFinSF (train): 26
Número de valores mayores a 0 en LowQualFinSF (test): 14


Dado que hay muy pocos valores mayores a 0 tanto en los conjuntos de `train` como de `test`, se decidió mantener el valor 0 cuando no hay área sin terminar y asignar 1 cuando sí existe área sin terminar de baja calidad. Esto permite que el modelo distinga de forma más clara entre las viviendas con y sin área sin terminar.

In [250]:
df['LowQualFinSF'] = df['LowQualFinSF'].apply(lambda x: 0 if x == 0 else 1)

El precio promedio de las viviendas con área sin terminar es menor que el de las viviendas sin área sin terminar. Esto sugiere que tener un área sin terminar podría estar asociado con precios más bajos, lo cual tiene sentido en términos de valor inmobiliario. 

Pese a eso, los datos de la variable **LowQualFinSF** son tan escasos que no aportan suficiente información relevante. Como consecuencia, **LowQualFinSF** se elimina, ya que su presencia en el modelo podría introducir ruido sin mejorar la capacidad predictiva del mismo.

In [251]:
resultado = analizar_precio_viviendas_por_variable(df, 'LowQualFinSF')
resultado

,LowQualFinSF,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
0,0,181433.747559,2879,98.629668,1434,1445
1,1,152652.000000,40,1.370332,26,14


47. [GrLivArea](../docs/descripcion_variables.md#variable-grlivarea): Pies cuadrados de área habitable sobre el nivel del suelo.

GrLivArea es la suma de las áreas de 1stFlrSF y 2ndFlrSF.

Hay una correlación positiva entre el área habitable y el precio de la vivienda, a mayor GrLivArea, mayor SalePrice; es decir, el precio de la vivienda tiende a aumentar a medida que aumenta el área habitable sobre el nivel del suelo.

In [252]:
# Scatter Plot
fig = px.scatter(df, x="GrLivArea", y="SalePrice",
                 title="Gráfico de Dispersión de Precio de Venta vs. Área Habitable Sobre el Suelo",
                 labels={"SalePrice": "Precio de Venta (USD)", "GrLivArea": "Área Habitable Sobre el Suelo (pies cuadrados)"},
                 trendline="ols")
fig.show()

Se pueden observar algunos valores atípicos en el gráfico de dispersión donde GrLivArea es muy grande, pero el SalePrice es bajo. Estos outliers podrían afectar a las predicciones.Tras elegir el modelo de RandomForestRegressor, aunque es robusto frente a outlier, se probará a eliminarlos o a imputarlos con la media de train a veri si mejora el modelo.

Se observa por separado entre train y test la distribución de los datos del área habitable.

In [253]:
fig = px.box(df, 
             x='GrLivArea', 
             color='Dataset', 
             title='Box Plot de GrLivArea - Train vs Test')

fig.update_layout(xaxis_title='GrLivArea_log', 
                  yaxis_title='Distribución', 
                  title_x=0.5)
fig.show()

In [254]:
# Se transforma a logarítmica la variable para observar los valores atípicos.
df['GrLivArea_log'] = (df['GrLivArea'] + 1).apply(np.log)

In [255]:
fig = px.box(df, 
             x='GrLivArea_log', 
             color='Dataset', 
             title='Box Plot de GrLivArea_log - Train vs Test')

fig.update_layout(xaxis_title='GrLivArea_log', 
                  yaxis_title='Distribución', 
                  title_x=0.5)
fig.show()

### Evaluación del Impacto de Limitar Outliers en GrLivArea

La mejora en RMSLE es mínima y el modelo muestra una ligera disminución en el rendimiento general después de aplicar los límites superior e inferior a los outliers. Las métricas de error, como MAE, RMSE y MSE, han aumentado un poco, lo que sugiere que al limitar los outliers, el modelo puede haber perdido parte de la capacidad para ajustarse a algunos patrones importantes en los datos. Se opta por dejar los outliers sin cambios ya que son pocos y no afectan mucho la predicción.

```python
# Sustituir los outliers en el conjunto completo (train + test) con los límites

train_df = df[df['Dataset'] == 'train']

Q1 = train_df['GrLivArea_log'].quantile(0.25)
Q3 = train_df['GrLivArea_log'].quantile(0.75)
IQR = Q3 - Q1

lower_fence = Q1 - 1.5 * IQR
upper_fence = Q3 + 1.5 * IQR

df['GrLivArea_log'] = df['GrLivArea_log'].apply(lambda x: lower_fence if x < lower_fence else upper_fence if x > upper_fence else x)

# Destransformar la columna GrLivArea
df['GrLivArea'] = np.exp(df['GrLivArea_log']) - 1


In [256]:
# Scatter Plot
fig = px.scatter(df, x="GrLivArea_log", y="SalePrice",
                 title="Gráfico de Dispersión de Precio de Venta vs. Área Habitable Sobre el Suelo",
                 labels={"SalePrice": "Precio de Venta (USD)", "GrLivArea_log": "Área Habitable Sobre el Suelo (pies cuadrados)"},
                 trendline="ols")
fig.show()

In [257]:
df.drop(columns=['GrLivArea_log'], inplace=True)

In [258]:
# Introduzco las columnas sobrantes después del análisis de las características 6
columnas_a_analizar = ["SalePrice", "Heating", "HeatingQC", "CentralAir", "Electrical", "1stFlrSF", "2ndFlrSF", 
                           "LowQualFinSF", "GrLivArea"]

visualizar_correlaciones(df, columnas_a_analizar)

### 7. **Características de las Habitaciones y Funcionalidad**
> **Objetivo**: Analizar cómo la distribución de las habitaciones y la funcionalidad del hogar influyen en el valor.

48. [BsmtFullBath](../docs/descripcion_variables.md#variable-bsmtfullbath): Baños completos en el sótano.
49. [BsmtHalfBath](../docs/descripcion_variables.md#variable-bsmthalfbath): Baños medios en el sótano.
50. [FullBath](../docs/descripcion_variables.md#variable-fullbath): Baños completos sobre el nivel del suelo.
51. [HalfBath](../docs/descripcion_variables.md#variable-halfbath): Baños medios sobre el nivel del suelo.
52. [BedroomAbvGr](../docs/descripcion_variables.md#variable-bedroom): Número de dormitorios sobre el nivel del sótano.
53. [KitchenAbvGr](../docs/descripcion_variables.md#variable-kitchen): Número de cocinas.
54. [KitchenQual](../docs/descripcion_variables.md#variable-kitchenqual): Calidad de la cocina.
55. [TotRmsAbvGrd](../docs/descripcion_variables.md#variable-totrmsabvgrd): Total de habitaciones sobre el nivel del suelo.
56. [Functional](../docs/descripcion_variables.md#variable-functional): Calificación de funcionalidad del hogar.

In [259]:
caracteristicas_habitaciones = ["BsmtFullBath", "BsmtHalfBath", "FullBath", "HalfBath", "BedroomAbvGr", "KitchenAbvGr", 
                           "KitchenQual", "TotRmsAbvGrd", "Functional"]

df[["SalePrice"] + caracteristicas_habitaciones].head()

,SalePrice,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional
0,208500.0,1.0,0.0,2,1,3,1,Gd,8,Typ
1,181500.0,0.0,1.0,2,0,3,1,TA,6,Typ
2,223500.0,1.0,0.0,2,1,3,1,Gd,6,Typ
3,140000.0,1.0,0.0,1,0,3,1,Gd,7,Typ
4,250000.0,1.0,0.0,2,1,4,1,Gd,9,Typ


In [260]:
resumen = resumen_columnas(df, caracteristicas_habitaciones)
resumen

,Tipo de Dato,Nº Valores Únicos,Nº de NaN,Porcentaje NaN Train,Porcentaje NaN Test,Valores Únicos Train,Valores Únicos Test
BsmtFullBath,float64,4,2,0.0,0.13708,[],[]
BsmtHalfBath,float64,3,2,0.0,0.13708,[],[]
FullBath,int64,5,0,0.0,0.00000,[],[]
HalfBath,int64,3,0,0.0,0.00000,[],[]
BedroomAbvGr,int64,8,0,0.0,0.00000,[],[]
KitchenAbvGr,int64,4,0,0.0,0.00000,[],[]
KitchenQual,object,4,1,0.0,0.06854,[],[]
TotRmsAbvGrd,int64,14,0,0.0,0.00000,[],[]
Functional,object,7,2,0.0,0.13708,[],[]


In [261]:
valores_unicos = obtener_valores_unicos(df, caracteristicas_habitaciones)

for key, value in valores_unicos.items():
    print(f"'{key}': {value},")

'BsmtFullBath': [0.0, 1.0, 2.0, 3.0],
'BsmtHalfBath': [0.0, 1.0, 2.0],
'FullBath': [0, 1, 2, 3, 4],
'HalfBath': [0, 1, 2],
'BedroomAbvGr': [0, 1, 2, 3, 4, 5, 6, 8],
'KitchenAbvGr': [0, 1, 2, 3],
'KitchenQual': ['Ex', 'Fa', 'Gd', 'TA'],
'TotRmsAbvGrd': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
'Functional': ['Maj1', 'Maj2', 'Min1', 'Min2', 'Mod', 'Sev', 'Typ'],


48. [BsmtFullBath](../docs/descripcion_variables.md#variable-bsmtfullbath): Baños completos en el sótano.

In [262]:
resultado = analizar_precio_viviendas_por_variable(df, 'BsmtFullBath')
resultado

,BsmtFullBath,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
2,2.0,213063.066667,38,1.301816,15,23
1,1.0,202522.918367,1172,40.150737,588,584
3,3.0,179000.000000,2,0.068517,1,1
0,0.0,165521.640187,1705,58.410415,856,849


In [263]:
filas_nan = df[df['BsmtFullBath'].isna()]
filas_nan

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Alley,LotShape,LandContour,LotConfig,LandSlope,...,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Dataset,Street_Pave,Antigüedad_Remodelacion,TieneSotano,NumeroSotanos
2120,2121,82165.207953,19218.605050,99.0,5940,172000.576834,68155.136005,161800.144292,517.243045,170349.227179,...,4,2008,ConLD,Abnorml,NaN,test,True,4,1,1
2188,2189,82165.207953,150511.492846,123.0,47007,172000.576834,68155.136005,161800.144292,127519.371522,170349.227179,...,7,2008,WD,Normal,NaN,test,True,37,0,0


In [264]:
num_nan = df['BsmtFullBath'].isna().sum()
print(num_nan)

moda_BsmtFullBath= df[df['Dataset'] == 'train']['BsmtFullBath'].mode()[0]

df['BsmtFullBath'].fillna(moda_BsmtFullBath, inplace=True)

2


49. [BsmtHalfBath](../docs/descripcion_variables.md#variable-bsmthalfbath): Baños medios en el sótano.

In [265]:
resultado = analizar_precio_viviendas_por_variable(df, 'BsmtHalfBath')
resultado

,BsmtHalfBath,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
0,0.0,181230.330189,2742,93.936280,1378,1364
1,1.0,176098.125000,171,5.858171,80,91
2,2.0,160850.500000,4,0.137033,2,2


In [266]:
filas_nan = df[df['BsmtHalfBath'].isna()]
filas_nan

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Alley,LotShape,LandContour,LotConfig,LandSlope,...,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Dataset,Street_Pave,Antigüedad_Remodelacion,TieneSotano,NumeroSotanos
2120,2121,82165.207953,19218.605050,99.0,5940,172000.576834,68155.136005,161800.144292,517.243045,170349.227179,...,4,2008,ConLD,Abnorml,NaN,test,True,4,1,1
2188,2189,82165.207953,150511.492846,123.0,47007,172000.576834,68155.136005,161800.144292,127519.371522,170349.227179,...,7,2008,WD,Normal,NaN,test,True,37,0,0


In [267]:
num_nan = df['BsmtHalfBath'].isna().sum()
print(num_nan)

moda_BsmtHalfBath= df[df['Dataset'] == 'train']['BsmtHalfBath'].mode()[0]
print(moda_BsmtHalfBath)
df['BsmtHalfBath'].fillna(moda_BsmtHalfBath, inplace=True)

2
0.0


### Creación variable Baños Totales

Se crea una nueva variable para contar los baños totales de la casa, ponderando los medios baños como 0.5. Pese a su alta correlación con la variable objetivo, no mejora el rendimiento del modelo y se decide eliminar.

In [268]:
df['BañosTotales'] = df['BsmtFullBath'] + df['BsmtHalfBath'] * 0.5 + df['FullBath'] + df['HalfBath'] * 0.5

In [269]:
resultado = analizar_precio_viviendas_por_variable(df, 'BañosTotales')
resultado

,BañosTotales,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
7,4.5,386107.142857,17,0.582391,7,10
6,4.0,319118.769231,31,1.062008,13,18
5,3.5,273512.902778,289,9.900651,144,145
4,3.0,230525.672043,378,12.949640,186,192
3,2.5,199723.983051,558,19.116136,295,263
9,6.0,179000.000000,2,0.068517,1,1
2,2.0,158116.103070,902,30.900993,456,446
8,5.0,145900.000000,3,0.102775,1,2
1,1.5,142692.372093,293,10.037684,129,164
0,1.0,110869.671053,443,15.176430,228,215


In [270]:
codificacion_ponderada(df, 'BsmtFullBath', 'SalePrice')
codificacion_ponderada(df, 'BsmtHalfBath', 'SalePrice')

50. [FullBath](../docs/descripcion_variables.md#variable-fullbath): Baños completos sobre el nivel del suelo.

In [271]:
resultado = analizar_precio_viviendas_por_variable(df, 'FullBath')
resultado

,FullBath,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
3,3,347822.909091,64,2.192532,33,31
2,2,213009.825521,1530,52.415211,768,762
0,0,165200.888889,12,0.411100,9,3
1,1,134751.440000,1309,44.844125,650,659
4,4,NaN,4,0.137033,0,4


51. [HalfBath](../docs/descripcion_variables.md#variable-halfbath): Baños medios sobre el nivel del suelo.

In [272]:
resultado = analizar_precio_viviendas_por_variable(df, 'HalfBath')
resultado

,HalfBath,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
1,1,212721.960748,1060,36.313806,535,525
0,0,162534.884995,1834,62.829736,913,921
2,2,162028.916667,25,0.856458,12,13


52. [BedroomAbvGr](../docs/descripcion_variables.md#variable-bedroom): Número de dormitorios sobre el nivel del sótano.

In [273]:
resultado = analizar_precio_viviendas_por_variable(df, 'BedroomAbvGr')
resultado

,BedroomAbvGr,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
0,0,221493.166667,8,0.274066,6,2
4,4,220421.253521,400,13.703323,213,187
7,8,200000.000000,1,0.034258,1,0
3,3,181056.870647,1596,54.676259,804,792
5,5,180819.047619,48,1.644399,21,27
1,1,173162.420000,103,3.528606,50,53
2,2,158197.659218,742,25.419664,358,384
6,6,143779.000000,21,0.719424,7,14


53. [KitchenAbvGr](../docs/descripcion_variables.md#variable-kitchen): Número de cocinas.

In [274]:
resultado = analizar_precio_viviendas_por_variable(df, 'KitchenAbvGr')
resultado

,KitchenAbvGr,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
1,1,183388.790230,2785,95.409387,1392,1393
2,2,131096.153846,129,4.419322,65,64
0,0,127500.000000,3,0.102775,1,2
3,3,109500.000000,2,0.068517,2,0


54. [KitchenQual](../docs/descripcion_variables.md#variable-kitchenqual): Calidad de la cocina.

In [275]:
resultado = analizar_precio_viviendas_por_variable(df, 'KitchenQual')
resultado

,KitchenQual,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
0,Ex,328554.670000,205,7.022953,100,105
2,Gd,212116.023891,1151,39.431312,586,565
3,TA,139962.511565,1492,51.113395,735,757
1,Fa,105565.205128,70,2.398082,39,31


In [276]:
filas_nan = df[df['KitchenQual'].isna()]
filas_nan

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Alley,LotShape,LandContour,LotConfig,LandSlope,...,YrSold,SaleType,SaleCondition,SalePrice,Dataset,Street_Pave,Antigüedad_Remodelacion,TieneSotano,NumeroSotanos,BañosTotales
1555,1556,82165.207953,150511.492846,72.0,10632,172000.576834,68155.136005,161800.144292,127519.371522,170349.227179,...,2010,COD,Normal,NaN,test,True,33,1,0,1.5


In [277]:
df.loc[df['KitchenAbvGr'] == 0, 'KitchenQual'] = 'NoAplica'

In [278]:
# ver como categorizo el nan
num_nan = df['KitchenQual'].isna().sum()
print(num_nan)

moda_KitchenQual= df[df['Dataset'] == 'train']['KitchenQual'].mode()[0]
print(moda_KitchenQual)
df['KitchenQual'].fillna(moda_KitchenQual, inplace=True)

1
TA


In [279]:
resultado = analizar_precio_viviendas_por_variable(df, 'KitchenQual')
resultado

,KitchenQual,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
0,Ex,328554.670000,205,7.022953,100,105
2,Gd,212116.023891,1151,39.431312,586,565
4,TA,139979.490463,1490,51.044878,734,756
3,NoAplica,127500.000000,3,0.102775,1,2
1,Fa,105565.205128,70,2.398082,39,31


In [280]:
# categorias_KitchenQual = ['Ex', 'Gd', 'TA', 'Fa']
# df = aplicar_codificacion_ordinal_especifica(df, 'KitchenQual', categorias_KitchenQual)

escala_calidad = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NoAplica': 0}
df['KitchenQual'] = df['KitchenQual'].map(escala_calidad)

55. [TotRmsAbvGrd](../docs/descripcion_variables.md#variable-totrmsabvgrd): Total de habitaciones sobre el nivel del suelo.

In [281]:
resultado = analizar_precio_viviendas_por_variable(df, 'TotRmsAbvGrd')
resultado

,TotRmsAbvGrd,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
9,11,318022.000000,32,1.096266,18,14
8,10,296279.170213,80,2.740665,47,33
10,12,280971.454545,16,0.548133,11,5
7,9,252988.173333,143,4.898938,75,68
6,8,213427.529412,347,11.887633,187,160
12,14,200000.000000,1,0.034258,1,0
5,7,196666.784195,649,22.233642,329,320
4,6,161303.296020,844,28.914012,402,442
3,5,141550.749091,583,19.972593,275,308
2,4,122844.628866,196,6.714628,97,99


56. [Functional](../docs/descripcion_variables.md#variable-functional): Calificación de funcionalidad del hogar.

- **Typ** es la mejor opción porque indica que la vivienda es funcional sin restricciones significativas.
- **Mod** sigue, ya que la vivienda es funcional, pero presenta algunas limitaciones menores que no afectan significativamente su uso.
- **Min1** y **Min2** indican que la funcionalidad es mínima. **Min1** suele ser ligeramente mejor que **Min2**.
- **Maj1** y **Maj2** sugieren problemas mayores que afectan el uso de la vivienda. **Maj1** es menos severo que **Maj2**.
- **Sev** es el peor caso, ya que describe una vivienda con graves problemas de funcionalidad que hacen difícil o imposible su uso adecuado.

In [282]:
resultado = analizar_precio_viviendas_por_variable(df, 'Functional')
resultado

,Functional,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
6,Typ,183429.147059,2717,93.079822,1360,1357
4,Mod,168393.333333,35,1.199041,15,20
0,Maj1,153948.142857,19,0.650908,14,5
2,Min1,146385.483871,65,2.226790,31,34
3,Min2,144240.647059,70,2.398082,34,36
5,Sev,129000.000000,2,0.068517,1,1
1,Maj2,85800.000000,9,0.308325,5,4


In [283]:
filas_nan = df[df['Functional'].isna()]
filas_nan

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Alley,LotShape,LandContour,LotConfig,LandSlope,...,YrSold,SaleType,SaleCondition,SalePrice,Dataset,Street_Pave,Antigüedad_Remodelacion,TieneSotano,NumeroSotanos,BañosTotales
2216,2217,82165.207953,150511.492846,80.0,14584,172000.576834,104491.877911,4899.887292,127519.371522,8664.844861,...,2008,WD,Abnorml,NaN,test,True,0,0,0,1.0
2473,2474,82165.207953,19218.605050,60.0,10320,4520.642671,104491.877911,161800.144292,32712.462964,170349.227179,...,2007,COD,Abnorml,NaN,test,True,40,1,0,2.0


In [284]:
num_nan = df['Functional'].isna().sum()
print(num_nan)

moda_Functional= df[df['Dataset'] == 'train']['Functional'].mode()[0]
print(moda_Functional)
df['Functional'].fillna(moda_Functional, inplace=True)

2
Typ


In [285]:
mapeo = {

    'Sev': 'Maj2',
     'Maj1': 'Maj2',
}

# Reemplazar valores en la columna 'Functional'
df['Functional'] = df['Functional'].replace(mapeo)

In [286]:
resultado = analizar_precio_viviendas_por_variable(df, 'Functional')
resultado

,Functional,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
4,Typ,183429.147059,2719,93.148338,1360,1359
3,Mod,168393.333333,35,1.199041,15,20
1,Min1,146385.483871,65,2.226790,31,34
2,Min2,144240.647059,70,2.398082,34,36
0,Maj2,135663.700000,30,1.027749,20,10


In [287]:
# mayor a menor funcionalidad

orden_funcionalidad = ['Typ', 'Mod', 'Min1', 'Min2', 'Maj1', 'Maj2', 'Sev']
df = aplicar_codificacion_ordinal_especifica(df, 'Functional', orden_funcionalidad)

In [288]:
# Introduzco las columnas sobrantes después del análisis de las características 7
columnas_a_analizar = ["SalePrice", "BañosTotales", "BsmtFullBath", "BsmtHalfBath", "FullBath", "HalfBath", "BedroomAbvGr", "KitchenAbvGr", 
                           "KitchenQual", "TotRmsAbvGrd", "Functional" ]

visualizar_correlaciones_grandes(df, columnas_a_analizar)

### 8. **Características de las Áreas de Entretenimiento y Exteriores**
> **Objetivo**: Examinar las áreas de entretenimiento y espacios exteriores que podrían agregar valor a la propiedad.

57. [Fireplaces](../docs/descripcion_variables.md#variable-fireplaces): Número de chimeneas.
58. [FireplaceQu](../docs/descripcion_variables.md#variable-fireplacequ): Calidad de la chimenea.
59. [GarageType](../docs/descripcion_variables.md#variable-garagetype): Ubicación del garaje.
60. [GarageYrBlt](../docs/descripcion_variables.md#variable-garageyrblt): Año en que se construyó el garaje.
61. [GarageFinish](../docs/descripcion_variables.md#variable-garagefinish): Acabado interior del garaje.
62. [GarageCars](../docs/descripcion_variables.md#variable-garagecars): Capacidad del garaje en automóviles.
63. [GarageArea](../docs/descripcion_variables.md#variable-garagearea): Tamaño del garaje en pies cuadrados.
64. [GarageQual](../docs/descripcion_variables.md#variable-garagequal): Calidad del garaje.
65. [GarageCond](../docs/descripcion_variables.md#variable-garagecond): Condición del garaje.
66. [PavedDrive](../docs/descripcion_variables.md#variable-paveddrive): Entrada pavimentada.
67. [WoodDeckSF](../docs/descripcion_variables.md#variable-wooddecksf): Área de la terraza de madera en pies cuadrados.
68. [OpenPorchSF](../docs/descripcion_variables.md#variable-openporchsf): Área del porche abierto en pies cuadrados.
69. [EnclosedPorch](../docs/descripcion_variables.md#variable-enclosedporch): Área del porche cerrado en pies cuadrados.
70. [3SsnPorch](../docs/descripcion_variables.md#variable-3ssnporch): Área del porche de tres estaciones en pies cuadrados.
71. [ScreenPorch](../docs/descripcion_variables.md#variable-screenporch): Área del porche con mosquitero en pies cuadrados.
72. [PoolArea](../docs/descripcion_variables.md#variable-poolarea): Área de la piscina en pies cuadrados.
73. [PoolQC](../docs/descripcion_variables.md#variable-poolqc): Calidad de la piscina.
74. [Fence](../docs/descripcion_variables.md#variable-fence): Calidad de la cerca.

In [289]:
caracteristicas_exteriores = ["Fireplaces", "FireplaceQu", "GarageType", "GarageYrBlt", "GarageFinish", "GarageCars", 
                            "GarageArea", "GarageQual", "GarageCond", "PavedDrive", "WoodDeckSF", "OpenPorchSF", 
                            "EnclosedPorch", "3SsnPorch", "ScreenPorch", "PoolArea", "PoolQC", "Fence"]

df[["SalePrice"] + caracteristicas_exteriores].head()

,SalePrice,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence
0,208500.0,0,NaN,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,NaN,NaN
1,181500.0,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,NaN,NaN
2,223500.0,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,NaN,NaN
3,140000.0,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,NaN,NaN
4,250000.0,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,NaN,NaN


In [290]:
resumen = resumen_columnas(df, caracteristicas_exteriores)
resumen

,Tipo de Dato,Nº Valores Únicos,Nº de NaN,Porcentaje NaN Train,Porcentaje NaN Test,Valores Únicos Train,Valores Únicos Test
Fireplaces,int64,5,0,0.000000,0.000000,[],[]
FireplaceQu,object,5,1420,47.260274,50.034270,[],[]
GarageType,object,6,157,5.547945,5.209047,[],[]
GarageYrBlt,float64,103,159,5.547945,5.346127,[],[]
GarageFinish,object,3,159,5.547945,5.346127,[],[]
GarageCars,float64,6,1,0.000000,0.068540,[],[]
GarageArea,float64,603,1,0.000000,0.068540,[],[]
GarageQual,object,5,159,5.547945,5.346127,[Ex],[]
GarageCond,object,5,159,5.547945,5.346127,[],[]
PavedDrive,object,3,0,0.000000,0.000000,[],[]


In [291]:
valores_unicos = obtener_valores_unicos(df, caracteristicas_exteriores)

for key, value in valores_unicos.items():
    print(f"'{key}': {value},")

'Fireplaces': [0, 1, 2, 3, 4],
'FireplaceQu': ['Ex', 'Fa', 'Gd', 'Po', 'TA'],
'GarageType': ['2Types', 'Attchd', 'Basment', 'BuiltIn', 'CarPort', 'Detchd'],
'GarageYrBlt': [1895.0, 1896.0, 1900.0, 1906.0, 1908.0, 1910.0, 1914.0, 1915.0, 1916.0, 1917.0, 1918.0, 1919.0, 1920.0, 1921.0, 1922.0, 1923.0, 1924.0, 1925.0, 1926.0, 1927.0, 1928.0, 1929.0, 1930.0, 1931.0, 1932.0, 1933.0, 1934.0, 1935.0, 1936.0, 1937.0, 1938.0, 1939.0, 1940.0, 1941.0, 1942.0, 1943.0, 1945.0, 1946.0, 1947.0, 1948.0, 1949.0, 1950.0, 1951.0, 1952.0, 1953.0, 1954.0, 1955.0, 1956.0, 1957.0, 1958.0, 1959.0, 1960.0, 1961.0, 1962.0, 1963.0, 1964.0, 1965.0, 1966.0, 1967.0, 1968.0, 1969.0, 1970.0, 1971.0, 1972.0, 1973.0, 1974.0, 1975.0, 1976.0, 1977.0, 1978.0, 1979.0, 1980.0, 1981.0, 1982.0, 1983.0, 1984.0, 1985.0, 1986.0, 1987.0, 1988.0, 1989.0, 1990.0, 1991.0, 1992.0, 1993.0, 1994.0, 1995.0, 1996.0, 1997.0, 1998.0, 1999.0, 2000.0, 2001.0, 2002.0, 2003.0, 2004.0, 2005.0, 2006.0, 2007.0, 2008.0, 2009.0, 2010.0, 2207.0],
'G

57. [Fireplaces](../docs/descripcion_variables.md#variable-fireplaces): Número de chimeneas.

In [292]:
resultado = analizar_precio_viviendas_por_variable(df, 'Fireplaces')
resultado

,Fireplaces,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
3,3,252000.000000,11,0.376841,5,6
2,2,240588.539130,219,7.502569,115,104
1,1,211843.909231,1268,43.439534,650,618
0,0,141331.482609,1420,48.646797,690,730
4,4,NaN,1,0.034258,0,1


Se agrupa "4" con "3" porque mejora la estabilidad y generalización del modelo al evitar categorías poco representadas que el modelo no puede aprender correctamente.

In [293]:
df.loc[df['Fireplaces'] == 4, 'Fireplaces'] = 3

### Creación de la Variable `HasFireplaces`

Para explorar el impacto de la presencia o ausencia de chimeneas, se creó una nueva variable binaria llamada `HasFireplaces` para decir si la casa tiene chimeneas o no. Se borra posteriormente porque no ayuda a predecir mejor al modelo XGBRegressor. 

In [294]:
df['HasFireplaces'] = df['Fireplaces'] != 0

df['HasFireplaces'].value_counts()

HasFireplaces
True     1499
False    1420
Name: count, dtype: int64

In [295]:
distribucion_target_con_variable(df, 'SalePrice', 'HasFireplaces', title='Impacto de las chimeneas en el Precio de Venta')

58. [FireplaceQu](../docs/descripcion_variables.md#variable-fireplacequ): Calidad de la chimenea.

In [296]:
num_nan = df['FireplaceQu'].isna().sum()
print(num_nan)


1420


In [297]:
df.loc[(df['Fireplaces'] == 0) & (df['FireplaceQu'].isna()), 'FireplaceQu'] = 'NoAplica'

In [298]:
# Todos los valores NaN eran las que no tenían chimenea
num_nan = df['FireplaceQu'].isna().sum()
print(num_nan)

0


In [299]:
resultado = analizar_precio_viviendas_por_variable(df, 'FireplaceQu')
resultado

,FireplaceQu,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
0,Ex,337712.500000,43,1.473107,24,19
2,Gd,226351.415789,744,25.488181,380,364
5,TA,205723.488818,592,20.280918,313,279
1,Fa,167298.484848,74,2.535115,33,41
3,NoAplica,141331.482609,1420,48.646797,690,730
4,Po,129764.150000,46,1.575882,20,26


In [300]:
# categorias = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NoAplica']

# # Aplicar la codificación 
# df = aplicar_codificacion_ordinal_especifica(df, 'FireplaceQu', categorias)


escala_calidad = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NoAplica': 0}
df['FireplaceQu'] = df['FireplaceQu'].map(escala_calidad)

62. [GarageCars](../docs/descripcion_variables.md#variable-garagecars): Capacidad del garaje en automóviles.

### Decisión sobre la Codificación de la Variable `GarageCars`

Se realizaron pruebas para evaluar diferentes formas de codificar la variable `GarageCars` puesto que está entre las primeras feature importances del modelo XGBRegressor, se hizo al final de esta sección para que las transformaciones aplicadas no afectaran a otras variables. 

- **Codificación Ponderada**: Se aplicó codificación ponderada introduciendo la media global para el valor `GarageCars = 5`, ya que este valor solo aparece en el conjunto de test. Este enfoque **empeoró el modelo**, aumentando el RMSLE.

- **Codificación con `get_dummies`**: Se generaron variables dummy para cada valor de `GarageCars`, eliminando la columna base para evitar multicolinealidad. Este enfoque **no empeoró el modelo** (mantuvo el mismo RMSLE), pero **capturó peor los valores altos**, lo cual no es ideal en este caso.

Dado que las pruebas indicaron que ninguna codificación mejora significativamente el rendimiento del modelo y que mantener la variable como numérica permite capturar correctamente la relación jerárquica (0 < 1 < 2 < 3 < 4 < 5), **se decidió no aplicar ningún tipo de codificación a `GarageCars`**.

In [301]:
resultado = analizar_precio_viviendas_por_variable(df, 'GarageCars')
resultado

,GarageCars,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
3,3.0,309636.121547,374,12.812607,181,193
4,4.0,192655.800000,16,0.548133,5,11
2,2.0,183851.663835,1594,54.607742,824,770
1,1.0,128116.688347,776,26.584447,369,407
0,0.0,103317.283951,157,5.378554,81,76
5,5.0,NaN,1,0.034258,0,1


In [302]:
num_nan = df['GarageCars'].isna().sum()
print(num_nan)

1


Como es un valor de test no se elimina. La única información que se tiene es que es un Garaje separado, por lo que garaje tiene. 

In [303]:
filas_nan = df[df['GarageCars'].isna()]
filas_nan[["SalePrice", "GarageType", "GarageYrBlt", "GarageFinish", "GarageCars", 
                            "GarageArea", "GarageQual", "GarageCond", "Dataset"]]

,SalePrice,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,Dataset
2576,NaN,Detchd,NaN,NaN,NaN,NaN,NaN,NaN,test


Se decide reemplazar el valor nulo utilizando la moda de train donde GarageType = Detchd el resultado es 2, que es también la moda general de train.

In [304]:
moda_GarageCars = df[(df['GarageType'] == 'Detchd') & (df['Dataset'] == 'train')]['GarageCars'].mode()[0]
print(moda_GarageCars)

df['GarageCars'].fillna(moda_GarageCars, inplace=True)

2.0


### Creación de la Variable `HasGarage`

Para explorar el impacto de la presencia o ausencia de garajes, se creó una nueva variable binaria llamada `HasGarage` para decir si la casa tiene garaje o no. 

In [305]:
df['HasGarage'] = df['GarageCars'] != 0

df['HasGarage'].value_counts()

HasGarage
True     2762
False     157
Name: count, dtype: int64

Se observa en la distribución de `HasGarage` con `SalePrice` cómo tener garaje influye significativamente en el precio de la casa.

In [306]:
distribucion_target_con_variable(df, 'SalePrice', 'HasGarage', title='Impacto del Garage en el Precio de Venta')

59. [GarageType](../docs/descripcion_variables.md#variable-garagetype): Ubicación del garaje.

In [307]:
num_nan = df['GarageType'].isna().sum() 
print(num_nan)

157


Cuando la propiedad no tiene garaje, se reemplazan los valores nulos en la columna GarageType con 'NoGaraje'. Además, se verifica que todos los valores nulos (NaN) correspondían efectivamente a propiedades sin garaje.

In [308]:
df.loc[(df['GarageCars'] == 0) & (df['GarageType'].isna()), 'GarageType'] = 'No Garaje'

In [309]:
num_nan = df['GarageType'].isna().sum()
print(num_nan)

0


In [310]:
resultado = analizar_precio_viviendas_por_variable(df, 'GarageType')
resultado

,GarageType,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
3,BuiltIn,254751.738636,186,6.372045,88,98
1,Attchd,202892.656322,1723,59.027064,870,853
2,Basment,160570.684211,36,1.233299,19,17
0,2Types,151283.333333,23,0.787941,6,17
5,Detchd,134091.162791,779,26.687222,387,392
4,CarPort,109962.111111,15,0.513875,9,6
6,No Garaje,103317.283951,157,5.378554,81,76


Se realiza una codificación ponderada para `GarageType` porque la utilización de LOO (Leave-One-Out) disminuye el rendimiento del modelo.

In [311]:
codificacion_ponderada(df, 'GarageType', 'SalePrice')

60. [GarageYrBlt](../docs/descripcion_variables.md#variable-garageyrblt): Año en que se construyó el garaje.

In [312]:
resultado = analizar_precio_viviendas_por_variable(df, 'GarageYrBlt')
resultado

,GarageYrBlt,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
101,2010.0,337874.666667,5,0.171292,3,2
99,2008.0,306569.034483,61,2.089757,29,32
100,2009.0,306111.000000,29,0.993491,21,8
97,2006.0,262001.338983,115,3.939705,59,56
85,1994.0,258100.000000,39,1.336074,18,21
...,...,...,...,...,...,...
1,1896.0,NaN,1,0.034258,0,1
9,1917.0,NaN,2,0.068517,0,2
11,1919.0,NaN,1,0.034258,0,1
35,1943.0,NaN,1,0.034258,0,1


In [313]:
print(sorted(df['GarageYrBlt'].unique()))

[1915.0, 1920.0, 1930.0, 1931.0, 1935.0, 1939.0, 1945.0, 1948.0, 1950.0, 1953.0, 1954.0, 1956.0, 1957.0, 1958.0, 1959.0, 1960.0, 1961.0, 1962.0, 1963.0, 1964.0, 1965.0, 1966.0, 1967.0, 1968.0, 1970.0, 1973.0, 1974.0, 1976.0, 1977.0, 1981.0, 1983.0, 1985.0, 1987.0, 1989.0, 1990.0, 1991.0, 1993.0, 1995.0, 1997.0, 1998.0, 1999.0, 2000.0, 2001.0, 2002.0, 2003.0, 2004.0, 2005.0, 2006.0, 2007.0, 2008.0, nan, 1895.0, 1896.0, 1900.0, 1906.0, 1908.0, 1910.0, 1914.0, 1916.0, 1917.0, 1918.0, 1919.0, 1921.0, 1922.0, 1923.0, 1924.0, 1925.0, 1926.0, 1927.0, 1928.0, 1929.0, 1932.0, 1933.0, 1934.0, 1936.0, 1937.0, 1938.0, 1940.0, 1941.0, 1942.0, 1943.0, 1946.0, 1947.0, 1949.0, 1951.0, 1952.0, 1955.0, 1969.0, 1971.0, 1972.0, 1975.0, 1978.0, 1979.0, 1980.0, 1982.0, 1984.0, 1986.0, 1988.0, 1992.0, 1994.0, 1996.0, 2009.0, 2010.0, 2207.0]


Como no tiene sentido el año 2207 voy a ver las otras columnas con fechas para ver qué valor imputarle

In [314]:
df_filtrado = df[df['GarageYrBlt'] == 2207][['YearBuilt', 'Antigüedad_Remodelacion', 'YearRemodAdd', 'YrSold', 'GarageYrBlt', 'MoSold']]
df_filtrado

,YearBuilt,Antigüedad_Remodelacion,YearRemodAdd,YrSold,GarageYrBlt,MoSold
2592,2006,1,2007,2007,2207.0,9


Al calcular la media para `GarageYrBlt`, me da un valor de 1980. Sin embargo, este valor no tiene sentido como año de construcción del garaje, ya que la casa se construyó en 2006.

Al revisar más a fondo, es probable que el error de `2207` se deba a una confusión entre los años, dado que la casa fue vendida en septiembre de 2007. Esto sugiere que hubo tiempo suficiente para construir el garaje en 2007.

Además, es probable que el año que hubo una remodelación `YearRemodAdd` también hubiera en el garaje, ya que coincidiría con el año 2007, indicando que tanto la casa como el garaje fueron probablemente renovados en esa misma fecha.

Por lo tanto, tiene más sentido imputar el valor de `GarageYrBlt` como 2007 en lugar de 1980.

In [315]:
df['GarageYrBlt'].replace(2207, 2007, inplace=True)

# Relleno de valores nulos en `GarageYrBlt`

Para rellenar los valores nulos en la columna `GarageYrBlt`, que representan las propiedades sin garaje, se probaron varios enfoques:

2. **Rellenarlos con `0`**: Asignando un valor fijo que indique la ausencia de información.  
3. **Usar la mediana**: Rellenando los valores nulos con la mediana de los años de construcción del garaje.  
4. **Rellenarlos con el año de construcción más viejo**: Usando el valor más antiguo registrado en la columna.  
5. **Rellenarlos con el año de construcción de la casa**

Tras comparar los resultados obtenidos con cada enfoque, se observó que **mantener los valores nulos (`NaN`) produce los mejores resultados en las predicciones del modelo**. Aprovechando que `XGBRegressor` soporta valores nulos directamente, se decide dejar los valores nulos sin modificar.


In [316]:
num_nan = df['GarageYrBlt'].isna().sum()
print(num_nan)

159


En la antigüedad de la remodelación no está incluida la construcción del garaje, ya que una propiedad puede haber experimentado renovaciones sin que necesariamente se haya modificado o reconstruido el garaje.

In [317]:
df[['GarageCars', 'YearBuilt', 'GarageYrBlt', 'Antigüedad_Remodelacion', 'YearRemodAdd', 'YrSold' ]].head()

,GarageCars,YearBuilt,GarageYrBlt,Antigüedad_Remodelacion,YearRemodAdd,YrSold
0,2.0,2003,2003.0,0,2003,2008
1,2.0,1976,1976.0,0,1976,2007
2,2.0,2001,2001.0,1,2002,2008
3,3.0,1915,1998.0,55,1970,2006
4,3.0,2000,2000.0,0,2000,2008


Se rellenan los valores faltantes de GarageYrBlt con YearBuilt solo en las casas que tienen garaje (GarageCars != 0).

In [318]:
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(df['YearBuilt'].where((df['GarageCars'] != 0)))

In [319]:
num_nan = df['GarageYrBlt'].isna().sum()
print(num_nan)

157


Se comprueba si hay valores de la construcción del garaje anteriores al año de construcción de la casa o posteriores al año de venta

In [320]:
def verificar_filas_false(df): 
    def verificar_fila(row):
        if pd.isna(row['GarageYrBlt']) or pd.isna(row['YearBuilt']) or pd.isna(row['YrSold']):
            return True  # Considerar filas con NaN como válidas, se tratarán de otra manera
        return row['YearBuilt'] <= row['GarageYrBlt'] <= row['YrSold']

    # Aplicar la lógica inversa (~) para obtener filas incorrectas
    filas_false = df[~df.apply(verificar_fila, axis=1)]
    return filas_false

filas_no_entre = verificar_filas_false(df)

print(len(filas_no_entre))

18


Hay 18 filas donde el año de construcción del garaje es anterior al año de construcción de la casa, se hizo antes el garaje que la casa.

In [321]:
filas_no_entre[["YearBuilt", "GarageYrBlt", "YrSold"]]

,YearBuilt,GarageYrBlt,YrSold
29,1927,1920.0,2008
93,1910,1900.0,2007
324,1967,1961.0,2010
600,2005,2003.0,2006
736,1950,1949.0,2006
1103,1959,1954.0,2006
1376,1930,1925.0,2008
1414,1923,1922.0,2008
1418,1963,1962.0,2008
1521,1959,1956.0,2010


Como todas son inferiores al año de construcción, se va a considerar que el año de construcción del garaje es el mismo que cuando terminaron de construir toda la casa.

In [322]:
df['GarageYrBlt'] = df.apply(lambda row: row['YearBuilt'] if row['GarageYrBlt'] < row['YearBuilt'] else row['GarageYrBlt'], axis=1)

Se reemplazan los NaN para otros modelos con la mediana de GarageYrBlt, que en la gran mayoría es inferior al año de construcción de la casa. Se prueban varios métodos como sustituirlos por 0 cuando no tiene garaje, no tiene año de construcción de garaje, pero así es como se obtienen mejores resultados.

In [323]:
# Calcular la mediana de 'GarageYrBlt' donde 'GarageCars' es igual a 0 y 'Dataset' es igual a 'train', excluyendo los NaN
mediana_GarageYrBlt = df.loc[(df['Dataset'] == 'train'), 'GarageYrBlt'].dropna().median()

# Mostrar la mediana calculada
print(mediana_GarageYrBlt)

# Llenar los valores nulos en 'GarageYrBlt' con la mediana calculada
df['GarageYrBlt'].fillna(mediana_GarageYrBlt, inplace=True)


1980.0


In [324]:
num_nan = df['GarageYrBlt'].isna().sum()
print(num_nan)

0


# Se crea una nueva variable Antigüedad_Remodelacion_Garaje

El objetivo de crear `Antigüedad_Remodelacion_Garaje` es reemplazar la columna `GarageYrBlt`, que presenta el problema de tener años de construcción del garaje en registros donde no existe garaje, dificultando la interpretación del modelo. Al utilizar la nueva variable, se espera que el modelo pueda manejar de manera más eficiente los datos relacionados con los garajes y obtener resultados más precisos.

- Para los registros donde no existe un garaje, se asigna el valor `0`. Esto indica que no hay antigüedad de remodelación para estos casos, ya que no se tiene un garaje.
- En los registros que sí tienen un garaje, se calcula la antigüedad de la remodelación sumando 1 año. Esto se hace para facilitar la comprensión, ya que los registros sin garaje tienen asignado el valor 0 (sin antigüedad).

In [325]:
# df["Antigüedad_Remodelacion_Garaje"] = np.where(df['GarageYrBlt'] != 0, df['GarageYrBlt'] - df['YearBuilt'] + 1, 0)

In [326]:
# df[["Antigüedad_Remodelacion_Garaje", 'GarageCars', 'YearBuilt', 'GarageYrBlt', 'Antigüedad_Remodelacion', 'YearRemodAdd', 'YrSold']].head()

61. [GarageFinish](../docs/descripcion_variables.md#variable-garagefinish): Acabado interior del garaje.

In [327]:
num_nan = df['GarageFinish'].isna().sum()
print(num_nan)


159


In [328]:
df.loc[(df['GarageCars'] == 0) & (df['GarageFinish'].isna()), 'GarageFinish'] = 'No Garaje'

In [329]:
num_nan = df['GarageFinish'].isna().sum()
print(num_nan)

2


In [330]:
# Obtener la moda de 'GarageFinish' donde 'GarageCars' no es 0 y el dataset es 'train'
moda_GarageFinish = df.loc[(df['GarageCars'] != 0) & (df['Dataset'] == 'train'), 'GarageFinish'].mode()[0]
print(moda_GarageFinish)

# Llenar los valores nulos en 'GarageFinish' con la moda calculada
df['GarageFinish'].fillna(moda_GarageFinish, inplace=True)

Unf


In [331]:
mapeo = {
    'Fin': 'Terminado',
    'RFn': 'Terminado con calefacción',
    'Unf': 'Sin terminar',
}

df['GarageFinish'] = df['GarageFinish'].replace(mapeo)

In [332]:
resultado = analizar_precio_viviendas_por_variable(df, 'GarageFinish')
resultado

,GarageFinish,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
2,Terminado,240052.690341,719,24.631723,352,367
3,Terminado con calefacción,202068.869668,811,27.783487,422,389
1,Sin terminar,142156.423140,1232,42.206235,605,627
0,No Garaje,103317.283951,157,5.378554,81,76


Tener calefacción en el garaje parece no ser algo que aporte valor a la casa.

In [333]:
# parece que aunque el garaje esté Terminado y además tenga calefacción, que podría pensarse que es mejor,
# no hace que aumente el precio, por lo que codifico ponderado
codificacion_ponderada(df, 'GarageFinish', 'SalePrice')

63. [GarageArea](../docs/descripcion_variables.md#variable-garagearea): Tamaño del garaje en pies cuadrados.

In [334]:
resultado = analizar_precio_viviendas_por_variable(df, 'GarageArea')
resultado

,GarageArea,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
492,832.0,755000.0,1,0.034258,1,0
480,813.0,745000.0,1,0.034258,1,0
475,807.0,625000.0,1,0.034258,1,0
574,1020.0,582933.0,1,0.034258,1,0
418,716.0,556581.0,1,0.034258,1,0
...,...,...,...,...,...,...
593,1200.0,NaN,1,0.034258,0,1
595,1231.0,NaN,1,0.034258,0,1
597,1314.0,NaN,1,0.034258,0,1
598,1348.0,NaN,1,0.034258,0,1


In [335]:
num_nan = df['GarageArea'].isna().sum()
print(num_nan)

1


In [336]:
fila_nan_GarageArea = df[df['GarageArea'].isna()]
fila_nan_GarageArea[["GarageType", "GarageYrBlt", "GarageFinish", "GarageCars", 
                            "GarageArea", "GarageQual", "GarageCond", "Dataset"]]

,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,Dataset
2576,35856.01668,1923.0,59168.479803,2.0,NaN,NaN,NaN,test


In [337]:
# Calcular la media del área de garaje para viviendas con 2 garajes
media_garage_area = df[df['GarageCars'] == 2]['GarageArea'].mean()

# Imputar el valor faltante con la media calculada y almacenar el índice de la fila imputada
indice_imputado = df['GarageArea'].isna()
df.loc[indice_imputado, 'GarageArea'] = media_garage_area

# Imprimir la fila que fue imputada
print("Valor imputado en 'GarageArea':")
print(df.loc[indice_imputado, ['GarageCars', 'GarageArea']])

Valor imputado en 'GarageArea':
      GarageCars  GarageArea
2576         2.0  519.432873


64. [GarageQual](../docs/descripcion_variables.md#variable-garagequal): Calidad del garaje.

In [338]:
resultado = analizar_precio_viviendas_por_variable(df, 'GarageQual')
resultado

,GarageQual,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
0,Ex,241000.000000,3,0.102775,3,0
2,Gd,215860.714286,24,0.822199,14,10
4,TA,187489.836003,2604,89.208633,1311,1293
1,Fa,123573.354167,124,4.248030,48,76
3,Po,100166.666667,5,0.171292,3,2


In [339]:
num_nan = df['GarageQual'].isna().sum()
print(num_nan)

159


In [340]:
df.loc[(df['GarageCars'] == 0) & (df['GarageQual'].isna()), 'GarageQual'] = 'NoAplica'

In [341]:
num_nan = df['GarageQual'].isna().sum()
print(num_nan)

2


In [342]:
# Como es un valor de test no lo elimino

filas_nan = df[df['GarageQual'].isna()]

filas_nan[["SalePrice", "GarageType", "GarageYrBlt", "GarageFinish", "GarageCars", 
                            "GarageArea", "GarageQual", "GarageCond", "Dataset"]]

,SalePrice,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,Dataset
2126,NaN,35856.01668,1910.0,59168.479803,1.0,360.000000,NaN,NaN,test
2576,NaN,35856.01668,1923.0,59168.479803,2.0,519.432873,NaN,NaN,test


In [343]:
moda_GarageQual= df[df['Dataset'] == 'train']['GarageQual'].mode()[0]
print(moda_GarageQual)

df['GarageQual'].fillna(moda_GarageQual, inplace=True)

TA


In [344]:
num_nan = df['GarageQual'].isna().sum()
print(num_nan)

0


In [345]:
# cambio ex por gd ya que solo esta en train
df['GarageQual'] = df['GarageQual'].replace({'Ex': 'Gd'})

In [346]:
resultado = analizar_precio_viviendas_por_variable(df, 'GarageQual')
resultado

,GarageQual,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
1,Gd,220297.058824,27,0.924974,17,10
4,TA,187489.836003,2606,89.277150,1311,1295
0,Fa,123573.354167,124,4.248030,48,76
2,NoAplica,103317.283951,157,5.378554,81,76
3,Po,100166.666667,5,0.171292,3,2


In [347]:
df['GarageQual'] = df['GarageQual'].replace({'NoAplica': 'Po'})

In [348]:
# categorias = ['Gd', 'TA', 'Fa', 'Po', 'NoAplica']

# df = aplicar_codificacion_ordinal_especifica(df, 'GarageQual', categorias)


escala_calidad = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 0}
df['GarageQual'] = df['GarageQual'].map(escala_calidad)

65. [GarageCond](../docs/descripcion_variables.md#variable-garagecond): Condición del garaje.

In [349]:
resultado = analizar_precio_viviendas_por_variable(df, 'GarageCond')
resultado


,GarageCond,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
4,TA,187885.735294,2654,90.921548,1326,1328
2,Gd,179930.000000,15,0.513875,9,6
0,Ex,124000.000000,3,0.102775,2,1
1,Fa,114654.028571,74,2.535115,35,39
3,Po,108500.000000,14,0.479616,7,7


In [350]:
num_nan = df['GarageCond'].isna().sum()
print(num_nan)


159


In [351]:
df.loc[(df['GarageCars'] == 0) & (df['GarageCond'].isna()), 'GarageCond'] = 'NoAplica'

In [352]:
num_nan = df['GarageCond'].isna().sum()
print(num_nan)

2


In [353]:
# Como es un valor de test no lo elimino

filas_nan = df[df['GarageCond'].isna()]

filas_nan[["SalePrice", "GarageType", "GarageYrBlt", "GarageFinish", "GarageCars", 
                            "GarageArea", "GarageQual", "GarageCond", "Dataset"]]

,SalePrice,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,Dataset
2126,NaN,35856.01668,1910.0,59168.479803,1.0,360.000000,3,NaN,test
2576,NaN,35856.01668,1923.0,59168.479803,2.0,519.432873,3,NaN,test


In [354]:
# Calcular la moda de GarageCond en el DataFrame
moda_GarageCond= df[df['Dataset'] == 'train']['GarageCond'].mode()[0]
print(moda_GarageCond)
# Imputar los valores faltantes en GarageCond
df['GarageCond'].fillna(moda_GarageCond, inplace=True)


TA


In [355]:
num_nan = df['GarageCond'].isna().sum()
print(num_nan)

0


In [356]:
# categorias = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NoAplica']

# df = aplicar_codificacion_ordinal_especifica(df, 'GarageCond', categorias)


escala_calidad = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NoAplica': 0}
df['GarageCond'] = df['GarageCond'].map(escala_calidad)


66. [PavedDrive](../docs/descripcion_variables.md#variable-paveddrive): Entrada pavimentada.

In [357]:
df.loc[df['GarageCars'] == 0, 'PavedDrive'] = 'N'

In [358]:
resultado = analizar_precio_viviendas_por_variable(df, 'PavedDrive')
resultado


,PavedDrive,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
2,Y,189194.489559,2559,87.667009,1293,1266
1,P,135103.571429,59,2.021240,28,31
0,N,113191.158273,301,10.311751,139,162


In [359]:
num_nan = df['PavedDrive'].isna().sum()
print(num_nan)

0


In [360]:
categorias = ['Y', 'P', 'N']

df = aplicar_codificacion_ordinal_especifica(df, 'PavedDrive', categorias)


67. [WoodDeckSF](../docs/descripcion_variables.md#variable-wooddecksf): Área de la terraza de madera en pies cuadrados.

In [361]:
resultado = analizar_precio_viviendas_por_variable(df, 'WoodDeckSF')
resultado


,WoodDeckSF,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
290,361,625000.0,1,0.034258,1,0
301,382,603475.0,2,0.068517,2,0
27,52,582933.0,3,0.102775,1,2
351,503,538000.0,1,0.034258,1,0
86,126,446261.0,2,0.068517,1,1
...,...,...,...,...,...,...
369,657,NaN,1,0.034258,0,1
372,684,NaN,1,0.034258,0,1
373,690,NaN,1,0.034258,0,1
377,870,NaN,1,0.034258,0,1


In [362]:
num_nan = df['WoodDeckSF'].isna().sum()
print(num_nan)


0


68. [OpenPorchSF](../docs/descripcion_variables.md#variable-openporchsf): Área del porche abierto en pies cuadrados.

In [363]:
resultado = analizar_precio_viviendas_por_variable(df, 'OpenPorchSF')
resultado


,OpenPorchSF,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
58,67,611657.0,3,0.102775,1,2
216,260,475000.0,1,0.034258,1,0
195,229,465000.0,1,0.034258,1,0
223,274,451950.0,2,0.068517,1,1
150,170,428616.5,3,0.102775,2,1
...,...,...,...,...,...,...
242,382,NaN,1,0.034258,0,1
245,444,NaN,1,0.034258,0,1
246,484,NaN,1,0.034258,0,1
250,570,NaN,1,0.034258,0,1


In [364]:
num_nan = df['OpenPorchSF'].isna().sum()
print(num_nan)


0


69. [EnclosedPorch](../docs/descripcion_variables.md#variable-enclosedporch): Área del porche cerrado en pies cuadrados.

In [365]:
resultado = analizar_precio_viviendas_por_variable(df, 'EnclosedPorch')
resultado


,EnclosedPorch,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
15,37,438780.0,1,0.034258,1,0
166,291,430000.0,1,0.034258,1,0
94,162,335000.0,1,0.034258,1,0
102,174,287000.0,1,0.034258,1,0
177,386,265979.0,1,0.034258,1,0
...,...,...,...,...,...,...
176,368,NaN,1,0.034258,0,1
178,429,NaN,1,0.034258,0,1
179,432,NaN,1,0.034258,0,1
181,584,NaN,1,0.034258,0,1


In [366]:
num_nan = df['EnclosedPorch'].isna().sum()
print(num_nan)


0


70. [3SsnPorch](../docs/descripcion_variables.md#variable-3ssnporch): Área del porche de tres estaciones en pies cuadrados.

In [367]:
resultado = analizar_precio_viviendas_por_variable(df, '3SsnPorch')
resultado


,3SsnPorch,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
25,304,394617.000000,1,0.034258,1,0
9,153,392500.000000,3,0.102775,1,2
24,290,262500.000000,1,0.034258,1,0
11,168,238000.000000,3,0.102775,3,0
22,245,231500.000000,1,0.034258,1,0
16,196,228500.000000,1,0.034258,1,0
15,182,222000.000000,1,0.034258,1,0
7,144,211500.000000,2,0.068517,2,0
21,238,194500.000000,1,0.034258,1,0
17,216,184500.000000,2,0.068517,2,0


In [368]:
num_nan = df['3SsnPorch'].isna().sum()
print(num_nan)


0


71. [ScreenPorch](../docs/descripcion_variables.md#variable-screenporch): Área del porche con mosquitero en pies cuadrados.

In [369]:
resultado = analizar_precio_viviendas_por_variable(df, 'ScreenPorch')
resultado


,ScreenPorch,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
78,210,538000.0,3,0.102775,1,2
116,410,475000.0,1,0.034258,1,0
57,170,414950.0,2,0.068517,2,0
96,260,314813.0,1,0.034258,1,0
100,266,311872.0,2,0.068517,1,1
...,...,...,...,...,...,...
105,280,NaN,1,0.034258,0,1
111,342,NaN,1,0.034258,0,1
112,348,NaN,1,0.034258,0,1
119,490,NaN,1,0.034258,0,1


In [370]:
num_nan = df['ScreenPorch'].isna().sum()
print(num_nan)


0


72. [PoolArea](../docs/descripcion_variables.md#variable-poolarea): Área de la piscina en pies cuadrados.

In [371]:
describe_train_test(df, 'PoolArea')

,Train,Test
count,1460.000000,1459.000000
mean,2.758904,1.744345
std,40.177307,30.491646
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,738.000000,800.000000


In [372]:
resultado = analizar_precio_viviendas_por_variable(df, 'PoolArea')
resultado


,PoolArea,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
8,555,745000.000000,1,0.034258,1,0
12,738,274970.000000,1,0.034258,1,0
7,519,250000.000000,1,0.034258,1,0
6,512,235000.000000,1,0.034258,1,0
11,648,181000.000000,1,0.034258,1,0
0,0,180404.663455,2906,99.554642,1453,1453
10,576,171000.000000,1,0.034258,1,0
5,480,160000.000000,1,0.034258,1,0
1,144,NaN,1,0.034258,0,1
2,228,NaN,1,0.034258,0,1


In [373]:
num_nan = df['PoolArea'].isna().sum()
print(num_nan)


0


73. [PoolQC](../docs/descripcion_variables.md#variable-poolqc): Calidad de la piscina.

In [374]:
resultado = analizar_precio_viviendas_por_variable(df, 'PoolQC')
resultado


,PoolQC,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
0,Ex,490000.0,4,0.137033,2,2
1,Fa,215500.0,2,0.068517,2,0
2,Gd,201990.0,4,0.137033,3,1


In [375]:
df.loc[df['PoolArea'] == 0, 'PoolQC'] = 'No Aplica'

In [376]:
filas_nan = df[df['PoolQC'].isna()]
filas_nan[["PoolArea", "PoolQC", "Dataset"]]

,PoolArea,PoolQC,Dataset
2420,368,NaN,test
2503,444,NaN,test
2599,561,NaN,test


In [377]:
moda_PoolQC = df[(df['Dataset'] == 'train') & (df['PoolArea'] > 0)]['PoolQC'].mode()[0]
print(moda_PoolQC)

df['PoolQC'].fillna(moda_PoolQC, inplace=True)

Gd


In [378]:
num_nan = df['PoolQC'].isna().sum()
print(num_nan)

0


In [379]:
resultado = analizar_precio_viviendas_por_variable(df, 'PoolQC')
resultado

,PoolQC,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
0,Ex,490000.000000,4,0.137033,2,2
1,Fa,215500.000000,2,0.068517,2,0
2,Gd,201990.000000,7,0.239808,3,4
3,No Aplica,180404.663455,2906,99.554642,1453,1453


In [380]:
# df.drop(columns=['PoolQC'], inplace=True)

escala_calidad = {'Ex': 5, 'Gd': 4, 'Fa': 2, 'No Aplica': 0}
df['PoolQC'] = df['PoolQC'].map(escala_calidad)

In [381]:
df['TienePiscina'] = df['PoolArea'] > 0

74. [Fence](../docs/descripcion_variables.md#variable-fence): Calidad de la cerca.

In [382]:
resultado = analizar_precio_viviendas_por_variable(df, 'Fence')
resultado


,Fence,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
0,GdPrv,178927.457627,118,4.042480,59,59
2,MnPrv,148751.089172,329,11.270983,157,172
1,GdWo,140379.314815,112,3.836930,54,58
3,MnWw,134286.363636,12,0.411100,11,1


In [383]:
num_nan = df['Fence'].isna().sum()
print(num_nan)


2348


In [384]:
total_filas = df.shape[0]
porcentaje_nan_fence = (num_nan / total_filas) * 100

print(f"Porcentaje de NaN en 'Fence': {porcentaje_nan_fence:.2f}%")

Porcentaje de NaN en 'Fence': 80.44%


In [385]:
# de momento sustituyo los nan por no procede, para analizar la relacion de fence con otras variables, antes de eliminarla
df['Fence'].fillna('Desconocida', inplace=True)

In [386]:
resultado = analizar_precio_viviendas_por_variable(df, 'Fence')
resultado

,Fence,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
0,Desconocida,187596.837998,2348,80.438506,1179,1169
1,GdPrv,178927.457627,118,4.042480,59,59
3,MnPrv,148751.089172,329,11.270983,157,172
2,GdWo,140379.314815,112,3.836930,54,58
4,MnWw,134286.363636,12,0.411100,11,1


In [387]:
categorias = ['GdPrv', 'MnPrv', 'GdWo', 'MnWw', 'Desconocida']


df = aplicar_codificacion_ordinal_especifica(df, 'Fence', categorias)

In [388]:
# # Introduzco las columnas sobrantes después del análisis de las características 8
columnas_a_analizar = ["SalePrice", "Fireplaces", 'HasFireplaces', "FireplaceQu", "GarageType", 
                       "GarageYrBlt", "GarageFinish", "GarageCars", 
                        "GarageArea", "GarageQual", "GarageCond", "PavedDrive", "WoodDeckSF", "OpenPorchSF", 
                        "EnclosedPorch", "3SsnPorch", "ScreenPorch", "PoolArea", "PoolQC", "TienePiscina",
                        "Fence"]

visualizar_correlaciones_grandes(df, columnas_a_analizar)


In [389]:
# # Introduzco las columnas sobrantes después del análisis de las características 8
columnas_a_analizar = ["SalePrice", "Fireplaces", 'HasFireplaces', "FireplaceQu", "GarageType", 
                       "GarageYrBlt", "GarageFinish", "GarageCars", 
                        "GarageArea", "GarageQual", "GarageCond", "PavedDrive", "WoodDeckSF", "OpenPorchSF", 
                        "EnclosedPorch", "3SsnPorch", "ScreenPorch", "PoolArea", "Fence"]

visualizar_correlaciones_grandes(df, columnas_a_analizar)

### 9. **Características Misceláneas y de Venta**
> **Objetivo**: Analizar cómo las características misceláneas y la temporalidad de la venta afectan el precio.

75. [MiscFeature](../docs/descripcion_variables.md#variable-miscfeature): Característica miscelánea no cubierta en otras categorías.
76. [MiscVal](../docs/descripcion_variables.md#variable-miscval): Valor en dólares de la característica miscelánea.
77. [MoSold](../docs/descripcion_variables.md#variable-mosold): Mes de venta.
78. [YrSold](../docs/descripcion_variables.md#variable-yrsold): Año de venta.
79. [SaleType](../docs/descripcion_variables.md#variable-saletype): Tipo de venta.
80. [SaleCondition](../docs/descripcion_variables.md#variable-salecondition): Condición de la venta.

In [390]:
caracteristicas_venta = ['MiscFeature', 'MiscVal', 'MoSold', 'YrSold', 'SaleType', 'SaleCondition']

df[["SalePrice"] + caracteristicas_venta].head()

,SalePrice,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,208500.0,NaN,0,2,2008,WD,Normal
1,181500.0,NaN,0,5,2007,WD,Normal
2,223500.0,NaN,0,9,2008,WD,Normal
3,140000.0,NaN,0,2,2006,WD,Abnorml
4,250000.0,NaN,0,12,2008,WD,Normal


In [391]:
resumen = resumen_columnas(df, caracteristicas_venta)
resumen

,Tipo de Dato,Nº Valores Únicos,Nº de NaN,Porcentaje NaN Train,Porcentaje NaN Test,Valores Únicos Train,Valores Únicos Test
MiscFeature,object,4,2814,96.30137,96.504455,[TenC],[]
MiscVal,int64,38,0,0.00000,0.000000,[],[]
MoSold,int64,12,0,0.00000,0.000000,[],[]
YrSold,int64,5,0,0.00000,0.000000,[],[]
SaleType,object,9,1,0.00000,0.068540,[],[]
SaleCondition,object,6,0,0.00000,0.000000,[],[]


In [392]:
valores_unicos = obtener_valores_unicos(df, caracteristicas_venta)

for key, value in valores_unicos.items():
    print(f"'{key}': {value},")

'MiscFeature': ['Gar2', 'Othr', 'Shed', 'TenC'],
'MiscVal': [0, 54, 80, 300, 350, 400, 420, 450, 455, 460, 480, 490, 500, 560, 600, 620, 650, 700, 750, 800, 900, 1000, 1150, 1200, 1300, 1400, 1500, 1512, 2000, 2500, 3000, 3500, 4500, 6500, 8300, 12500, 15500, 17000],
'MoSold': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
'YrSold': [2006, 2007, 2008, 2009, 2010],
'SaleType': ['COD', 'CWD', 'Con', 'ConLD', 'ConLI', 'ConLw', 'New', 'Oth', 'WD'],
'SaleCondition': ['Abnorml', 'AdjLand', 'Alloca', 'Family', 'Normal', 'Partial'],


75. [MiscFeature](../docs/descripcion_variables.md#variable-miscfeature): Característica miscelánea no cubierta en otras categorías.

- La única información relevante que proporciona esta variable es la presencia de un cobertizo en la propiedad (`Shed`). Las demás categorías (`Gar2`, `TenC`, `Othr`) son demasiado infrecuentes para ser útiles en el análisis, y la información sobre un segundo garaje (`Gar2`) ya está cubierta por otras columnas específicas de garaje en el dataset. Por lo tanto, la columna `MiscFeature` se transformará en una nueva variable binaria llamada `HasShed`, que indicará si la casa cuenta con un cobertizo (1) o no (0) y borro `MiscFeature`.

- Igualmente, dado que esta variable no presenta una correlación significativa con la variable objetivo, consideraré fusionarla con otra variable que represente elementos adicionales de la propiedad.

In [393]:
df['MiscFeature'].fillna('SinExtras', inplace=True)


In [394]:
resultado = analizar_precio_viviendas_por_variable(df, 'MiscFeature')
resultado

,MiscFeature,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
4,TenC,250000.000000,1,0.034258,1,0
3,SinExtras,182046.410384,2814,96.402878,1406,1408
0,Gar2,170750.000000,5,0.171292,2,3
2,Shed,151187.612245,95,3.254539,49,46
1,Othr,94000.000000,4,0.137033,2,2


### Decisión sobre `MiscFeature`

El modelo empeora al utilizar `get_dummies` y `codificación_ponderada` si se mantiene la variable `MiscFeature`. Por lo tanto, se decide transformar esta variable para intentar que sea más útil, aunque probablemente se acabe eliminando más adelante.

Se intentó combinar las variables con y sin extras como `cobertizo`, `garaje doble`, `otro` o `cancha`. Muestra más correlación con `MiscVal`, como es lógico, que convertida en `HasShed`, pero empeora el modelo. Aunque se observó una pequeña mejora en **RMSLE**, el resto de las métricas como **R²**, **MAE**, **RMSE**, y **MSE** empeoraron ligeramente. 

Por lo tanto, se opta por cambiar la variable a una representación más simple: si tiene o no cobertizo, denominada `HasShed`, y se decide eliminar la variable `MiscFeature` y el resto de valores que no implican tener cobertizo.

In [395]:
df['HasShed'] = df['MiscFeature'].apply(lambda x: 1 if x == 'Shed' else 0)

resultado = analizar_precio_viviendas_por_variable(df, 'HasShed')
resultado

,HasShed,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
0,0,181953.758327,2824,96.745461,1411,1413
1,1,151187.612245,95,3.254539,49,46


In [396]:
df.drop('MiscFeature', axis=1, inplace=True)

76. [MiscVal](../docs/descripcion_variables.md#variable-miscval): Valor en dólares de la característica miscelánea.

- **Correlación entre `MiscVal` y `HasShed`**: 
  - Se observa una correlación positiva débil de **0.230508** entre `MiscVal` y `HasShed`. Esto sugiere que las propiedades que tienen un cobertizo (`HasShed = 1`) tienden a tener un valor misceláneo más alto. Sin embargo, la relación no es lo suficientemente fuerte como para ser considerada significativa en el análisis.

- **Correlación entre `MiscVal` y `SalePrice`**: 
  - La correlación entre `MiscVal` y `SalePrice` es de **-0.02119**, indicando una relación muy débil y negativa. Esto sugiere que el valor misceláneo no tiene una influencia significativa en el precio de venta de las viviendas. 

### Decisión

- **Omitir `MiscVal`**: Dado que la correlación con `SalePrice` es casi nula, ya que probablemente no aportará información útil al modelo predictivo.

In [397]:
correlacion = df[['MiscVal', 'SalePrice', 'HasShed']].corr()
correlacion

,MiscVal,SalePrice,HasShed
MiscVal,1.000000,-0.021190,0.230508
SalePrice,-0.021190,1.000000,-0.069771
HasShed,0.230508,-0.069771,1.000000


Se prueba a borrar la columna, el modelo ha tenido una mejora notable manteniendo y no borrando la columna MiscVal, por lo que se deja. La mejora en R² es especialmente destacable, ya que muestra una mayor capacidad del modelo para explicar la variabilidad de los datos.

77. [MoSold](../docs/descripcion_variables.md#variable-mosold): Mes de venta.

In [398]:
resultado = analizar_precio_viviendas_por_variable(df, 'MoSold')
resultado

,MoSold,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
8,9,195683.206349,158,5.412813,63,95
10,11,192210.911392,142,4.864680,79,63
11,12,186518.966102,104,3.562864,59,45
6,7,186331.192308,446,15.279205,234,212
7,8,184651.827869,233,7.982186,122,111
0,1,183256.258621,122,4.179514,58,64
2,3,183253.924528,232,7.947927,106,126
9,10,179563.977528,173,5.926687,89,84
1,2,177882.000000,133,4.556355,52,81
5,6,177395.735178,503,17.231929,253,250


### Estacionalidad de Ventas vs. Precio
Los precios de las viviendas están influenciados por factores como la oferta, la demanda, la ubicación y las características de las propiedades, más que por la estacionalidad. Durante los picos de ventas (como junio o julio), los precios pueden ser más bajos debido al aumento en el volumen de propiedades disponibles, mientras que en meses con menos ventas, los precios podrían ser más altos debido a una oferta limitada.

### Correlación baja entre MoSold y SalePrice
Si el análisis de correlación muestra una relación baja entre `MoSold` (mes de venta) y `SalePrice` (precio de venta), esto sugiere que el mes de venta no tiene un impacto directo significativo sobre el precio de las viviendas. La estacionalidad afecta principalmente al número de ventas, pero no necesariamente a los precios.

In [399]:
conteo_mosold = df['MoSold'].value_counts().sort_index()

fig = px.bar(
    x=conteo_mosold.index, 
    y=conteo_mosold.values, 
    labels={'x': 'Mes de Venta (MoSold)', 'y': 'Número de Ventas'},
    title='Conteo de Ventas por Mes (MoSold)',
)

fig.update_layout(
    title_x=0.5,
    height=600,
    xaxis=dict(tickmode='array', tickvals=conteo_mosold.index),
)

fig.show()

Se realizaron varias pruebas para procesar la variable `MoSold` y capturar las variaciones estacionales en los datos.

En primer lugar, se mantuvo la variable en 12 columnas, una por cada mes. Aunque esta codificación permitió representar los meses de manera independiente, no captó la naturaleza cíclica de la variable, lo que dificultó que el modelo entendiera las relaciones entre meses consecutivos. Como resultado, la importancia de esta variable en el modelo fue baja.

Posteriormente, se utilizó una **codificación por estaciones** (invierno, primavera, verano, otoño), agrupando los meses según su estación. Este enfoque ayudó a capturar cierta estacionalidad, pero redujo la precisión al eliminar la diferenciación entre los meses dentro de una misma estación.

Finalmente, se implementó una **codificación cíclica** utilizando funciones trigonométricas (**seno** y **coseno**) para representar la continuidad entre los meses. Este método permitió al modelo entender mejor la proximidad y las transiciones entre meses consecutivos, mejorando la interpretación de las variaciones estacionales. 

Además, se probó una **codificación ponderada**, que asigna valores numéricos en función de su relación directa con la variable objetivo. Este enfoque resultó ser el más efectivo, ya que aprovechó mejor la relación entre `MoSold` y la variable objetivo.

In [400]:
codificacion_ponderada(df, 'MoSold', 'SalePrice')

78. [YrSold](../docs/descripcion_variables.md#variable-yrsold): Año de venta.

Se detectaron **incoherencias** en los datos de `YrSold` que fueron solventadas anteriormente.

In [401]:
resultado = analizar_precio_viviendas_por_variable(df, 'YrSold')
resultado

,YrSold,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
1,2007,186063.151976,691,23.672491,329,362
0,2006,182549.458599,619,21.205892,314,305
3,2009,179432.103550,648,22.199383,338,310
4,2010,177393.674286,339,11.613566,175,164
2,2008,177360.838816,622,21.308667,304,318


In [402]:
sorted(df["YrSold"].unique())

[2006, 2007, 2008, 2009, 2010]

In [403]:
# Gráfico de caja
fig = px.box(df, x='YrSold', y='SalePrice', 
             title='Distribución del Precio de Venta por Año de Venta',
             labels={'YrSold': 'Año de Venta', 'SalePrice': 'Precio de Venta'},
             points="all")  
fig.show()


In [404]:
# Agrupar y calcular la suma acumulada de YrSold por año
cumsum_by_year = df.groupby('YrSold').size().cumsum()

# Crear el gráfico de línea usando los años como eje x y la suma acumulada en y
fig = px.line(x=cumsum_by_year.index, y=cumsum_by_year.values, title='Suma acumulada de YrSold por Año')

# Configuración del layout
fig.update_layout(
    xaxis_title='Año',
    yaxis_title='Suma Acumulada de YrSold',
    title_x=0.5
)

# Mostrar el gráfico
fig.show()


### Creación de la variable Antigüedad

In [405]:
df['Antiguedad'] = df['YrSold'] - df['YearBuilt']

In [406]:
df['Antiguedad'].sort_values().unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 117, 118,
       119, 120, 122, 125, 126, 127, 128, 129, 135, 136], dtype=int64)

In [407]:
resultado = analizar_precio_viviendas_por_variable(df, 'Antiguedad')
resultado

,Antiguedad,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
113,114,475000.0,2,0.068517,1,1
114,115,325000.0,1,0.034258,1,0
123,128,295000.0,2,0.068517,1,1
11,11,271389.0,47,1.610140,22,25
75,75,267500.0,3,0.102775,2,1
...,...,...,...,...,...,...
101,101,NaN,1,0.034258,0,1
111,112,NaN,2,0.068517,0,2
112,113,NaN,1,0.034258,0,1
116,118,NaN,2,0.068517,0,2


In [408]:
# df = df.drop("YrSold", axis=1)

79. [SaleType](../docs/descripcion_variables.md#variable-saletype): Tipo de venta.

In [409]:
resultado = analizar_precio_viviendas_por_variable(df, 'SaleType')
resultado

,SaleType,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
6,New,274945.418033,239,8.187736,122,117
2,Con,269600.000000,5,0.171292,2,3
1,CWD,210600.000000,12,0.411100,4,8
4,ConLI,200390.000000,9,0.308325,5,4
8,WD,173401.836622,2525,86.502227,1267,1258
0,COD,143973.255814,87,2.980473,43,44
5,ConLw,143700.000000,8,0.274066,5,3
3,ConLD,138780.888889,26,0.890716,9,17
7,Oth,119850.000000,7,0.239808,3,4


In [410]:
# Al haber solo 1 valor faltante en esta columna, aplico la moda
num_nan = df['SaleType'].isna().sum()
print(num_nan)
moda_SaleType= df[df['Dataset'] == 'train']['SaleType'].mode()[0]
print(moda_SaleType)
df['SaleType'].fillna(moda_SaleType, inplace=True)

1
WD


In [411]:
resultado = analizar_precio_viviendas_por_variable(df, 'SaleType')
resultado

,SaleType,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
6,New,274945.418033,239,8.187736,122,117
2,Con,269600.000000,5,0.171292,2,3
1,CWD,210600.000000,12,0.411100,4,8
4,ConLI,200390.000000,9,0.308325,5,4
8,WD,173401.836622,2526,86.536485,1267,1259
0,COD,143973.255814,87,2.980473,43,44
5,ConLw,143700.000000,8,0.274066,5,3
3,ConLD,138780.888889,26,0.890716,9,17
7,Oth,119850.000000,7,0.239808,3,4


In [412]:
codificacion_ponderada(df, 'SaleType', 'SalePrice')

80. [SaleCondition](../docs/descripcion_variables.md#variable-salecondition): Condición de la venta.

In [413]:
resultado = analizar_precio_viviendas_por_variable(df, 'SaleCondition')
resultado

,SaleCondition,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
5,Partial,272291.752000,245,8.393285,125,120
4,Normal,175202.219533,2402,82.288455,1198,1204
2,Alloca,167377.416667,24,0.822199,12,12
3,Family,149600.000000,46,1.575882,20,26
0,Abnorml,146526.623762,190,6.509078,101,89
1,AdjLand,104125.000000,12,0.411100,4,8


In [414]:
mapeo= {
    'Abnorml': 'Venta anormal',
    'Family': 'Venta anormal'
}

df['SaleCondition'] = df['SaleCondition'].replace(mapeo)

In [415]:
resultado = analizar_precio_viviendas_por_variable(df, 'SaleCondition')
resultado

,SaleCondition,Precio_Promedio,Num_Viviendas,Porcentaje_Viviendas,Num_V_Train,Num_V_Test
3,Partial,272291.752000,245,8.393285,125,120
2,Normal,175202.219533,2402,82.288455,1198,1204
1,Alloca,167377.416667,24,0.822199,12,12
4,Venta anormal,147034.619835,236,8.084961,121,115
0,AdjLand,104125.000000,12,0.411100,4,8


In [416]:
codificacion_ponderada(df, 'SaleCondition', 'SalePrice')

In [417]:
columnas_a_analizar = ['HasShed', 'Antiguedad', 'SaleType', 'SaleCondition', 'MoSold', 'YrSold']

visualizar_correlaciones(df, columnas_a_analizar)

In [418]:
mapeo_train_test = {
    'train': 1,
    'test': 2
}

df['Dataset'] = df['Dataset'].map(mapeo_train_test)

## Guardado del DataFrame Limpio

Después de realizar el primer análisis exploratorio de datos y realizar las operaciones de limpieza necesarias, se procede a almacenar el DataFrame limpio, el cual está listo para la segunda fase de transformaciones.

In [419]:
df.to_pickle('../data/Inmobiliaria_Horizonte_limpio.pkl')

In [420]:
nombres_columnas_nan = df.columns[df.isnull().any()].tolist()
print(nombres_columnas_nan)

['SalePrice']
